# Towards Automatic Testing of Reference Point Based Interactive Methods

http://link.springer.com/chapter/10.1007%2F978-3-319-45823-6_45


# Reference point NSGAII

In [12]:
from random import random
from joblib import Parallel, delayed
from scipy import spatial,stats
from scipy.optimize import differential_evolution
import numpy as np
import pyMOEA
reload(pyMOEA)
from collections import defaultdict
import itertools                               
import operator
def make_run(problem,ref,nf=3,runs=10,evals=10000):
    nsgaii=[]
    ach=[]
    dist=[]
    best=[0,0]
    for f in range(runs):
        nsgaii.append(rNSGAII_solution(problem,ref,nf=nf,evals=evals))
        nd=spatial.distance.euclidean(ref,nsgaii[-1])

        ach.append(ACH_solution(problem,ref,nf=nf))
        ad=spatial.distance.euclidean(ref,ach[-1])
        dist.append((ad,nd))
        if ad<nd:
            best[0]+=1

        else:
            best[1]+=1
    return (best,ach,nsgaii,dist)

def random_pref(nf=3):
    aspir=[0.0]*nf
    w=[None]*nf
    for k in range(nf):
        w[k]=random()
        if random()<.5:
            aspir[k]=random()
    return aspir,w

def agent_run(nf=3,jobs=8,evals=20000,runs=10,single_pref=True,
              methods=[pyMOEA.ACH_solution,pyMOEA.rNSGAII_solution],
              problems=['DTLZ1','DTLZ2','DTLZ3','DTLZ4']):
    ideal=[0.0]*nf
    nadir=[1.0]*nf
    bounds=(ideal,nadir)
    pref_list=[]

    if single_pref:
        pref_list=[random_pref(nf)]*runs
    else:
        pref_list=[random_pref(nf) for i in range(runs)]
    results=defaultdict(list)
    for r in range(runs):
        res=Parallel(n_jobs=jobs)(
           delayed(pyMOEA.agent_paraller)(target,pref_list[r],evals=evals) 
                for target in itertools.product(methods,problems)         
        )
        for r in res:
            results[r[2]].append((r[0],r[1],r[3]))
            #objs[-1],preference,(objs,refs)


    return results



def print_results(results,POs=None,evals=20000):
    
    if POs is None:
        POs={}
    full_dist=[]
    for nf,method,problem in sorted(results.keys(),key=operator.itemgetter(1,0,2)):
        print nf,problem,method
        nf = int(nf)
        dist=[]
        for res in results[(nf,method,problem)]:
            obj=res[0]
            ref=tuple(res[1][0])
            if POs.has_key((nf,problem,ref)):
                PO=POs[nf,problem,ref]
            else:
                print "Projection not available"
                #PO = pyMOEA.ACH_solution(problem,ref,evals=evals)
                #POs[nf,problem,ref]=PO
                continue
            dist.append(spatial.distance.euclidean(obj,PO))
        mean, sigma = np.mean(dist), np.std(dist)
        print np.mean(dist),np.std(dist),np.min(dist),stats.norm.interval(0.68, loc=mean, scale=sigma / np.sqrt(len(dist)))
    return PO

def prj_ref(results,PO=None,evals=20000):
    proj_refs=[]
    if PO is None:
        PO={}
    for nf,method,problem in sorted(results.keys(),key=operator.itemgetter(0,1,2)):
        #if nf==4 or problem=="DTLZ4":
        #    continue
        dist=[]
        for res in results[(nf,method,problem)]:
            obj=res[0]
            ref=tuple(res[1][0])
            if not POs.has_key((nf,problem,ref)):
                print (nf,problem,ref)
                proj_refs.append((nf,problem,ref))
    res=Parallel(n_jobs=8)(
           delayed(pyMOEA.proj_ref)(job[0],job[1],job[2],evals=evals) 
                for job in proj_refs        
        )
    for r in res:
        PO[tuple(r[:3])]=r[3]
    return PO
    

Agent tests
==
Run the agent tests and save the result dict to results.txt file

result.txt
===
    key: (nf,method,problem)
    value: List of 
Problems
===

DTLZ1
DTLZ2
DTLZ3
DTLZ5



In [1]:
for i in range(4):
    for nf in [2,3,4]:
        res=agent_run(nf=nf,runs=1,single_pref=False)
        for k in res.keys():
            results[(nf,k[0].__name__,k[1])].extend(res[k])
        print "======================================================================="
        print nf
        %store results

NameError: name 'agent_run' is not defined

In [ ]:
# Save current results to results.txt
%store results > results.txt

In [22]:
# %load POs.txt
POs={(2, 'DTLZ1', (0.0, 0.0)): [0.9996202295774317, 0.999620247411252],
 (2, 'DTLZ1', (0.5389452641563056, 0.0)): [0.7693460434444864,
                                           0.2304007853878284],
 (2, 'DTLZ1', (0.5858121009812931, 0.0)): [1.0426530846225495,
                                           0.45684098391572636],
 (2, 'DTLZ1', (0.8233781347997915, 0.0)): [1.6611827132258348,
                                           0.8378045895518204],
 (2, 'DTLZ2', (0.0, 0.0)): [0.711152421836636, 0.7111524202065502],
 (2, 'DTLZ2', (0.5389452641563056, 0.0)): [0.9232191471779909,
                                           0.3842738690370552],
 (2, 'DTLZ2', (0.5858121009812931, 0.0)): [0.9364944662355161,
                                           0.3506823570517402],
 (2, 'DTLZ2', (0.8233781347997915, 0.0)): [0.9879163787777763,
                                           0.16453822895069764],
 (2, 'DTLZ3', (0.0, 0.0)): [2.827353169668456, 2.8273531548130486],
 (2, 'DTLZ3', (0.5389452641563056, 0.0)): [1.6574106539115356,
                                           1.1184653730011365],
 (2, 'DTLZ3', (0.5858121009812931, 0.0)): [3.105045999477425,
                                           2.5192338847303692],
 (2, 'DTLZ3', (0.8233781347997915, 0.0)): [3.9217300040722023,
                                           3.0983518276413724],
 (2, 'DTLZ4', (0.0, 0.0)): [0.7106808418207432, 0.7106804684467795],
 (2, 'DTLZ4', (0.5389452641563056, 0.0)): [0.9245186412162422,
                                           0.38557310365703024],
 (2, 'DTLZ4', (0.5858121009812931, 0.0)): [0.9382105691804341,
                                           0.35239822673891047],
 (2, 'DTLZ4', (0.8233781347997915, 0.0)): [0.9873778901098069,
                                           0.1639997152530594],
 (3, 'DTLZ1', (0.0, 0.0, 0.0)): [0.33324894211065503,
                                 0.33324894837542496,
                                 0.33324894109088377],
 (3, 'DTLZ1', (0.0, 0.0, 0.9829301429354526)): [0.17218804562608803,
                                                0.17218803613710965,
                                                1.1551181906384043],
 (3, 'DTLZ1', (0.4981520234633334, 0.0, 0.6987592904740502)): [0.4802917966071218,
                                                               4.802922769132099e-07,
                                                               0.5194545482915622],
 (3, 'DTLZ1', (0.6635589568190845, 0.5338175168551759, 0.0)): [0.4776727045971085,
                                                               0.522072866255858,
                                                               1.2546357118376136e-06],
 (3, 'DTLZ2', (0.0, 0.0, 0.0)): [0.5809912481255258,
                                 0.5810283907928809,
                                 0.5810283833706297],
 (3, 'DTLZ2', (0.0, 0.0, 0.9829301429354526)): [0.016879798792547206,
                                                0.00922297782592784,
                                                0.9998159124875309],
 (3, 'DTLZ2', (0.4981520234633334, 0.0, 0.6987592904740502)): [0.5972363964292842,
                                                               0.0900916744140716,
                                                               0.7978436621048797],
 (3, 'DTLZ2', (0.6635589568190845, 0.5338175168551759, 0.0)): [0.7664224375956328,
                                                               0.636680989813231,
                                                               0.09797362809485514],
 (3, 'DTLZ3', (0.0, 0.0, 0.0)): [1.7314662408543526,
                                 1.7314659227174545,
                                 1.7314662348027547],
 (3, 'DTLZ3', (0.0, 0.0, 0.9829301429354526)): [3.6853926994971618,
                                                3.685392637721368,
                                                4.668322791607256],
 (3, 'DTLZ3', (0.4981520234633334, 0.0, 0.6987592904740502)): [1.215587602967616,
                                                               0.717435565944194,
                                                               1.4161948489323364],
 (3, 'DTLZ3', (0.6635589568190845, 0.5338175168551759, 0.0)): [3.7184274319191624,
                                                               3.5886859918917793,
                                                               3.053293701745042],
 (3, 'DTLZ4', (0.0, 0.0, 0.0)): [0.5808875346599789,
                                 0.5808870071870679,
                                 0.5808871472662809],
 (3, 'DTLZ4', (0.0, 0.0, 0.9829301429354526)): [0.016610097028522456,
                                                2.7619724309074816e-23,
                                                0.9999221255571907],
 (3, 'DTLZ4', (0.4981520234633334, 0.0, 0.6987592904740502)): [0.6003517748750212,
                                                               1.6177400414746182e-44,
                                                               0.8009584998129857],
 (3, 'DTLZ4', (0.6635589568190845, 0.5338175168551759, 0.0)): [0.7684034324856207,
                                                               0.6386619863742377,
                                                               0.094131719925676]}
%store POs

Stored 'POs' (dict)


In [54]:
results={(2, 'rNSGAII_solution', 'DTLZ3'): [([4.818625645332685, 1.7452724793238283], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[4.818625645332685, 1.7452724793238283]], [[0.5858121009812931, 0.0]])), ([3.4068899599421663, 2.3840987081526301], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[3.4068899599421663, 2.3840987081526301]], [[0.8233781347997915, 0.0]])), ([1.2044390057137151, 1.6711615065206951], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[2.7335801500649848, 1.41694463871976], [1.2044390057137151, 1.6711615065206951]], [[0.0, 0.0], [1.0, 0.70847231935987998]])), ([5.3503972268326354, 0.053729431047934498], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[3.0951164558429785, 0.6703568755216992], [2.0260119220915138, 0.20557811614585983], [5.3503972268326354, 0.053729431047934498]], [[0.5389452641563056, 0.0], [1.0, 0.6703568755216992], [1.0, 0.20557811614585983]])), ([2.4267476260087517, 1.8961258075895178], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.00078935330047023757, 2.0453444843549522], [0.00024415053350377263, 4.1105347025746113], [2.4267476260087517, 1.8961258075895178]], [[0.0, 0.0], [0.00039467665023511879, 1.0], [0.00012207526675188631, 1.0]])), ([1.3008838897908459, 2.8973464232695618], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[1.3008838897908459, 2.8973464232695618]], [[0.0, 0.0]])), ([4.3789804821176981, 2.5646968203393463], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[4.3789804821176981, 2.5646968203393463]], [[0.8521054652075405, 0.5432635491282871]])), ([0.00091843232353661661, 2.9284190148688656], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[1.0069293729978777, 0.27751145308492442], [0.00091843232353661661, 2.9284190148688656]], [[0.2295261844565868, 0.17176071551877525], [1.0, 0.22463608430184984]])), ([3.9013435469888987e-06, 2.056067772370668], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[1.7451678756136932, 2.5231637991977403], [0.03320646706072173, 2.0508005670552225], [0.032380517954174126, 2.154123160899506], [0.17764909186850569, 2.4449734299827548], [0.95769954840471694, 0.68283906636201031], [1.1849274287200176, 3.2251719572927553], [0.0050183960111047071, 1.1147495707071371], [0.39284617428080382, 3.055332018558877], [0.0074869187984282198, 2.1596338694719837], [3.9013435469888987e-06, 2.056067772370668]], [[0.0, 0.9893548997434237], [0.87258393780684662, 1.0], [0.016603233530360865, 1.0], [0.016190258977087063, 1.0], [0.088824545934252847, 1.0], [0.47884977420235847, 1.0], [0.59246371436000878, 0.68283906636201031], [0.0025091980055523535, 1.0], [0.19642308714040191, 1.0], [0.0037434593992141099, 1.0], [1.9506717734944494e-06, 1.0]])), ([1.0135989916373791, 1.1328118355296455], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.45537609963886955, 4.0101559205438768], [1.0135989916373791, 1.1328118355296455]], [[0.21536704667589168, 0.875015209332337], [0.45537609963886955, 1.0]]))], (4, 'rNSGAII_solution', 'DTLZ2'): [([0.34063539062998888, 1.0408705851798509e-09, 0.9140709855077721, 0.22087218668409736], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.35992905203292924, 9.6601548386173308e-10, 0.91262425453748997, 0.20160708845809044], [0.33998884228452925, 1.0248532785204337e-10, 0.87965221421797213, 0.33267862953588273], [0.30148445771105564, 6.1112905716573097e-11, 0.92694636684225973, 0.23612874137699078], [0.25781133862541855, 6.1787506618307707e-11, 0.90367893402278321, 0.34198372800445986], [0.23053230848529749, 6.7891302289579676e-09, 0.91075655731000127, 0.34267801528528352], [0.38882265685537859, 1.2437102050070509e-09, 0.90896328196448095, 0.18365527155683753], [0.22808458766340692, 5.9712491772381902e-08, 0.94585317822663395, 0.24200520355519198], [0.36356250640109122, 2.5557164023658532e-09, 0.89597318152533223, 0.25507302804347481], [0.35973452816480411, 4.6180994995668396e-08, 0.89720745705808713, 0.25624390471173825], [0.34063539062998888, 1.0408705851798509e-09, 0.9140709855077721, 0.22087218668409736]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.35992905203292924, 4.8300774193086654e-10, 0.80315868559819903, 0.20160708845809044], [0.33998884228452925, 5.1242663926021686e-11, 0.78667266543844017, 0.33267862953588273], [0.35992905203292924, 3.0556452858286548e-11, 0.81031974175058386, 0.33267862953588273], [0.33998884228452925, 3.0893753309153853e-11, 0.79868602534084565, 0.33267862953588273], [0.33998884228452925, 3.3945651144789838e-09, 0.80222483698445468, 0.33267862953588273], [0.38882265685537859, 6.2185510250352546e-10, 0.80132819931169452, 0.18365527155683753], [0.33998884228452925, 2.9856245886190951e-08, 0.81977314744277097, 0.33267862953588273], [0.33998884228452925, 1.2778582011829266e-09, 0.79483314909212011, 0.33267862953588273], [0.33998884228452925, 2.3090497497834198e-08, 0.79545028685849761, 0.33267862953588273], [0.38882265685537859, 5.2043529258992547e-10, 0.8038820510833401, 0.33267862953588273]])), ([0.3774027049742662, 0.41054477171236237, 0.53928571118021162, 0.63103964283247727], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.39013918299634232, 0.42499218473384642, 0.55814279085056495, 0.59636857658595377], [0.36364453424468157, 0.40252571830277006, 0.59690418427086545, 0.59550860486051305], [0.36491523039788337, 0.39583604026543528, 0.54479273544775675, 0.64294717272821877], [0.36003241940671932, 0.38681578712950548, 0.56115875308053764, 0.63709321089568638], [0.38669575198949396, 0.40118905369913105, 0.60225349988963961, 0.57171187466353202], [0.39728061439577211, 0.43347132216227457, 0.54137091773147894, 0.60099767244538105], [0.34086644411108125, 0.37321304099701008, 0.56440970199183615, 0.65273216489144459], [0.3558820231020664, 0.37817392888111134, 0.55227733973082938, 0.65217817101313325], [0.37356820717748185, 0.41404501912861946, 0.60252420487325975, 0.57102487615425224], [0.3774027049742662, 0.41054477171236237, 0.53928571118021162, 0.63103964283247727]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.39013918299634232, 0.35825705370774108, 0.55814279085056495, 0.59636857658595377], [0.36364453424468157, 0.34702382049220293, 0.59690418427086545, 0.59550860486051305], [0.36491523039788337, 0.34367898147353554, 0.59690418427086545, 0.59550860486051305], [0.36364453424468157, 0.33916885490557064, 0.54479273544775675, 0.64294717272821877], [0.36491523039788337, 0.34635548819038342, 0.56115875308053764, 0.59550860486051305], [0.36491523039788337, 0.36249662242195518, 0.60225349988963961, 0.57171187466353202], [0.38669575198949396, 0.33236748183932296, 0.56440970199183615, 0.57171187466353202], [0.38669575198949396, 0.40118905369913105, 0.60225349988963961, 0.57171187466353202], [0.39728061439577211, 0.41404501912861946, 0.60252420487325975, 0.57102487615425224], [0.3774027049742662, 0.35103334719699908, 0.56440970199183615, 0.65273216489144459]])), ([0.3716693043802558, 0.24816425461333205, 0.72974536954833225, 0.61228513876937507], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.4148476368534661, 0.25084101186803481, 0.72401774693717746, 0.61967716954034968], [0.3958884485772845, 0.28146912375728461, 0.69959623080972999, 0.57938951211139122], [0.38363816383645405, 0.21068742769665061, 0.7537550712400144, 0.60551746168436715], [0.4098603260091222, 0.2977848638490812, 0.71822469381960741, 0.58105244847136395], [0.3654729386755568, 0.28390275747776822, 0.71652120126353402, 0.62146262701471677], [0.40176967731185748, 0.25885267612815055, 0.72786421022587611, 0.58496691299820847], [0.39459566231976118, 0.21177586044226321, 0.71952599248579818, 0.63492763921459217], [0.4709336932486427, 0.27710644097533355, 0.67857048808740028, 0.59051595091898457], [0.46243749845704213, 0.20582174650489374, 0.70212962181458471, 0.62065530219997345], [0.3716693043802558, 0.24816425461333205, 0.72974536954833225, 0.61228513876937507]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.4148476368534661, 0.25084101186803481, 0.72401774693717746, 0.7248543591008092], [0.4148476368534661, 0.25084101186803481, 0.72401774693717746, 0.70471053038633003], [0.3958884485772845, 0.25084101186803481, 0.72401774693717746, 0.71777450517281793], [0.3958884485772845, 0.2977848638490812, 0.71822469381960741, 0.70554199856631639], [0.4148476368534661, 0.25084101186803481, 0.72401774693717746, 0.7257470878379928], [0.3958884485772845, 0.28390275747776822, 0.71652120126353402, 0.70749923082973865], [0.3958884485772845, 0.28146912375728461, 0.69959623080972999, 0.57938951211139122], [0.38363816383645405, 0.27710644097533355, 0.67857048808740028, 0.7102737497901267], [0.4709336932486427, 0.27710644097533355, 0.72401774693717746, 0.72534342543062114], [0.3958884485772845, 0.27710644097533355, 0.69959623080972999, 0.58496691299820847]])), ([0.041355422472121837, 0.78171163709510993, 0.27034004904774539, 0.5625273082234391], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.27337223747191136, 0.77223249449335618, 0.48823900913221485, 0.30665752852260197], [0.42863996373467406, 0.89992406202978381, 0.00069365246149461456, 0.087862962827725252], [0.0018200521747896894, 0.84330815738531917, 0.36084465512465363, 0.39919767086565594], [0.011884186418476994, 0.80836167671706283, 0.33233313152502131, 0.48581228722735859], [0.010097868157984384, 0.83930296740934329, 0.17493059077637166, 0.51499643666973383], [0.0096859680741373296, 0.81092948204414494, 0.39396734318860649, 0.43351766195936181], [0.37079749037006798, 0.82058465177577278, 0.17040355809158325, 0.40131484481626911], [0.011237298293041539, 0.82188700788760127, 3.0373665159877125e-08, 0.56954888040293872], [0.37089003674316662, 0.82459659300363586, 0.0081383270413966889, 0.43029962794951909], [0.041355422472121837, 0.78171163709510993, 0.27034004904774539, 0.5625273082234391]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.27337223747191136, 0.77223249449335618, 0.24411950456610743, 0.30665752852260197], [0.42863996373467406, 0.89992406202978381, 0.00034682623074730728, 0.087862962827725252], [0.0064295093345402513, 0.89992406202978381, 0.18042232756232682, 0.087862962827725252], [0.011461576456383904, 0.89992406202978381, 0.16616656576251065, 0.087862962827725252], [0.0105684173261376, 0.89992406202978381, 0.36084465512465363, 0.087862962827725252], [0.010362467284214073, 0.89992406202978381, 0.19698367159430324, 0.087862962827725252], [0.42863996373467406, 0.89992406202978381, 0.085201779045791626, 0.087862962827725252], [0.011138132393666176, 0.82188700788760127, 1.5186832579938562e-08, 0.56954888040293872], [0.42863996373467406, 0.89992406202978381, 0.0040691635206983445, 0.087862962827725252], [0.42863996373467406, 0.82058465177577278, 0.13517002452387269, 0.40131484481626911]])), ([0.37668833205808328, 0.92635044651184961, 0.0, 0.0], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.33057705368561141, 0.94381322553471203, 0.0, 0.0], [0.33771366466098102, 0.94132678676408676, 0.0, 8.5662680461407412e-17], [0.35293429430119766, 0.93569438429706786, 0.0, 4.347902627226207e-16], [0.33315462199554707, 0.94293868322167373, 0.0, 1.8954038069960844e-16], [0.35383787577328141, 0.94052759620460313, 0.0, 0.0], [0.35156661290689106, 0.93703195013304974, 6.4830278083502129e-17, 0.0], [0.32888900170313878, 0.94438066413765054, 0.0, 0.0], [0.3476433948756022, 0.94165366955781427, 6.8997527653671849e-18, 0.0], [0.32430097251777762, 0.94598382133414938, 0.0, 0.0], [0.37668833205808328, 0.92635044651184961, 0.0, 0.0]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.33057705368561141, 0.94381322553471203, 0.0, 0.0], [0.36602371393044941, 0.94132678676408676, 0.0, 4.2831340230703706e-17], [0.37363402875055773, 0.93569438429706786, 0.0, 2.1739513136131035e-16], [0.36374419259773239, 0.93569438429706786, 0.0, 9.4770190349804221e-17], [0.37408581948659958, 0.94052759620460313, 0.0, 0.0], [0.37295018805340441, 0.94052759620460313, 3.2415139041751065e-17, 0.0], [0.36161138245152824, 0.94052759620460313, 0.0, 0.0], [0.35293429430119766, 0.93569438429706786, 3.4498763826835925e-18, 0.0], [0.35931736785884771, 0.94052759620460313, 0.0, 0.0], [0.38551104762900051, 0.92635044651184961, 0.0, 0.0]])), ([4.8292294672503441e-14, 4.1114625077775409e-15, 1.0000008522768238, 3.2326730600700383e-06], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[2.6207429211132817e-14, 7.2234126722060369e-13, 1.0000001499360287, 1.703509844319395e-05], [1.0873657970968404e-16, 2.8926843090597497e-14, 1.0000005697490795, 3.8014577236069191e-05], [2.1664002404830111e-14, 2.9057751110427649e-13, 1.0000007470961656, 3.5630575511673463e-05], [6.0279555489052737e-17, 1.07600088103652e-17, 1.0000004191206697, 4.9641096241467671e-05], [1.089384966607267e-14, 1.0192071078506615e-15, 1.0000004854648781, 2.1919288946316802e-05], [5.0515230701909547e-11, 6.532035217615027e-12, 1.0000008160150018, 8.4524303235663541e-05], [7.9582089505507794e-05, 6.5989975190131517e-12, 1.0000019026216518, 1.2017554214495686e-08], [2.1187749405166941e-08, 3.9356652972148242e-05, 1.0000010140807982, 2.2456290388392688e-10], [8.6164500696154612e-08, 9.1425640599322796e-11, 1.0000003102549286, 1.9995463178795381e-06], [4.8292294672503441e-14, 4.1114625077775409e-15, 1.0000008522768238, 3.2326730600700383e-06]], [[0.0, 0.0, 0.9612129216540486, 0.0], [1.3103714605566408e-14, 3.6117063361030184e-13, 0.98060653579503865, 8.5175492215969752e-06], [2.6207429211132817e-14, 7.2234126722060369e-13, 0.98060674570156403, 1.9007288618034595e-05], [1.0832001202415056e-14, 1.4528875555213825e-13, 0.98060683437510709, 1.7815287755836731e-05], [3.0139777744526369e-17, 5.3800044051826e-18, 0.98060667038735916, 2.4820548120733835e-05], [5.4469248330363348e-15, 5.0960355392533073e-16, 0.98060670355946333, 1.0959644473158401e-05], [2.5257615350954773e-11, 3.2660176088075135e-12, 0.98060686883452519, 4.226215161783177e-05], [3.9791044752753897e-05, 3.2994987595065759e-12, 0.98060741213785019, 6.0087771072478429e-09], [1.0593874702583471e-08, 1.9678326486074121e-05, 0.98060696786742341, 1.1228145194196344e-10], [4.3082250348077306e-08, 4.5712820299661398e-11, 0.98060661595448861, 1.703509844319395e-05], [2.4146147336251721e-14, 2.0557312538887704e-15, 0.98060688696543619, 1.6163365300350191e-06]])), ([5.0581005797413596e-14, 0.96004999901920529, 1.2643181288544932e-05, 0.27983963204479678], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[9.3181332364520072e-10, 0.93016027355262954, 1.5759105331236806e-10, 0.36716029853674809], [2.3608430145693889e-14, 0.92233537844493452, 3.4388997729093096e-12, 0.38639202719959043], [1.702210246513077e-10, 0.94013539484209308, 1.0467583358520068e-05, 0.34080292079300173], [9.6925831431972223e-12, 0.92313283558269377, 1.1611919220069e-11, 0.38448183419525866], [3.1742550474747579e-11, 0.92173445979141333, 3.606397941888956e-11, 0.38782984357929678], [1.0219398960576794e-08, 0.95359503779709831, 8.0044365041963289e-09, 0.30109783621434888], [2.8255260850076066e-08, 0.9197899524240668, 2.9424534388935339e-07, 0.39241209901098606], [5.7994769627835182e-10, 0.91849316930962732, 1.4425431898442717e-10, 0.39544573549220791], [1.1169593980085707e-05, 0.92220212181007688, 4.553741415392841e-10, 0.38670953301777744], [5.0581005797413596e-14, 0.96004999901920529, 1.2643181288544932e-05, 0.27983963204479678]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [9.3181332364520072e-10, 0.93016027355262954, 7.8795526656184031e-11, 0.22818271517604352], [1.1804215072846945e-14, 0.93016027355262954, 1.7194498864546548e-12, 0.23779857950746469], [8.5110512325653849e-11, 0.94013539484209308, 1.0467583358520068e-05, 0.21500402630417034], [4.8462915715986111e-12, 0.93016027355262954, 1.0467583358520068e-05, 0.23684348300529881], [1.5871275237373789e-11, 0.94013539484209308, 1.803198970944478e-11, 0.23851748769731787], [5.1096994802883969e-09, 0.95359503779709831, 4.0022182520981644e-09, 0.30109783621434888], [1.4127630425038033e-08, 0.95359503779709831, 1.4712267194467669e-07, 0.24080861541316251], [1.0219398960576794e-08, 0.95359503779709831, 7.2127159492213586e-11, 0.24232543365377343], [5.5847969900428537e-06, 0.95359503779709831, 2.2768707076964205e-10, 0.2379573324165582], [2.5290502898706798e-14, 0.94013539484209308, 6.3215906442724661e-06, 0.18452238193006787]])), ([0.51932930718767112, 1.0179038536723131e-06, 0.67835523112907503, 0.5197846148002232], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.43402240689466137, 6.1334598504550978e-10, 0.7894637769105497, 0.4363456747251096], [0.51235704913918367, 6.9406753811911998e-09, 0.73239782905344886, 0.45071870181983203], [0.50051059823258492, 1.9567866306212109e-09, 0.69913789756671429, 0.51274538528878499], [0.43289119322036651, 3.3637447703745227e-10, 0.8282067820296154, 0.35888988130046651], [0.52552757724846311, 0.0031555603748401875, 0.79638712482485641, 0.3006426591798454], [0.45888565314271262, 6.8577786886579053e-07, 0.7414540717975856, 0.49090067715285424], [0.4169744950898433, 1.1107786890423849e-09, 0.71771032491153086, 0.55775672145426336], [0.54491551340015731, 2.6410350017609139e-08, 0.70486659957605291, 0.45416157953217184], [0.54938131388826605, 1.6731530163929493e-06, 0.72449794483055774, 0.41688655852829509], [0.51932930718767112, 1.0179038536723131e-06, 0.67835523112907503, 0.5197846148002232]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.31821172339882176, 3.0667299252275489e-10, 0.7894637769105497, 0.4363456747251096], [0.35737904452108293, 3.4703376905955999e-09, 0.7894637769105497, 0.4363456747251096], [0.35145581906778356, 9.7839331531060544e-10, 0.7894637769105497, 0.4363456747251096], [0.31764611656167435, 1.6818723851872614e-10, 0.8282067820296154, 0.35888988130046651], [0.36396430857572265, 0.0015777801874200937, 0.8282067820296154, 0.35888988130046651], [0.33064334652284744, 3.4288893443289527e-07, 0.8282067820296154, 0.35888988130046651], [0.30968776749641275, 5.5538934452119246e-10, 0.71771032491153086, 0.55775672145426336], [0.37365827665156975, 1.3205175008804569e-08, 0.8282067820296154, 0.35888988130046651], [0.37589117689562412, 8.3657650819647467e-07, 0.71771032491153086, 0.41688655852829509], [0.36086517354532666, 5.0895192683615653e-07, 0.79638712482485641, 0.55775672145426336]])), ([9.057206540583859e-11, 0.0037335116049315468, 0.57002059637057156, 0.82207308647776245], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[3.2128256486782587e-07, 0.35779537454044158, 0.49120141328564254, 0.79442832355230564], [2.0011028094786714e-09, 0.17755631055048307, 0.6639853851321853, 0.72645708321690061], [4.4024042750441439e-09, 0.30865788130477456, 0.58448549628642432, 0.75050878131289278], [1.7430891433936467e-06, 0.002265781225347617, 0.66903651670702324, 0.74324584034467656], [2.4156906209578954e-10, 0.00404079923793863, 0.66991740526954402, 0.7424275886537206], [1.5657649728982062e-05, 0.54644940091966476, 0.35499840321239695, 0.75862082322622626], [0.010559417837917326, 0.417981980075676, 0.24363486558240707, 0.88237538894111955], [2.0889150170422783e-07, 0.32203036685127473, 0.50430452602812614, 0.80132270907627312], [1.7946333386450466e-07, 0.22690990874536868, 0.57606950340599139, 0.78568074672279575], [9.057206540583859e-11, 0.0037335116049315468, 0.57002059637057156, 0.82207308647776245]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [3.2128256486782587e-07, 0.18082848085179329, 0.49120141328564254, 0.79442832355230564], [1.0005514047393357e-09, 0.090708948856814034, 0.6639853851321853, 0.72645708321690061], [2.2012021375220719e-09, 0.17755631055048307, 0.6639853851321853, 0.72645708321690061], [8.7154457169682335e-07, 0.0030636841942463071, 0.6639853851321853, 0.74324584034467656], [1.2078453104789477e-10, 0.0039511932005418136, 0.66991740526954402, 0.7424275886537206], [7.8288248644910308e-06, 0.27515549404140488, 0.6639853851321853, 0.75050878131289278], [4.4024042750441439e-09, 0.2109217836194105, 0.6639853851321853, 0.72645708321690061], [1.0444575085211391e-07, 0.417981980075676, 0.6639853851321853, 0.79442832355230564], [8.9731666932252328e-08, 0.11538574795425684, 0.6639853851321853, 0.72645708321690061], [4.5286032702919295e-11, 0.003797549384038272, 0.57002059637057156, 0.82207308647776245]])), ([3.4718069938983321e-07, 0.78052241241612041, 9.4041044020052875e-11, 0.625129658009832], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[4.5580837062252531e-11, 0.76937179977757975, 4.7550175332067612e-14, 0.63880241579292785], [2.3340456592034791e-05, 0.77231845024980728, 9.1394024678915277e-11, 0.63523677530618639], [1.2850384163207086e-07, 0.77002888212115184, 3.166518692401038e-10, 0.63801157016513055], [6.3857328595231393e-14, 0.77044337593015599, 2.1569279979090558e-09, 0.63750980740564389], [3.3879978829953894e-11, 0.79162027607311369, 2.4336279951303753e-11, 0.61101460818865438], [1.5798700678692505e-07, 0.77197625783877022, 1.7356882423591897e-05, 0.63565213991273917], [2.0536323351120488e-13, 0.76926925428869664, 2.0856102220434491e-09, 0.63892689931041713], [2.2288902454026234e-10, 0.77457379594412812, 6.2892472939379882e-09, 0.63248596150051828], [3.8379382070492198e-08, 0.78531027351704397, 1.7074248721287783e-11, 0.61910525686738427], [3.4718069938983321e-07, 0.78052241241612041, 9.4041044020052875e-11, 0.625129658009832]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [2.2790418531126266e-11, 0.76937179977757975, 2.3775087666033806e-14, 0.63880241579292785], [1.1670228296017395e-05, 0.76937179977757975, 4.5697012339457638e-11, 0.63880241579292785], [6.4251920816035432e-08, 0.77231845024980728, 1.583259346200519e-10, 0.63880241579292785], [3.1928664297615696e-14, 0.77044337593015599, 1.0784639989545279e-09, 0.63750980740564389], [1.6939989414976947e-11, 0.79162027607311369, 1.2168139975651876e-11, 0.61101460818865438], [7.8993503393462523e-08, 0.79162027607311369, 8.6784412117959487e-06, 0.61101460818865438], [1.0268161675560244e-13, 0.79162027607311369, 1.0428051110217246e-09, 0.61101460818865438], [1.1144451227013117e-10, 0.79162027607311369, 3.1446236469689941e-09, 0.61101460818865438], [1.9189691035246099e-08, 0.79162027607311369, 2.4336279951303753e-11, 0.61101460818865438], [1.7359034969491661e-07, 0.79162027607311369, 4.7020522010026438e-11, 0.61101460818865438]]))], (2, 'rNSGAII_solution', 'DTLZ2'): [([0.90201649409318907, 0.43170209947892485], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[0.89433366568546324, 0.44740131768403124], [0.86922719382382707, 0.49441541163970643], [0.87162696356165481, 0.4901723571147667], [0.89388697463118683, 0.44829650389901737], [0.87009812649958507, 0.49288315918052461], [0.90081348115312743, 0.4342098844739235], [0.88073569682315089, 0.47360856598933981], [0.89794867541521872, 0.44100536090968423], [0.87271131919789791, 0.48823718448415016], [0.90201649409318907, 0.43170209947892485]], [[0.5858121009812931, 0.0], [0.74007288333337817, 0.44740131768403124], [0.72751964740256003, 0.49441541163970643], [0.89433366568546324, 0.44740131768403124], [0.73984953780623997, 0.49441541163970643], [0.72795511374043909, 0.49441541163970643], [0.74331279106721027, 0.49441541163970643], [0.73327389890222205, 0.49441541163970643], [0.74188038819825586, 0.49441541163970643], [0.72926171008959551, 0.49441541163970643], [0.74391429753724103, 0.49441541163970643]])), ([0.979260774017774, 0.20260551431502524], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.98170288098554737, 0.19042893010424952], [0.95021474994573751, 0.31159809659225429], [0.94606290922669145, 0.32398430928649818], [0.9876038104850412, 0.17743337424955971], [0.94811275092982361, 0.31793563419513476], [0.96319640395583994, 0.26879969288527217], [0.98663090521516372, 0.16299786934586782], [0.95512179012038823, 0.29651323079045255], [0.95976471850135281, 0.28081183404236332], [0.979260774017774, 0.20260551431502524]], [[0.8233781347997915, 0.0], [0.98170288098554737, 0.095214465052124761], [0.98170288098554737, 0.15579904829612715], [0.98170288098554737, 0.16199215464324909], [0.9876038104850412, 0.088716687124779856], [0.9876038104850412, 0.15896781709756738], [0.9876038104850412, 0.13439984644263608], [0.98663090521516372, 0.081498934672933912], [0.98663090521516372, 0.14825661539522628], [0.98663090521516372, 0.14040591702118166], [0.98663090521516372, 0.10130275715751262]])), ([0.70635824710476569, 0.70785459826795716], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[0.71121949368926662, 0.70297039857164201], [0.71860025463998511, 0.6954237589086899], [0.696103540399164, 0.7179423830818622], [0.70774724590431204, 0.70646588138750832], [0.70209232629208052, 0.71208771061283949], [0.70316085937110229, 0.71103173601417213], [0.67338456362435795, 0.73929352762611844], [0.70413734482018342, 0.71006393861002748], [0.70561545497406064, 0.70859599983675514], [0.70635824710476569, 0.70785459826795716]], [[0.0, 0.0], [0.71121949368926662, 0.351485199285821], [0.71860025463998511, 0.34771187945434495], [0.71860025463998511, 0.3589711915409311], [0.71860025463998511, 0.35323294069375416], [0.71860025463998511, 0.35604385530641974], [0.71860025463998511, 0.35551586800708607], [0.71860025463998511, 0.6954237589086899], [0.71860025463998511, 0.35503196930501374], [0.71860025463998511, 0.35429799991837757], [0.71860025463998511, 0.35392729913397858]])), ([0.85932010104501155, 0.51211739905669862], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[0.93166020790515769, 0.37302185156954659], [0.92173552442098827, 0.38781995708436312], [0.87762490492258638, 0.48165676942528424], [0.85912364795469842, 0.51176990868285333], [0.8677234411076753, 0.49704866218308602], [0.8545075448499968, 0.51944074321616662], [0.85329661567961423, 0.52142664524158444], [0.86889303311124411, 0.49500612847456538], [0.86692810581614166, 0.49855387707939336], [0.85932010104501155, 0.51211739905669862]], [[0.5389452641563056, 0.0], [0.7353027360307316, 0.37302185156954659], [0.73034039428864694, 0.38781995708436312], [0.70828508453944594, 0.48165676942528424], [0.69903445605550196, 0.51176990868285333], [0.7033343526319904, 0.51176990868285333], [0.69672640450315115, 0.51944074321616662], [0.69612093991795998, 0.52142664524158444], [0.7039191486337748, 0.52142664524158444], [0.70293668498622364, 0.52142664524158444], [0.69913268260065853, 0.52142664524158444]])), ([0.7007182194694993, 0.71343869775923074], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.71554546129743746, 0.69856730957249791], [0.7111606184484387, 0.7030298334331736], [0.715795494782963, 0.69831080138166768], [0.70555457366783736, 0.70865614945085753], [0.70344024296002883, 0.71075494462779309], [0.71404402690760749, 0.70010246118154773], [0.70463886499902018, 0.70956661237659802], [0.70469562794128593, 0.70951036851891591], [0.71209424018278711, 0.70208423337962633], [0.7007182194694993, 0.71343869775923074]], [[0.0, 0.0], [0.35777273064871873, 0.69856730957249791], [0.35558030922421935, 0.7030298334331736], [0.3578977473914815, 0.7030298334331736], [0.35277728683391868, 0.70865614945085753], [0.35172012148001441, 0.71075494462779309], [0.35702201345380374, 0.71075494462779309], [0.35231943249951009, 0.71075494462779309], [0.35234781397064296, 0.71075494462779309], [0.35604712009139355, 0.71075494462779309], [0.35035910973474965, 0.71343869775923074]])), ([0.70272491566332973, 0.71146184644814714], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.70148802416123301, 0.7126814893666914], [0.71227744588406217, 0.70189851163954242], [0.70882056695497186, 0.705389059563587], [0.71050183597388372, 0.70369581772681034], [0.70502193839694138, 0.70918555708691433], [0.71380574194220969, 0.70034425159193769], [0.70708104115320536, 0.70713293344130768], [0.7002732725178078, 0.71387576194837743], [0.70250648998667409, 0.71167807602862709], [0.70272491566332973, 0.71146184644814714]], [[0.0, 0.0], [0.70148802416123301, 0.3563407446833457], [0.71227744588406217, 0.35094925581977121], [0.71227744588406217, 0.3526945297817935], [0.71227744588406217, 0.35184790886340517], [0.71227744588406217, 0.35459277854345717], [0.71380574194220969, 0.35017212579596885], [0.71380574194220969, 0.35356646672065384], [0.71380574194220969, 0.35693788097418871], [0.71380574194220969, 0.35583903801431355], [0.71380574194220969, 0.70034425159193769]])), ([0.84563087686052396, 0.53377240272847859], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.84566486331757507, 0.53371962205246748], [0.84549197865412462, 0.5339928244129345], [0.84122889009758905, 0.54068373553754612], [0.8458477708832457, 0.53343123603220832], [0.84510577680617627, 0.53460338720373535], [0.84542380920521898, 0.53410042708810557], [0.84546228076384877, 0.53403794658656356], [0.84560372146957763, 0.53381478279941763], [0.84578365246382325, 0.53353143123865632], [0.84563087686052396, 0.53377240272847859]], [[0.8521054652075405, 0.5432635491282871], [0.84888516426255778, 0.53849158559037735], [0.84879872193083261, 0.53862818677061086], [0.84122889009758905, 0.54197364233291667], [0.84122889009758905, 0.53834739258024777], [0.84860562100685843, 0.53893346816601118], [0.84876463720637974, 0.53868198810819634], [0.84878387298569469, 0.53865074785742539], [0.84885459333855906, 0.53853916596385232], [0.84894455883568187, 0.53839749018347172], [0.84886817103403223, 0.53851797592838291]])), ([0.72187861568884493, 0.69202092883240673], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[0.72685350589864872, 0.686793547095364], [0.76578502884276056, 0.6430974208696375], [0.74111250721215516, 0.67138289903813664], [0.77835019621672508, 0.6278310893647614], [0.76352301612277629, 0.64578176352568395], [0.72120861230071631, 0.69271859149217518], [0.74583989077801216, 0.66612580694848478], [0.76876416305414819, 0.63953316013229589], [0.72505978866370291, 0.68868707138109042], [0.72187861568884493, 0.69202092883240673]], [[0.2295261844565868, 0.17176071551877525], [0.72685350589864872, 0.686793547095364], [0.76578502884276056, 0.40742906819420638], [0.76578502884276056, 0.42157180727845595], [0.77835019621672508, 0.39979590244176832], [0.77835019621672508, 0.4087712395222296], [0.77835019621672508, 0.43223965350547522], [0.77835019621672508, 0.41894326123363002], [0.77835019621672508, 0.40564693782553557], [0.77835019621672508, 0.6278310893647614], [0.77835019621672508, 0.43189082217559099]])), ([6.2395406169268833e-05, 1.0000004450566375], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[0.00042481966181649673, 1.0000005608290539], [5.2828744620148272e-05, 1.0000013461971502], [1.095739058155433e-05, 1.0000005670741721], [0.00016104039100299004, 1.0000014485028819], [1.8867387375276774e-05, 1.0000014709058769], [2.5567561236912245e-05, 1.0000030760414749], [3.1672125112747742e-05, 1.000001924510032], [8.5981277492604987e-06, 1.0000012280057151], [8.4540061927668587e-05, 1.0000017445411846], [6.2395406169268833e-05, 1.0000004450566375]], [[0.0, 0.9893548997434237], [0.00021240983090824837, 1.0], [2.6414372310074136e-05, 1.0], [5.4786952907771648e-06, 1.0], [8.052019550149502e-05, 1.0], [9.4336936876383872e-06, 1.0], [1.2783780618456122e-05, 1.0], [1.5836062556373871e-05, 1.0], [4.2990638746302494e-06, 1.0], [4.2270030963834294e-05, 1.0], [3.1197703084634417e-05, 1.0]])), ([0.3600340591503382, 0.93294064207623151], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.34935378961767666, 0.9369919310137812], [0.25723859181029063, 0.96634818484208185], [0.26182773019279654, 0.96529712518046962], [0.27823338242120244, 0.96101992078774567], [0.31629643138428754, 0.95138086401067534], [0.35789460619568575, 0.9337625978462647], [0.35357726145157869, 0.93540622615235858], [0.35486189715455296, 0.93492150929718632], [0.23682556487829762, 0.97160493529498149], [0.3600340591503382, 0.93294064207623151]], [[0.21536704667589168, 0.875015209332337], [0.34935378961767666, 0.9060035701730591], [0.34935378961767666, 0.92068169708720937], [0.34935378961767666, 0.92015616725640337], [0.34935378961767666, 0.91801756506004129], [0.34935378961767666, 0.91319803667150623], [0.35789460619568575, 0.90438890358930091], [0.35789460619568575, 0.90521071774234785], [0.35789460619568575, 0.9337625978462647], [0.35789460619568575, 0.92331007231365925], [0.3600340591503382, 0.90397792570428426]]))], (4, 'rNSGAII_solution', 'DTLZ1'): [([0.061003937511619359, 2.3962835195091377e-13, 0.45956227512554265, 3.9011260170060119e-14], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.00085844740756019602, 1.9821318514030397e-10, 0.66320206345982968, 3.0374917088131893e-14], [0.061879502359919646, 3.2734267702579479e-12, 0.45363123651843829, 2.0081409085837288e-12], [0.051188926099713414, 1.543066815616408e-10, 0.53167595789086397, 1.2942200292462497e-16], [0.062927643615979995, 3.046987328502959e-12, 0.44247987681048245, 1.0450094766378501e-11], [0.062371215073961137, 3.2268616916816698e-15, 0.44306167354501075, 1.3508906013425864e-11], [0.064039577528087172, 4.7098343217313806e-11, 0.44537808465162942, 3.1583954247629589e-10], [0.098910012265639044, 2.2633277161926523e-12, 0.71383859309329023, 4.2652612168199137e-11], [0.0132253974834924, 4.2633967615382323e-14, 0.49472550992079545, 0.0], [0.059817714999463638, 1.2992563273096725e-10, 0.44554085080146438, 8.163433411806484e-14], [0.061003937511619359, 2.3962835195091377e-13, 0.45956227512554265, 3.9011260170060119e-14]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.00085844740756019602, 9.9106592570151987e-11, 0.66320206345982968, 1.5187458544065946e-14], [0.065900726816398511, 3.2734267702579479e-12, 0.57366217658867313, 1.0040704542918644e-12], [0.00085844740756019602, 7.7153340780820398e-11, 0.61268453727488603, 6.4711001462312483e-17], [0.066424797444428682, 1.5234936642514795e-12, 0.56808649673469525, 5.2250473831892504e-12], [0.06614658317341926, 1.6134308458408349e-15, 0.56837739510195939, 6.754453006712932e-12], [0.066980764400482271, 2.3549171608656903e-11, 0.56953560065526876, 1.5791977123814795e-10], [0.098910012265639044, 1.1316638580963261e-12, 0.70376585487609922, 4.2652612168199137e-11], [0.051188926099713414, 2.1316983807691162e-14, 0.59420931328985183, 0.0], [0.051188926099713414, 6.4962816365483626e-11, 0.56961698373018621, 4.081716705903242e-14], [0.065462944392248368, 1.1981417597545689e-13, 0.44537808465162942, 1.9505630085030059e-14]])), ([0.11385280201657645, 0.11849390283367493, 0.20225569444757741, 0.15379312789037208], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.10921193808902489, 0.11494834321952864, 0.17815865312618678, 0.16648931755379884], [0.14891535020582491, 0.11620586408566946, 0.19953828726525719, 0.16124089465485744], [0.12679262874609398, 0.15645584577787558, 0.17583960384863559, 0.17572313068704354], [0.1344272783294288, 0.16629996711663764, 0.19390428903661699, 0.17724978102647576], [0.12137456042449178, 0.11015140710658021, 0.17538688880957801, 0.14314420723274432], [0.13515424864432113, 0.15330589203805714, 0.18179732050549524, 0.18551122290787375], [0.12058467259025805, 0.11950776606461898, 0.19243671473916998, 0.14584757411520802], [0.11747874880720172, 0.1224392239610252, 0.1855237409254348, 0.13340038572051854], [0.10339923397666176, 0.15982831030385397, 0.16726265607798663, 0.18104637493184736], [0.11385280201657645, 0.11849390283367493, 0.20225569444757741, 0.15379312789037208]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.10921193808902489, 0.20323513295058221, 0.17815865312618678, 0.16648931755379884], [0.14891535020582491, 0.20386389338365263, 0.19953828726525719, 0.16124089465485744], [0.14891535020582491, 0.22398888422975569, 0.17583960384863559, 0.17572313068704354], [0.14891535020582491, 0.22891094489913671, 0.19953828726525719, 0.17724978102647576], [0.12137456042449178, 0.20083666489410801, 0.17583960384863559, 0.17572313068704354], [0.12679262874609398, 0.22241390735984645, 0.17538688880957801, 0.14314420723274432], [0.12137456042449178, 0.20551484437312739, 0.19243671473916998, 0.17724978102647576], [0.12137456042449178, 0.20698057332133049, 0.17538688880957801, 0.17572313068704354], [0.11747874880720172, 0.22567511649274488, 0.1855237409254348, 0.13340038572051854], [0.10339923397666176, 0.20500791275765537, 0.16726265607798663, 0.18104637493184736]])), ([0.15568552285464726, 0.094201754805114016, 0.28432840869621695, 0.22310294700469202], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.20728352401048983, 0.19988854684735452, 0.76711774588675308, 0.00039642735687714934], [0.25644959205930895, 0.19497034040191963, 0.41482487321648731, 0.35535737051187938], [0.26856855805099022, 0.14431539183520756, 0.50638674506122872, 0.38247439771197406], [0.14967735604030433, 0.091213969857658866, 0.41652892183185192, 0.26135337124331881], [0.027734190296885283, 0.077846783533512889, 0.50918036925164756, 0.17203769781563269], [0.099899825216108407, 0.073035694387037331, 0.25911493367571603, 0.22085673152607843], [0.17657569163995113, 0.089751388389676903, 0.19706885195989668, 0.21702278249336363], [0.13975886950314231, 0.12375853701045259, 0.3412491033224847, 0.27774686142765781], [0.30076307724819162, 0.086956273541969206, 0.64991468254770302, 0.59170655405786887], [0.15568552285464726, 0.094201754805114016, 0.28432840869621695, 0.22310294700469202]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.20728352401048983, 0.19988854684735452, 0.76711774588675308, 0.415213988009073], [0.25644959205930895, 0.19497034040191963, 0.41482487321648731, 0.59269445958657407], [0.26856855805099022, 0.14431539183520756, 0.50638674506122872, 0.60625297318662141], [0.25644959205930895, 0.19497034040191963, 0.41482487321648731, 0.54569245995229387], [0.14967735604030433, 0.19497034040191963, 0.41482487321648731, 0.50103462323845083], [0.25644959205930895, 0.19497034040191963, 0.50638674506122872, 0.52544414009367357], [0.17657569163995113, 0.19497034040191963, 0.19706885195989668, 0.52352716557731616], [0.17657569163995113, 0.089751388389676903, 0.41652892183185192, 0.55388920504446326], [0.099899825216108407, 0.089751388389676903, 0.3412491033224847, 0.71086905135956879], [0.17657569163995113, 0.089751388389676903, 0.64991468254770302, 0.52656724783298037]])), ([0.014713296541785032, 0.48590545278678993, 0.0011807924156909577, 6.0710154386064695e-07], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.06638199157224739, 0.44818088052890137, 3.2900037951334317e-09, 9.7756700873226627e-06], [0.011950316908173605, 0.48812293174014965, 2.5762658793335086e-08, 1.040857482901646e-09], [0.013163923628189944, 0.49112835385073067, 4.0228797733093153e-08, 3.5169457661241512e-07], [0.011781016045397151, 0.49064189173905515, 5.5369142661985391e-09, 3.9794033461638526e-07], [0.019876010277325145, 0.48630117295106418, 5.8561582693888475e-07, 3.4721179177687073e-05], [0.013687482893150367, 0.48703764189723781, 0.0030669401880092941, 7.2927563374926058e-06], [0.011561319529553683, 0.48879251219180769, 1.4664081535940545e-05, 4.2432365254178702e-10], [0.01124959808191449, 0.49151415354796291, 0.00011269296653844029, 5.7201002918663991e-05], [0.03308008613665174, 0.48664469722535991, 1.5412240825775017e-08, 9.150720847843015e-10], [0.014713296541785032, 0.48590545278678993, 0.0011807924156909577, 6.0710154386064695e-07]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.06638199157224739, 0.44818088052890137, 1.6450018975667159e-09, 4.8878350436613313e-06], [0.011494641701232209, 0.48812293174014965, 1.2881329396667543e-08, 5.2042874145082299e-10], [0.01210144506124038, 0.48812293174014965, 2.0114398866546576e-08, 1.7584728830620756e-07], [0.011409991269843982, 0.49064189173905515, 2.7684571330992696e-09, 1.9897016730819263e-07], [0.015457488385807979, 0.48812293174014965, 2.9280791346944237e-07, 3.5169457661241512e-07], [0.011950316908173605, 0.48703764189723781, 0.0015334700940046471, 3.6463781687463029e-06], [0.011300143011922248, 0.48812293174014965, 7.3320407679702725e-06, 2.1216182627089351e-10], [0.011144282288102651, 0.49151415354796291, 5.6346483269220146e-05, 2.8600501459331996e-05], [0.03308008613665174, 0.48664469722535991, 7.7061204128875087e-09, 4.5753604239215075e-10], [0.012876131518037924, 0.49112835385073067, 0.00059039620784547887, 3.0355077193032348e-07]])), ([0.35523118856891772, 0.93915304855534032, 1.0336737929727158e-11, 1.4269957964994022e-13], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.37995227941929638, 0.82895149560777148, 2.0462193243830513e-11, 1.2079375250526361e-15], [0.41857330014531563, 1.0821204479997892, 4.7650595910110743e-14, 1.6580208120277606e-10], [0.18622196509897979, 0.83088203848880038, 2.0163214529244527e-12, 1.1292122832238202e-16], [0.32431855717114733, 0.69630460396232019, 2.1533799803542907e-12, 1.0142551149321412e-12], [0.32181409090049734, 0.70568156630151357, 1.3688992035936373e-15, 1.9679066801032556e-12], [0.16778842359649729, 0.38097903945934342, 1.8277628179491885e-16, 1.5773593118906032e-13], [0.31092528339314729, 0.70405323786651275, 1.9381863409402828e-14, 1.5516759252762659e-13], [0.39005488345421768, 1.0142466229147022, 3.6353095861031537e-11, 2.2949773386448102e-13], [0.087179922117378253, 0.89841777365717768, 1.337459422776268e-11, 2.3197745003483282e-14], [0.35523118856891772, 0.93915304855534032, 1.0336737929727158e-11, 1.4269957964994022e-13]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.38714302130960709, 0.82895149560777148, 1.0231096621915256e-11, 6.0396876252631803e-16], [0.40645353167261666, 1.0, 2.3825297955055371e-14, 8.2901040601388028e-11], [0.29027786414944878, 0.82895149560777148, 1.0081607264622263e-12, 5.6460614161191008e-17], [0.35932616018553254, 0.82895149560777148, 1.0766899901771453e-12, 5.0712755746607059e-13], [0.35807392705020757, 0.82895149560777148, 6.8444960179681864e-16, 9.839533400516278e-13], [0.28106109339820751, 0.70568156630151357, 9.1388140897459424e-17, 7.8867965594530161e-14], [0.35262952329653252, 0.82895149560777148, 9.6909317047014141e-15, 7.7583796263813294e-14], [0.39219432332706772, 1.0, 1.8176547930515769e-11, 1.1474886693224051e-13], [0.240756842658648, 0.83088203848880038, 6.6872971138813399e-12, 1.1598872501741641e-14], [0.37478247588441771, 0.82895149560777148, 5.1683689648635789e-12, 1.2079375250526361e-15]])), ([1.4009460508924754e-17, 5.9192512159237466e-18, 0.51673660809622612, 1.7740878620412027e-12], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[1.1588122691256046e-17, 7.057590994506136e-19, 0.52103922372178335, 3.1347275368243587e-13], [4.9318368928045503e-18, 3.013650874614285e-16, 0.95847467636117334, 2.1282413084326452e-15], [1.3734805881443206e-17, 1.8765827109166661e-18, 0.52745912456465527, 2.1901337697548456e-14], [0.0, 0.0, 0.50009937574363406, 0.0], [3.6093826547756916e-18, 8.2409581972125531e-18, 0.50612819890697547, 4.6351658356752145e-10], [3.4396636855141882e-16, 1.2307005199543411e-15, 0.98317469591273876, 0.022481536694247687], [4.0805739564317261e-18, 3.2139039881497155e-20, 0.86511993138633592, 3.4769233621319691e-14], [2.256944178580785e-17, 5.4917570637446887e-18, 0.51133841170997074, 5.676996780556679e-16], [4.7976938170926453e-18, 6.3819478750371605e-17, 1.0011072501261529, 5.5572615960461578e-16], [1.4009460508924754e-17, 5.9192512159237466e-18, 0.51673660809622612, 1.7740878620412027e-12]], [[0.0, 0.0, 0.9612129216540486, 0.0], [5.794061345628023e-18, 3.528795497253068e-19, 0.52103922372178335, 1.5673637684121794e-13], [2.4659184464022752e-18, 1.5068254373071425e-16, 0.95984379900761096, 1.0641206542163226e-15], [6.8674029407216029e-18, 3.013650874614285e-16, 0.74433602310935187, 1.0950668848774228e-14], [0.0, 0.0, 0.73065614869884132, 0.0], [1.8046913273878458e-18, 4.1204790986062766e-18, 0.73367056028051203, 2.3175829178376072e-10], [1.7198318427570941e-16, 6.1535025997717055e-16, 0.97219380878339368, 0.022481536694247687], [2.040286978215863e-18, 1.6069519940748578e-20, 0.50009937574363406, 1.7384616810659845e-14], [1.1284720892903925e-17, 2.7458785318723443e-18, 0.73627566668200961, 2.8384983902783395e-16], [2.3988469085463226e-18, 3.1909739375185803e-17, 0.98116008589010073, 2.7786307980230789e-16], [7.004730254462377e-18, 3.2139039881497155e-20, 0.73897476487513736, 8.8704393102060134e-13]])), ([8.5295083786966546e-15, 0.84819501915658868, 6.2770839613080487e-07, 0.1524235812495138], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[6.7702555404713306e-13, 0.91311349467662661, 8.6909612727348035e-13, 0.092527385415066096], [3.4614807717018931e-12, 0.90935116113440218, 3.5002228622917761e-13, 0.091324956121421552], [4.7989185358880519e-14, 0.43110473867165083, 1.0816866395902556e-14, 0.075101736863532317], [7.4498671967761779e-13, 0.43031787959673701, 1.0032725174577032e-15, 0.074384674812746687], [2.6906794086375604e-09, 0.90906003078603836, 1.8332038043671551e-09, 0.092336746452142973], [5.4282676564345937e-14, 0.45508098493866006, 7.2848734706169972e-12, 0.068723156064642313], [3.1926979360327141e-14, 0.42687141780273247, 1.6525692469509969e-08, 0.074407704023671375], [2.3997968284578247e-12, 0.90813474951735484, 1.3510801950605042e-07, 0.092716471334210099], [1.6468515334201109e-08, 0.92088317017536903, 1.0949626989924133e-05, 0.097831534687845376], [8.5295083786966546e-15, 0.84819501915658868, 6.2770839613080487e-07, 0.1524235812495138]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [6.7702555404713306e-13, 0.91311349467662661, 8.6909612727348035e-13, 0.090866258615202533], [1.7307403858509466e-12, 0.90935116113440218, 1.7501114311458881e-13, 0.090265043968380254], [2.3994592679440259e-14, 0.43110473867165083, 5.4084331979512782e-15, 0.082153434339435644], [3.724933598388089e-13, 0.43031787959673701, 5.0163625872885161e-16, 0.074384674812746687], [1.3453397043187802e-09, 0.90935116113440218, 9.1660190218357753e-10, 0.090770939133740958], [2.7141338282172968e-14, 0.43110473867165083, 3.6424367353084986e-12, 0.078964143939990628], [1.5963489680163571e-14, 0.43110473867165083, 8.2628462347549843e-09, 0.081806417919505159], [1.1998984142289124e-12, 0.90935116113440218, 6.7554009753025211e-08, 0.090960801574774527], [8.2342576671005543e-09, 0.90813474951735484, 5.4748134949620665e-06, 0.093518333251592173], [4.2647541893483273e-15, 0.90813474951735484, 3.1385419806540243e-07, 0.12081435653242638]])), ([0.20591045030352165, 1.4150743769857216e-14, 0.36382804396160623, 2.8147887348506455e-14], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.18432757158727248, 1.9461712306103011e-14, 0.32026312088551123, 2.4665967257129649e-13], [0.21086747360191777, 7.4915175787963944e-16, 0.37152332509861541, 7.3678209657767938e-13], [0.17369287562103625, 1.8628134351710179e-14, 0.33762191292802762, 5.5458290130267883e-12], [0.18962085968862466, 1.2460764292595647e-13, 0.33049423089270574, 8.0726636112915823e-13], [0.18239021258222174, 2.9849296225055151e-09, 0.32029672625838695, 3.0974251243120025e-14], [0.18116199145391887, 5.1187639488362162e-13, 0.3418839868138524, 3.8572843926417603e-12], [0.20309174203122257, 3.6154614947675074e-10, 0.34604703494509526, 1.2845536582136119e-08], [0.18536515278647614, 6.2973778139534113e-15, 0.31620668879983904, 1.4115201275106522e-12], [0.18800505925389224, 4.1745509105937617e-17, 0.33025707182589914, 1.3291429321541915e-14], [0.20591045030352165, 1.4150743769857216e-14, 0.36382804396160623, 2.8147887348506455e-14]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.19336430574512734, 9.7308561530515053e-15, 0.32026312088551123, 1.2332983628564825e-13], [0.20663425675244998, 3.7457587893981972e-16, 0.37152332509861541, 3.6839104828883969e-13], [0.18804695776200922, 9.3140671758550897e-15, 0.32026312088551123, 2.7729145065133941e-12], [0.19601094979580341, 6.2303821462978234e-14, 0.33049423089270574, 4.0363318056457912e-13], [0.19239562624260198, 1.4924648112527575e-09, 0.33049423089270574, 1.5487125621560012e-14], [0.18432757158727248, 2.5593819744181081e-13, 0.32026312088551123, 1.9286421963208802e-12], [0.2027463909671024, 1.8077307473837537e-10, 0.34604703494509526, 6.4227682910680595e-09], [0.19388309634472917, 3.1486889069767057e-15, 0.33049423089270574, 7.057600637553261e-13], [0.1952030495784372, 6.2973778139534113e-15, 0.33049423089270574, 8.0726636112915823e-13], [0.20415574510325191, 7.0753718849286082e-15, 0.36194763178718625, 1.4073943674253227e-14]])), ([0.45086949479073218, 0.0035150504538398646, 1.8170395709345527e-09, 0.046959625093037603], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[6.5395081211174195e-08, 0.37432441473339884, 7.6634457431074154e-12, 0.13169395676057186], [1.7721960707110141e-09, 0.0043729208108827843, 0.10668536894478009, 0.392881061807899], [9.8029781411360871e-09, 0.0040144996924501896, 0.32248393225383892, 0.17433322100348658], [0.13682480245864764, 0.0041718214719667088, 1.1518506948938595e-07, 0.36031081613185889], [3.1853513432296078e-08, 0.0039896873205245301, 0.34959232597019302, 0.15333525927294456], [1.0548048005697113e-09, 0.0061466132993639082, 6.4750769894310654e-10, 0.49411274851658599], [1.1583367254131723e-10, 0.0049926812383560022, 3.2882697417908777e-11, 0.49611330825062289], [6.1151357467459083e-12, 0.0030114759421595691, 0.37620131208979951, 0.12743145316599372], [5.9702454429691401e-10, 0.0033244605540930141, 0.084669283974453766, 0.4133043560235784], [0.45086949479073218, 0.0035150504538398646, 1.8170395709345527e-09, 0.046959625093037603]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [3.2697540605587097e-08, 0.18909300094827192, 3.8317228715537077e-12, 0.13169395676057186], [8.8609803535550703e-10, 0.0041172539870138903, 7.6634457431074154e-12, 0.13169395676057186], [4.9014890705680435e-09, 0.0039380434277975929, 0.32248393225383892, 0.17433322100348658], [6.5395081211174195e-08, 0.0040167043175558526, 5.7592534744692973e-08, 0.17433322100348658], [6.5395081211174195e-08, 0.0039256372418347632, 7.6634457431074154e-12, 0.36031081613185889], [5.2740240028485567e-10, 0.0050041002312544523, 3.2375384947155327e-10, 0.392881061807899], [5.7916836270658616e-11, 0.0044271342007504997, 1.6441348708954389e-11, 0.49611330825062289], [3.0575678733729542e-12, 0.0034365315526522829, 0.37620131208979951, 0.12743145316599372], [2.98512272148457e-10, 0.0035930238586190056, 0.37620131208979951, 0.15333525927294456], [0.45086949479073218, 0.0036883188084924309, 9.0851978546727635e-10, 0.046959625093037603]])), ([2.4847961185329234e-16, 0.5914757875028841, 9.2065139298732932e-14, 0.42533618857415983], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[4.6546154593579523e-14, 0.58524515528248822, 1.1110770254209628e-14, 0.42277286638836259], [1.7296755436743118e-14, 0.58385928387204544, 5.810525654795638e-12, 0.4197633100597849], [7.3517632270871043e-12, 0.58829078700021475, 6.2113041204209773e-14, 0.42140544645894368], [1.2013641186103767e-15, 0.31848261892269247, 3.7437763657582273e-13, 0.19543659694692717], [1.1168871286660287e-15, 0.58874584055874579, 1.2765573338529924e-13, 0.42354847382419147], [1.0804681642950225e-15, 0.3231110388809445, 1.1101615939926412e-08, 0.21524914750086097], [1.3998635618439762e-15, 0.29581939813772146, 3.2842550694325612e-17, 0.21278409928685235], [0.0, 0.58650298638314902, 1.4240214511354518e-10, 0.42148967061781245], [5.5027833856670009e-15, 0.59421644538119966, 1.5338981830919157e-12, 0.42798463467972869], [2.4847961185329234e-16, 0.5914757875028841, 9.2065139298732932e-14, 0.42533618857415983]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [2.3273077296789762e-14, 0.58524515528248822, 5.555385127104814e-15, 0.4259882420724681], [8.6483777183715591e-15, 0.58524515528248822, 2.905262827397819e-12, 0.42448346390817926], [3.6758816135435522e-12, 0.59204670077352195, 3.1056520602104887e-14, 0.42530453210775865], [6.0068205930518833e-16, 0.31848261892269247, 1.8718881828791137e-13, 0.19543659694692717], [5.5844356433301436e-16, 0.59227422755278747, 6.3827866692649621e-14, 0.42637604579038252], [5.4023408214751127e-16, 0.31848261892269247, 5.5508079699632062e-09, 0.19543659694692717], [6.9993178092198808e-16, 0.31848261892269247, 1.6421275347162806e-17, 0.19543659694692717], [0.0, 0.59115280046498908, 7.120107255677259e-11, 0.42354847382419147], [2.7513916928335004e-15, 0.59500952996401446, 7.6694909154595783e-13, 0.42859412621815113], [1.2423980592664617e-16, 0.59363920102485668, 4.6032569649366466e-14, 0.42798463467972869]]))], (2, 'rNSGAII_solution', 'DTLZ4'): [([0.92257127980620335, 0.38583600456681738], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[0.95153313001113848, 0.30754974523411976], [0.96984294658696057, 0.24373518413877374], [0.96296468896558907, 0.26963357675669458], [1.0000006712095251, 0.00048069491191451107], [1.0000006868134779, 4.3127793512306558e-05], [1.0000009892088935, 4.6588894686621724e-09], [0.94691794623526548, 0.32148308145575982], [1.0000002244776014, 3.2608837565612316e-97], [0.99654621901434071, 0.083100722877425759], [0.92257127980620335, 0.38583600456681738]], [[0.5858121009812931, 0.0], [0.76867261549621579, 0.30754974523411976], [0.77782752378412678, 0.30754974523411976], [0.77438839497344114, 0.30754974523411976], [0.79290638609540909, 0.00024034745595725553], [0.79290639389738549, 2.1563896756153279e-05], [1.0, 4.3127793512306558e-05], [0.76636502360827929, 0.32148308145575982], [0.79290616272944725, 1.6304418782806158e-97], [1.0, 3.2608837565612316e-97], [0.75419169039374823, 0.38583600456681738]])), ([1.0000000832280818, 0.0], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.9965978115083054, 0.082425041980212013], [1.0003147636699388, 4.0102111857856152e-05], [1.0000017326901316, 0.00026606607312436168], [1.0000000393594424, 2.8943637581735543e-06], [1.0000000948593697, 1.3826836241984621e-308], [1.0000000710571468, 6.7345797073799504e-183], [1.0000006345268619, 0.00056740766070824747], [1.0000001825385372, 3.1429358884721289e-195], [1.0000014206854737, 0.00027713558182885525], [1.0000000832280818, 0.0]], [[0.8233781347997915, 0.0], [0.9965978115083054, 0.041212520990106007], [1.0, 2.0051055928928076e-05], [1.0, 0.00013303303656218084], [1.0, 2.8943637581735543e-06], [1.0, 6.9134181209923103e-309], [1.0, 3.3672898536899752e-183], [1.0, 0.00028370383035412373], [1.0, 1.5714679442360644e-195], [1.0, 0.00013856779091442762], [1.0, 0.0]])), ([0.10680259103805674, 0.99428208996602574], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[8.4222648817079742e-05, 1.0000003780536528], [0.13974046050737837, 0.9901892130987151], [0.16626880069811453, 0.98608147676283431], [0.22318381431397741, 0.9747776512035039], [0.15830152050569701, 0.98739148853931291], [1.0000000153258095, 0.0], [1.000000291676596, 1.1329848835360409e-229], [0.23270523505830443, 0.97254816031176705], [0.12445643695791821, 0.99222564645590561], [0.10680259103805674, 0.99428208996602574]], [[0.0, 0.0], [4.2111324408539871e-05, 0.50000018902682641], [0.13974046050737837, 0.49509460654935755], [0.16626880069811453, 0.49304073838141715], [0.22318381431397741, 0.48738882560175195], [0.22318381431397741, 0.49369574426965646], [1.0, 0.0], [1.0, 5.6649244176802043e-230], [1.0, 0.0], [0.23270523505830443, 0.4961128232279528], [0.23270523505830443, 0.49714104498301287]])), ([0.92503311311904579, 0.37988758567285158], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[1.0000006505482015, 8.3936675370212877e-06], [1.000000211519581, 3.7372043481012537e-216], [0.99982896776761498, 0.018507026638671882], [0.92503311311904579, 0.37988758567285158]], [[0.5389452641563056, 0.0], [0.76947295735225363, 4.1968337685106439e-06], [1.0, 1.8686021740506269e-216], [1.0, 3.7372043481012537e-216]])), ([0.12678062879448254, 0.9919317370285945], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.023845232039107423, 0.99971789412902134], [1.0000000848917956, 2.3426083981674312e-286], [0.23902688062109972, 0.97102062649031562], [0.22634008209993581, 0.97405636620244263], [0.121417058478651, 0.99260261829396401], [0.14414614855405855, 0.98955761606266701], [0.23362471440194363, 0.97232768976907114], [0.25448216633738224, 0.96708027975855482], [0.19817243827189912, 0.98016781397255148], [0.12678062879448254, 0.9919317370285945]], [[0.0, 0.0], [0.011922616019553712, 0.99971789412902134], [0.5000000424458978, 1.1713041990837156e-286], [0.11951344031054986, 0.99971789412902134], [0.1131700410499679, 0.99971789412902134], [0.060708529239325501, 0.99971789412902134], [0.072073074277029275, 0.99971789412902134], [0.11681235720097181, 0.99260261829396401], [0.12724108316869112, 0.99260261829396401], [0.09908621913594956, 0.99260261829396401], [0.063390314397241271, 0.99971789412902134]])), ([0.12310609908208545, 0.99239450384017858], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.10946404408263147, 0.9939928398505522], [1.0000002251814883, 7.0699859782996109e-126], [0.17904721560994435, 0.983842116909036], [0.073800485523401196, 0.99727347017861823], [0.12310609908208545, 0.99239450384017858]], [[0.0, 0.0], [0.10946404408263147, 0.4969964199252761], [1.0, 3.5349929891498054e-126], [1.0, 0.491921058454518], [1.0, 7.0699859782996109e-126]])), ([0.84350684803532694, 0.54334732041744516], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.84321157914437217, 0.53758295340083428], [0.84438999434892825, 0.53573157206427457], [0.84335045505850226, 0.53736880376912188], [1.0000000726374829, 1.4931458091723272e-190], [0.84446832902775415, 0.53560846478584756], [1.0000001067717832, 0.0], [0.84477053360264087, 0.53513003260947178], [0.84262204069828073, 0.538506686506381], [0.84478925783083136, 0.53510153331515953], [0.84350684803532694, 0.54334732041744516]], [[0.8521054652075405, 0.5432635491282871], [0.84321157914437217, 0.5404232512645607], [0.84824772977823437, 0.53949756059628085], [0.84772796013302143, 0.54031617644870455], [0.84438999434892825, 0.27163177456414356], [0.84828689711764738, 0.53943600695706739], [0.84446832902775415, 0.27163177456414356], [0.84843799940509068, 0.53919679086887951], [0.84736375295291055, 0.54088511781733406], [0.84844736151918587, 0.53918254122172327], [0.84780615662143366, 0.54330543477286608]])), ([0.64866045987000498, 0.7610785573496901], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[0.66481864374726751, 0.74700532416433718], [0.72001436497966487, 0.69396026476751049], [0.63437695701472985, 0.7730244887714689], [0.88481432752589972, 0.4659484578374487], [0.65426518918301202, 0.75627066045031011], [0.67259228242368119, 0.74001474708706205], [0.69204154750766833, 0.72196486636456259], [1.000000210514769, 0.0], [0.80157385024710193, 0.59875838501746004], [0.64866045987000498, 0.7610785573496901]], [[0.2295261844565868, 0.17176071551877525], [0.66481864374726751, 0.74700532416433718], [0.72001436497966487, 0.43286049014314287], [0.72001436497966487, 0.47239260214512208], [0.88481432752589972, 0.31885458667811195], [0.88481432752589972, 0.46401568798454268], [0.88481432752589972, 0.45588773130291865], [0.88481432752589972, 0.44686279094166892], [1.0, 0.085880357759387627], [0.88481432752589972, 0.38525955026811765], [1.0, 0.0]])), ([0.00010729110885456562, 1.0000006057320192], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[8.2994313754268577e-05, 1.0000016840433155], [9.435952959904727e-05, 1.0000005451644449], [1.000000310702871, 0.0], [7.6795625743072469e-05, 1.0000020421329014], [1.57577288083764e-05, 1.0000009590373413], [1.0000000735025674, 0.0], [2.7525203593445415e-05, 1.0000019831372409], [3.6052295040166357e-05, 1.000002275439478], [4.6226427256189407e-05, 1.0000010283208201], [0.00010729110885456562, 1.0000006057320192]], [[0.0, 0.9893548997434237], [4.1497156877134288e-05, 1.0], [4.7179764799523635e-05, 1.0], [0.50000015535143549, 1.0], [3.8397812871536235e-05, 1.0], [7.8788644041881998e-06, 1.0], [0.50000003675128368, 1.0], [1.3762601796722707e-05, 1.0], [1.8026147520083178e-05, 1.0], [2.3113213628094703e-05, 1.0], [5.364555442728281e-05, 1.0]])), ([0.23960945865081854, 0.97087021149256825], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[1.000000317238984, 0.0], [0.23893727609926488, 0.97103698466839616], [0.23959430353935568, 0.97087379040527466], [0.24093390337305662, 0.9705437557943476], [0.24214496591481877, 0.9702409698140716], [0.23990055057108439, 0.97079979384974668], [0.23763942512492345, 0.97135603184816799], [0.23961927342173453, 0.97086790998419215], [0.24074830834041375, 0.97058836998009501], [0.23960945865081854, 0.97087021149256825]], [[0.21536704667589168, 0.875015209332337], [1.0, 0.4375076046661685], [0.23893727609926488, 0.92302609700036653], [0.23959430353935568, 0.92294449986880589], [0.24093390337305662, 0.9227794825633423], [0.24214496591481877, 0.9226280895732043], [0.24214496591481877, 0.92290750159104185], [0.24214496591481877, 0.92318562059025244], [0.24214496591481877, 0.92294155965826463], [0.24214496591481877, 0.92280178965621595], [0.24214496591481877, 0.92294271041245257]]))], (4, 'ACH_solution', 'DTLZ4'): [([0.3022689144408573, 3.4791933533647654e-62, 0.9260398314822165, 0.2311524719586939], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.30146908138371636, 0.09351055619359032, 0.9252401967211046, 0.22975858722161402], [0.28028770488844196, 0.22502626083275362, 0.9187193616187558, 0.22502604378653862], [0.30324223969943276, 9.294320528693865e-13, 0.927013308598561, 0.23252515206398303], [0.28694672949294153, 0.21706924038414885, 0.910762152793687, 0.21343186515995174], [0.286326157584536, 0.21640383257664894, 0.9100972876583452, 0.21496370211625854], [0.2861830145053133, 0.21680915668675496, 0.9105020550304905, 0.21479753445632083], [0.30195262670026946, 1.9444821511889767e-36, 0.9257237705534865, 0.2320107073498876], [0.3017451192482147, 3.824431266540302e-19, 0.9255162590549479, 0.23078828628377315], [0.28682947984451296, 0.21563466960305883, 0.9106006311306312, 0.21426249629898816], [0.3022689144408573, 3.4791933533647654e-62, 0.9260398314822165, 0.2311524719586939]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.30146908138371636, 0.093510556193590319, 0.92524019672110458, 0.22975858722161402], [0.30146908138371636, 0.093510556193590319, 0.8062062391388319, 0.22975858722161402], [0.30324223969943276, 4.647160264346933e-13, 0.8103532126287345, 0.22502604378653862], [0.28694672949294153, 0.21706924038414885, 0.8022276347262975, 0.21343186515995174], [0.286326157584536, 0.21706924038414885, 0.8018952021586266, 0.21496370211625854], [0.286326157584536, 0.21706924038414885, 0.910762152793687, 0.21343186515995174], [0.30146908138371636, 9.722410755944884e-37, 0.8097084436061973, 0.21496370211625854], [0.286326157584536, 1.912215633270151e-19, 0.8096046878569281, 0.21496370211625854], [0.286326157584536, 0.21706924038414885, 0.8021468738947697, 0.21496370211625854], [0.286326157584536, 1.7395966766823827e-62, 0.8098664740705623, 0.21496370211625854]])), ([0.4590098481194116, 0.47444923960679247, 0.5497777901196822, 0.5217497747245882], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.45826230007759944, 0.4737086094477361, 0.5490237679975424, 0.5209939506889254], [0.45839365206882127, 0.473839846236798, 0.549154589988077, 0.5211267600533948], [0.45692446114422813, 0.47237055933722066, 0.5476855280312078, 0.5196574793455611], [0.525224147778232, 6.759384626944425e-24, 0.6159854229181768, 0.5879572094449641], [0.45759271932802187, 0.4729948334874011, 0.5483539811781792, 0.5203245905094238], [0.45716428828322275, 0.47261042549882654, 0.5479254418946792, 0.5198973017874013], [0.5239794514653741, 0.08649982679941703, 0.614740957966984, 0.5867125208270849], [0.45844623013960245, 0.47342931421063045, 0.5492077366998305, 0.5211574835607938], [0.4571218000216859, 0.47256781686097954, 0.5478830494011633, 0.5198542783498694], [0.4590098481194116, 0.47444923960679247, 0.5497777901196822, 0.5217497747245882]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.45826230007759944, 0.38261526606468593, 0.54902376799754238, 0.5209939506889254], [0.45826230007759944, 0.3826808844592169, 0.54902376799754238, 0.5209939506889254], [0.45692446114422813, 0.3819462410094282, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.1457609613408179, 0.61598542291817682, 0.51965747934556106], [0.45692446114422813, 0.38225837808451846, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.38206617409023114, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.1890108747405264, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.47237055933722066, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.3820448697713077, 0.54768552803120785, 0.51965747934556106], [0.45692446114422813, 0.47237055933722066, 0.54768552803120785, 0.51965747934556106]])), ([0.3399807534277755, 1.5783675429829543e-32, 0.709737958019554, 0.6212730004568426], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.33929205998576445, 8.522706644614764e-12, 0.7090492674399337, 0.6205932732856494], [0.339789913330473, 4.173900777541963e-40, 0.7095469639819754, 0.6210844298604153], [0.3400333455454999, 1.018257130809358e-19, 0.7097918596342817, 0.6213393441636031], [0.33876329668404687, 4.167875518310206e-31, 0.7085204575329588, 0.6200060413490631], [0.3397196503000077, 2.4825705992912225e-12, 0.7094770176324406, 0.6200373251820714], [0.33940502626283503, 3.3461937271540206e-136, 0.7091622258617511, 0.6207091330733923], [0.3396885392004458, 4.925792625483946e-33, 0.7094455600993266, 0.6209928793473781], [0.33969565614270575, 3.619098690615024e-17, 0.7094526484346079, 0.6209999851112727], [0.33915630968781735, 1.2434326184255378e-30, 0.708913421674918, 0.6204604484688359], [0.3399807534277755, 1.5783675429829543e-32, 0.709737958019554, 0.6212730004568426]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.33929205998576445, 8.5227066446147636e-12, 0.70904926743993368, 0.7253124109734591], [0.33929205998576445, 8.5227066446147636e-12, 0.70904926743993368, 0.62059327328564939], [0.34003334554549991, 1.0182571308093581e-19, 0.70979185963428171, 0.725685446412436], [0.33876329668404687, 4.1678755183102064e-31, 0.70852045753295878, 0.725018795005166], [0.33876329668404687, 4.1678755183102064e-31, 0.70852045753295878, 0.62000604134906312], [0.34003334554549991, 8.5227066446147636e-12, 0.70979185963428171, 0.7253703408673305], [0.33929205998576445, 8.5227066446147636e-12, 0.70916222586175115, 0.7255122140043235], [0.33876329668404687, 4.1678755183102064e-31, 0.70852045753295878, 0.7255157668862708], [0.33876329668404687, 4.1678755183102064e-31, 0.70852045753295878, 0.7252459985650523], [0.33876329668404687, 4.1678755183102064e-31, 0.70852045753295878, 0.62000604134906312]])), ([0.37985459330207566, 0.7722716069265464, 0.36880358346551917, 0.36881461914457386], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.3806554652099985, 0.7730726785004676, 0.3694860783588084, 0.36196293537047697], [0.3804574583807565, 0.7723901498866186, 0.36849532601602886, 0.3694184505776778], [0.3791240976221356, 0.7715412669632804, 0.36351145005782565, 0.3680797780591599], [0.3792425579175303, 0.7716598365787943, 0.3682032734519021, 0.36820347431918055], [0.37814528312777973, 0.7705506043442094, 0.367103166775781, 0.3671062750430261], [0.3794498630373362, 0.7718671014004932, 0.3654169045651786, 0.3677287990251043], [0.3776153368233964, 0.7699348823357725, 0.36656796459160423, 0.36657633327461625], [0.3780273540424033, 0.7704442509800545, 0.36696059281921595, 0.3669869954215189], [0.3791886697870047, 0.7714889942527744, 0.36814953934431727, 0.3679339855237029], [0.37985459330207566, 0.7722716069265464, 0.36880358346551917, 0.36881461914457386]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.38065546520999849, 0.77307267850046757, 0.1847430391794042, 0.36196293537047697], [0.38065546520999849, 0.77239014988661858, 0.18424766300801443, 0.3694184505776778], [0.38065546520999849, 0.77307267850046757, 0.18175572502891282, 0.36196293537047697], [0.38065546520999849, 0.77307267850046757, 0.18410163672595106, 0.36807977805915992], [0.37814528312777973, 0.77055060434420941, 0.1835515833878905, 0.36710627504302612], [0.37814528312777973, 0.77055060434420941, 0.1827084522825893, 0.36807977805915992], [0.37761533682339637, 0.76993488233577245, 0.36948607835880842, 0.36657633327461625], [0.37912409762213561, 0.7715412669632804, 0.18348029640960797, 0.36807977805915992], [0.37761533682339637, 0.76993488233577245, 0.18407476967215863, 0.36657633327461625], [0.38065546520999849, 0.77307267850046757, 0.36948607835880842, 0.36196293537047697]])), ([0.2155602554399012, 0.977314598848869, 4.309392212481999e-50, 5.209628703444445e-25], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.21782957878928527, 0.9770099301325693, 6.621230037485446e-16, 1.366343574704681e-51], [0.21679126563995044, 0.9770852360660368, 7.278334469923606e-20, 7.497688595354096e-09], [0.21758217224066612, 0.9773162903905772, 2.9868856479070686e-134, 2.238869139536191e-24], [0.2142591898932107, 0.9771398124403688, 7.323812539813698e-42, 5.5868251163526885e-45], [0.21488031959772919, 0.9773155348825644, 3.533448759320125e-14, 1.81430842912938e-64], [0.21900359929215144, 0.9763823148128216, 4.829729755493483e-77, 1.0067825777145736e-47], [0.21672893203816207, 0.9773155964185427, 2.5190908250740233e-121, 2.028121916409797e-30], [0.2182038682648009, 0.9769351268173986, 2.0703746234345815e-18, 3.0204538983249518e-27], [0.21607389408479705, 0.977149818014104, 1.4885908486849237e-67, 1.3329014222374897e-10], [0.2155602554399012, 0.977314598848869, 4.309392212481999e-50, 5.209628703444445e-25]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.3060816709946015, 0.9771632421546133, 3.310615018742723e-16, 6.831717873523404e-52], [0.3055625144199341, 0.9772008951213469, 3.639167234961803e-20, 3.748844297677048e-09], [0.30595796772029193, 0.97700993013256932, 1.4934428239535343e-134, 1.1194345697680954e-24], [0.3042964765465642, 0.977228183308513, 3.661906269906849e-42, 2.7934125581763442e-45], [0.30460704139882344, 0.9773160445296107, 1.7667243796600624e-14, 9.0715421456469e-65], [0.3066686812460346, 0.9768494344947394, 2.4148648777467415e-77, 5.033912888572868e-48], [0.3055313476190399, 0.9773160752975999, 1.2595454125370117e-121, 1.0140609582048985e-30], [0.3062688157323593, 0.9771258404970278, 1.0351873117172907e-18, 1.5102269491624759e-27], [0.3052038286423574, 0.9772331860953806, 7.442954243424618e-68, 6.664507111187449e-11], [0.3049470093199095, 0.977315576512763, 2.1546961062409993e-50, 2.2388691395361909e-24]])), ([0.037516994220678064, 2.6706345339312987e-22, 0.9988530299344657, 0.037037012129050084], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[0.03810618948416051, 1.9543139338910887e-24, 0.9993191158417347, 0.034625830478047744], [0.03764064474337869, 7.371325640837224e-13, 0.9994098383364838, 2.1940774393047256e-17], [0.03827248724317382, 2.158538881332362e-07, 0.9994854320577602, 6.62845119816681e-104], [0.03826309004107937, 3.0391985003968864e-35, 0.9994760052878419, 4.3649832668726017e-69], [0.03835741872429209, 5.222934004037676e-189, 0.9995703280037226, 2.7268485092738946e-16], [0.03803463818984247, 8.768614283537221e-14, 0.999374724909503, 2.8692722926565365e-14], [0.03816096034855738, 2.1071877489789227e-39, 0.9993760697541791, 7.137668965981298e-30], [0.03828868240749154, 1.981270429619667e-46, 0.9995016041371748, 1.958170869891077e-20], [0.038263607480897674, 1.7240236355920934e-22, 0.9994765394541654, 7.961274690605909e-30], [0.037516994220678064, 2.6706345339312987e-22, 0.9988530299344657, 0.037037012129050084]], [[0.0, 0.0, 0.9612129216540486, 0.0], [0.038106189484160509, 9.771569669455444e-25, 0.9802660187478917, 0.034625830478047744], [0.038106189484160509, 3.685662820418612e-13, 0.9803113799952662, 1.0970387196523628e-17], [0.037640644743378693, 1.079269440666181e-07, 0.9803491768559044, 3.314225599083405e-104], [0.037640644743378693, 7.3713256408372238e-13, 0.9803444634709453, 2.1824916334363008e-69], [0.037640644743378693, 2.611467002018838e-189, 0.9803916248288855, 1.3634242546369473e-16], [0.038106189484160509, 4.3843071417686105e-14, 0.9802938232817757, 0.034625830478047744], [0.038034638189842471, 1.0535938744894614e-39, 0.99937472490950296, 3.568834482990649e-30], [0.038034638189842471, 9.906352148098334e-47, 0.9803572628956116, 9.790854349455385e-21], [0.038106189484160509, 8.620118177960467e-23, 0.980344730554107, 3.9806373453029544e-30], [0.038106189484160509, 2.6706345339312987e-22, 0.9800329757942572, 0.037037012129050084]])), ([0.1658949124117661, 0.9411197647944044, 0.1703854319422132, 0.2590451195061171], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[0.16825292130006714, 0.9389869963579555, 0.1629668272350419, 0.2565202032721218], [0.1674287573755651, 0.9381629273754939, 0.16463061916650823, 0.25658435055338896], [0.1676674869533039, 0.9384014663526604, 0.16688430534707482, 0.25558206142876844], [0.177614036086923, 0.9483480654753591, 1.980619248959184e-08, 0.26681827766973076], [0.1764718518927319, 0.9488130978837365, 1.423460163589946e-24, 0.2643816627700219], [0.16796591929613533, 0.9387000898462627, 0.16771568651209928, 0.2561497114789906], [0.17777447700039228, 0.9485085901856659, 1.960157068513765e-37, 0.26681266468448456], [0.17795687385612866, 0.9486910440190806, 4.2892325308666184e-70, 0.26715813998871135], [0.16797756784443638, 0.9387115587751425, 0.16572415874092827, 0.25716530318667685], [0.1658949124117661, 0.9411197647944044, 0.1703854319422132, 0.2590451195061171]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.16825292130006714, 0.93898699635795546, 0.16296682723504191, 0.1728626675437304], [0.16742875737556509, 0.93898699635795546, 0.16463061916650823, 0.17289474118436396], [0.16766748695330391, 0.93840146635266042, 0.16688430534707482, 0.1723935966220537], [0.16766748695330391, 0.9483480654753591, 9.90309624479592e-09, 0.17801170474253486], [0.16766748695330391, 0.93840146635266042, 7.11730081794973e-25, 0.17679339729268043], [0.16825292130006714, 0.93898699635795546, 0.16296682723504191, 0.1726774216471648], [0.16742875737556509, 0.93816292737549389, 9.800785342568824e-38, 0.17800889824991176], [0.16742875737556509, 0.93816292737549389, 2.1446162654333092e-70, 0.17818163590202515], [0.16742875737556509, 0.93816292737549389, 0.16463061916650823, 0.1731852175010079], [0.16766748695330391, 0.93840146635266042, 0.16688430534707482, 0.17412512566072802]])), ([0.5747967944887711, 5.977764780806632e-40, 0.7324628797891356, 0.37091080417062994], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.5764822118479451, 3.055358081026817e-54, 0.7341482806494637, 0.37401643778619864], [0.5752177359982257, 3.52406173794207e-75, 0.7328834649095627, 0.37274157648523015], [0.5751898470786166, 4.973180181329383e-52, 0.7312032298055416, 0.3727884330815474], [0.5745616802464746, 0.37216035531227304, 0.732227419181925, 2.277956115847708e-18], [0.5401994920916263, 0.33651100669999123, 0.6978648292817908, 0.33779842270950877], [0.573533940803802, 1.40975798949591e-122, 0.7312000290821028, 0.37112556450758155], [0.5748085172200139, 5.6164788382015e-25, 0.7323245123195614, 0.37240747696531673], [0.5393260008329542, 0.3347193943697467, 0.6969895431223053, 0.3369249164058972], [0.5746667942845476, 8.410846693487149e-43, 0.7323327511705641, 0.3722655284290372], [0.5747967944887711, 5.977764780806632e-40, 0.7324628797891356, 0.37091080417062994]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.38944162587546366, 1.5276790405134084e-54, 0.73414828064946369, 0.37401643778619864], [0.38880938795060394, 1.762030868971035e-75, 0.73288346490956269, 0.37274157648523015], [0.3887954434907994, 2.4865900906646916e-52, 0.73288346490956269, 0.37274157648523015], [0.3884813600747284, 0.37216035531227304, 0.73222741918192502, 1.138978057923854e-18], [0.37130026599730426, 0.37216035531227304, 0.73222741918192502, 2.277956115847708e-18], [0.3879674903533921, 7.04878994747955e-123, 0.7312000290821028, 0.37112556450758155], [0.38860477856149805, 2.80823941910075e-25, 0.69786482928179083, 0.33779842270950877], [0.3708635203679682, 0.33471939436974668, 0.69698954312230532, 0.33692491640589722], [0.3885339170937649, 4.2054233467435746e-43, 0.73222741918192502, 0.37112556450758155], [0.38859891719587664, 2.988882390403316e-40, 0.69698954312230532, 0.37112556450758155]])), ([0.48103471557735494, 0.48489614977386003, 0.48102192350564177, 0.5578513528532523], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[0.4818576881146068, 0.4857191870257872, 0.48185733367798006, 0.5586751937816267], [0.5513475654258797, 0.15165836629050755, 0.5513475588014402, 0.6281648398200863], [0.4814473383573966, 0.48530856943630585, 0.481445875897092, 0.5582473870145681], [0.48354156280536115, 0.4874030602473243, 0.48235606224667416, 0.5562327714975986], [0.48187843792695645, 0.4857390268535877, 0.4818781982732852, 0.5586963398690629], [0.483162453124198, 0.4870269572982677, 0.48316540977235817, 0.5599834481715329], [0.4848020768836189, 0.4886636450798268, 0.4848018535293955, 0.5616196250067871], [0.4800403606530642, 0.4839018871172811, 0.48004031057395985, 0.556857977844263], [0.48526407733577875, 0.4891255277935281, 0.48399536077208416, 0.562028582683813], [0.48103471557735494, 0.48489614977386003, 0.48102192350564177, 0.5578513528532523]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.48185768811460677, 0.2447903870944661, 0.48185733367798006, 0.55867519378162667], [0.48185768811460677, 0.07775997672682627, 0.55134755880144015, 0.55867519378162667], [0.48144733835739661, 0.48530856943630585, 0.48144587589709198, 0.55824738701456811], [0.5513475654258797, 0.24563232370523466, 0.48235606224667416, 0.62816483982008631], [0.48144733835739661, 0.24480030700836636, 0.48144587589709198, 0.62816483982008631], [0.48144733835739661, 0.24544427223070636, 0.48144587589709198, 0.55824738701456811], [0.48354156280536115, 0.2462626161214859, 0.48187819827328521, 0.62816483982008631], [0.48004036065306421, 0.24388173714021305, 0.48235606224667416, 0.55824738701456811], [0.48354156280536115, 0.24649355747833654, 0.55134755880144015, 0.62816483982008631], [0.5513475654258797, 0.2443788684685025, 0.55134755880144015, 0.62816483982008631]])), ([0.16995868306628956, 0.7659547966053127, 0.17015204208830698, 0.5993556787646566], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[0.1705951779967896, 0.7663977316545314, 0.1675510383024968, 0.5997983454908353], [0.17949049912862253, 0.7753074279547831, 4.183789819696149e-26, 0.6087084308984222], [0.16959606773175928, 0.7653986825814331, 0.16807379508974038, 0.598669838756298], [0.17943019589299497, 0.7752326462353586, 7.140929527889109e-18, 0.6086310141898683], [0.180418039135959, 0.7762205606964312, 0.0002033777801962094, 0.6095653632676981], [0.17910704809064557, 0.7749101691016179, 2.037730843471489e-44, 0.6083111403149827], [0.17123458887604867, 0.7670371366780159, 0.16902508176291806, 0.5999746417253706], [0.17556255465152654, 0.7713650601446814, 0.11324830508511873, 0.6047376003616387], [0.17276190390803217, 0.7685643710357539, 0.1687169207456014, 0.601806348379175], [0.16995868306628956, 0.7659547966053127, 0.17015204208830698, 0.5993556787646566]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.0852975889983948, 0.7663977316545314, 0.16755103830249679, 0.59979834549083533], [0.08974524956431126, 0.7663977316545314, 2.0918949098480746e-26, 0.60870843089842219], [0.16959606773175928, 0.76539868258143307, 0.16807379508974038, 0.59866983875629798], [0.17943019589299497, 0.77523264623535859, 3.570464763944554e-18, 0.60863101418986831], [0.0902090195679795, 0.76539868258143307, 0.0001016888900981047, 0.60863101418986831], [0.08955352404532278, 0.76539868258143307, 1.0188654217357445e-44, 0.60831114031498268], [0.08561729443802434, 0.77491016910161792, 0.16807379508974038, 0.59866983875629798], [0.08778127732576327, 0.77491016910161792, 0.16807379508974038, 0.59866983875629798], [0.08638095195401609, 0.76539868258143307, 0.16807379508974038, 0.59866983875629798], [0.08497934153314478, 0.76539868258143307, 0.16807379508974038, 0.59866983875629798]]))], (3, 'rNSGAII_solution', 'DTLZ1'): [([0.81924861070081523, 0.0, 1.0380645865056293], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.22062551158643612, 6.6134751151400921e-16, 0.29412262396325467], [0.20319152591939507, 2.2558791048443172e-17, 0.29933736600798294], [0.2249451670047154, 6.9177787121241875e-15, 0.28575305196913686], [0.21980476505382232, 6.3692433193206814e-15, 0.28470612556738345], [0.22353614127258159, 7.4452491263004571e-17, 0.28399343927590043], [0.44709937602611927, 4.8803090960243068e-12, 0.2140263624491949], [0.42013116632852476, 3.0696369952676792e-13, 0.58239973476789975], [0.20693319649043979, 2.8711822573272711e-11, 0.30467654334258831], [0.40207971386510344, 0.0, 0.60506091570176768], [0.81924861070081523, 0.0, 1.0380645865056293]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.22062551158643612, 3.306737557570046e-16, 0.49644095721865245], [0.20319152591939507, 1.1279395524221586e-17, 0.49904832824101653], [0.20319152591939507, 3.4588893560620937e-15, 0.49225617122159349], [0.20319152591939507, 3.1846216596603407e-15, 0.49173270802071678], [0.20319152591939507, 3.7226245631502286e-17, 0.49137636487497527], [0.20319152591939507, 2.4401545480121534e-12, 0.45639282646162255], [0.42013116632852476, 1.5348184976338396e-13, 0.64057951262097501], [0.42013116632852476, 1.4355911286636355e-11, 0.50171791690831924], [0.40207971386510344, 0.0, 0.65191010308790887], [0.81924861070081523, 0.0, 0.86841193848983966]])), ([0.0, 0.0, 0.75134427727930841], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[1.023783720896614e-13, 2.9636288875034809e-19, 1.0010112817665255], [2.6454037437809437e-22, 1.7718179700465341e-16, 0.50762516902046406], [4.9842437049997034e-14, 1.8955557171995265e-12, 0.50444554957082821], [0.0, 0.0, 0.50487841897838237], [7.9120240997339985e-18, 1.5557777780807912e-14, 0.50288783689733174], [0.0, 0.0, 0.50050916184699545], [2.2863335353313416e-24, 7.1011427599267516e-18, 0.81793575533498597], [5.5037500373031544e-17, 1.1119321906542484e-12, 1.002181377486453], [0.0, 0.0, 0.50227463458197796], [0.0, 0.0, 0.75134427727930841]], [[0.0, 0.0, 0.9829301429354526], [5.1189186044830702e-14, 2.9636288875034809e-19, 1.0], [1.3227018718904718e-22, 8.8590898502326703e-17, 0.50762516902046406], [2.4921218524998517e-14, 9.4777785859976324e-13, 0.50444554957082821], [0.0, 0.0, 0.50444554957082821], [3.9560120498669992e-18, 7.7788888904039558e-15, 0.50288783689733174], [0.0, 0.0, 0.50050916184699545], [1.1431667676656708e-24, 3.5505713799633758e-18, 0.50050916184699545], [2.7518750186515772e-17, 5.5596609532712421e-13, 0.50050916184699545], [0.0, 0.0, 0.50050916184699545]])), ([0.19505098069055041, 1.0216548550864073e-08, 0.30501843152363839], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.11148238292745934, 0.38846133996177917, 0.0031365424400087239], [3.0076372897117382e-10, 0.25061970279524032, 0.25110726798942101], [0.25447794378916949, 0.25203577136138317, 0.0011091656375080154], [0.00202814965205267, 0.48778030956077933, 0.011153817430513906], [0.0014957436811307784, 0.50091791513836526, 0.0070030398051019243], [0.44216053377265069, 0.0088646951692628335, 0.057738654150430661], [0.49942423270864145, 0.00062520629251726844, 0.0013712803454089199], [0.0010225559959046514, 0.4567592619389077, 0.045823516094020952], [0.24482806262465873, 0.253507235075425, 0.0031267800054436807], [0.19505098069055041, 1.0216548550864073e-08, 0.30501843152363839]], [[0.0, 0.0, 0.0], [0.11148238292745934, 0.38846133996177917, 0.0015682712200043619], [1.5038186448558691e-10, 0.38846133996177917, 0.12555363399471051], [0.25447794378916949, 0.25203577136138317, 0.0005545828187540077], [0.001014074826026335, 0.25061970279524032, 0.0055769087152569528], [0.11148238292745934, 0.38846133996177917, 0.0035015199025509622], [0.44216053377265069, 0.0044323475846314167, 0.028869327075215331], [0.49942423270864145, 0.00031260314625863422, 0.00068564017270445995], [0.00051127799795232568, 0.25061970279524032, 0.022911758047010476], [0.49942423270864145, 0.00062520629251726844, 0.0015633900027218403], [0.19505098069055041, 5.1082742754320367e-09, 0.15250921576181919]])), ([0.56462610702487659, 0.43741860502172419, 1.1124931110180213e-16], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[0.31219181220599984, 0.19412889745753217, 0.0], [0.27349381322417105, 0.23270214389196037, 2.1912005951881261e-13], [0.56490552522473358, 0.43935064315948058, 1.2866503621595044e-13], [0.74921566976299614, 0.21624921923956852, 1.0718813492378185e-16], [0.57472549192344524, 0.43624761178976801, 5.9487497702016134e-15], [0.29776751581937794, 0.21834323089346014, 4.0109862397995295e-16], [0.81673344090631372, 0.18466422198332177, 0.0], [0.55226206645906539, 0.45130532422648867, 1.2256019862923027e-15], [0.55665341840172833, 0.45173002548097313, 1.6333950243753171e-13], [0.56462610702487659, 0.43741860502172419, 1.1124931110180213e-16]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.4878753845125422, 0.19412889745753217, 0.0], [0.4685263850216278, 0.19412889745753217, 1.0956002975940631e-13], [0.61423224102190899, 0.43935064315948058, 6.433251810797522e-14], [0.70638731329104032, 0.21624921923956852, 5.3594067461890925e-17], [0.61914222437126487, 0.43624761178976801, 5.9487497702016134e-15], [0.48066323631923125, 0.43935064315948058, 2.0054931198997648e-16], [0.74014619886269917, 0.21624921923956852, 0.0], [0.60791051163907495, 0.43624761178976801, 6.1280099314615137e-16], [0.61010618761040636, 0.43624761178976801, 8.1669751218765854e-14], [0.61409253192198054, 0.43624761178976801, 5.5624655550901067e-17]])), ([7.9709868100371513e-08, 0.0090474695819126975, 0.49359154631169305], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[1.1235691113117064e-11, 0.062163417219605555, 0.44460146420467578], [2.5737838139245443e-08, 0.076820918327616575, 0.43621180850599656], [5.0081958479612245e-06, 0.0021656388900021719, 0.4983477412438343], [0.00095532830154711624, 3.0986433303266272e-10, 0.5019206509311045], [8.6832632492098424e-08, 0.0032958036171941755, 0.49822931074702365], [0.0036243343921812865, 6.2448978465511401e-12, 0.49641391517734718], [0.025174104081246818, 1.7739555839331731e-11, 0.47573269320110678], [1.1328600899332898e-11, 0.012132242587520818, 0.48821817102233683], [0.0033774730549450608, 3.4057592384772585e-07, 0.4977095760168781], [7.9709868100371513e-08, 0.0090474695819126975, 0.49359154631169305]], [[0.0, 0.0, 0.2524577834161418], [5.6178455565585319e-12, 0.062163417219605555, 0.34852962381040875], [1.2868919069622722e-08, 0.076820918327616575, 0.3443347959610692], [2.5040979239806122e-06, 0.076820918327616575, 0.37540276232998804], [0.00047766415077355812, 1.5493216651633136e-10, 0.37718921717362314], [4.3416316246049212e-08, 0.0016479018085970877, 0.37534354708158268], [0.0018121671960906432, 3.0986433303266272e-10, 0.37443584929674445], [2.5737838139245443e-08, 8.8697779196658654e-12, 0.36409523830862428], [5.664300449666449e-12, 0.076820918327616575, 0.37033797721923933], [2.5737838139245443e-08, 0.076820918327616575, 0.37508367971650991], [2.5737838139245443e-08, 0.0045237347909563488, 0.37302466486391739]])), ([2.2539312289211438e-10, 0.47140693732629313, 0.02940707881862344], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[2.9184145555033497e-11, 0.4645519629375745, 0.036909398196787527], [0.00092607875149968864, 0.4653036376553627, 0.035468141272476834], [1.0716619025306049e-08, 0.47164875710434634, 0.029789173462856294], [9.5243655203437345e-06, 0.46815032048307015, 0.034676641496679629], [0.0006389480597663161, 0.47092546117786532, 0.030902187295081408], [4.5857378253864501e-05, 0.47148353490376449, 0.029149051981461609], [0.01901697256940519, 0.45195136171914241, 0.029338272745392104], [4.2590164769822327e-12, 0.45223939558675685, 0.048825287864959932], [8.8865391035773759e-05, 0.47265054825517172, 0.029263429872781445], [2.2539312289211438e-10, 0.47140693732629313, 0.02940707881862344]], [[0.0, 0.35648910413162704, 0.027090592880264497], [1.4592072777516749e-11, 0.4645519629375745, 0.031999995538526012], [0.00046303937574984432, 0.4645519629375745, 0.031279367076370662], [5.3583095126530246e-09, 0.47164875710434634, 0.028439883171560397], [4.7621827601718673e-06, 0.47164875710434634, 0.030883617188472063], [0.00031947402988315805, 0.47164875710434634, 0.028996390087672953], [2.292868912693225e-05, 0.47148353490376449, 0.028119822430863053], [4.5857378253864501e-05, 0.47148353490376449, 0.028214432812828302], [2.1295082384911163e-12, 0.4645519629375745, 0.037957940372612214], [4.4432695517886879e-05, 0.47148353490376449, 0.028177011376522971], [1.1269656144605719e-10, 0.47140693732629313, 0.028248835849443966]])), ([0.29753521776393949, 4.9027645299789164e-13, 0.21317745956247752], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.59821410678522891, 0.0, 0.4038567795816832], [0.74912757695190579, 1.7465672370586891e-15, 0.38320509717754686], [0.2878496897546492, 3.9496117678719755e-11, 0.21407134762632613], [0.28944164562479191, 6.2697579882394156e-12, 0.21152852364678573], [0.61870754894765623, 3.0223748119447841e-15, 0.3843497534535294], [0.29782230219153433, 1.9838950628394889e-15, 0.21033601830740475], [0.29155727760511763, 0.0, 0.21073150375412109], [0.31498114846500613, 0.0, 0.18824827232195931], [0.30669984674547507, 2.383536620540835e-16, 0.19868701572728067], [0.29753521776393949, 4.9027645299789164e-13, 0.21317745956247752]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.59821410678522891, 0.0, 0.4038567795816832], [0.59821410678522891, 8.7328361852934456e-16, 0.45688988826746524], [0.74912757695190579, 1.9748058839359878e-11, 0.37232301349185482], [0.74912757695190579, 3.1348789941197078e-12, 0.37105160150208466], [0.59821410678522891, 1.511187405972392e-15, 0.45746221640545648], [0.74912757695190579, 9.9194753141974445e-16, 0.37045534883239417], [0.74912757695190579, 0.0, 0.37065309155575232], [0.74912757695190579, 0.0, 0.35941147583967142], [0.74912757695190579, 1.1917683102704175e-16, 0.3646308475423321], [0.74912757695190579, 2.4513822649894582e-13, 0.37187606945993057]])), ([0.0036163394319147549, 0.4982865473087843, 1.3206212141353291e-14], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.0043484857990254188, 0.49899839865079726, 1.2271847518780025e-12], [0.0047433581045020776, 0.49638399176450548, 1.4082385165276968e-11], [0.0047201867450253976, 0.4967744382199677, 7.948850813511932e-12], [0.00097586407463438787, 0.50360206650608286, 8.5388136183652564e-11], [0.0046922546538863321, 0.49579902603754011, 7.3532726054626478e-08], [0.0039432720780187185, 0.49705325708579529, 4.0715148957642017e-14], [0.0034423906517135239, 0.49770144786266368, 1.7526014987282937e-14], [8.1507210129379765e-06, 0.50060640374819887, 0.00023412096733715218], [0.0048602208141276101, 0.50252755833905793, 4.0896003671341296e-12], [0.0036163394319147549, 0.4982865473087843, 1.3206212141353291e-14]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.0054901064378292414, 0.49899839865079726, 6.1359237593900123e-13], [0.0056875425905675708, 0.49638399176450548, 7.0411925826384839e-12], [0.0056759569108292308, 0.49638399176450548, 3.974425406755966e-12], [0.0038037955756337265, 0.49899839865079726, 4.2694068091826282e-11], [0.0056619908652596985, 0.49638399176450548, 3.6766363027313239e-08], [0.0052874995773258913, 0.49638399176450548, 2.0357574478821009e-14], [0.0050370588641732942, 0.49638399176450548, 8.7630074936414687e-15], [0.0033199388988230014, 0.49770144786266368, 0.00011706048366857609], [0.005745973945380338, 0.50252755833905793, 2.0448001835670648e-12], [0.0046922546538863321, 0.49579902603754011, 6.6031060706766455e-15]])), ([6.0220414084229908e-17, 4.2851040197699128e-18, 0.50070175134818651], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[2.8767260739940392e-14, 9.4832505547746507e-19, 0.50080421355563198], [7.0313623079269515e-17, 4.0824050020381197e-23, 0.514920311260047], [0.0, 0.0, 0.50583503054536472], [5.7288472648818222e-19, 2.7680893738477321e-16, 0.66353739136198941], [2.0238668496439046e-17, 4.8999638558059432e-19, 0.50068983678356282], [0.0, 0.0, 0.61539268586320262], [7.6158428839453672e-17, 3.5058861114052642e-18, 0.5024576094270311], [1.999555167627296e-17, 2.8387260159209731e-16, 0.50122757011262709], [1.8800765780545357e-11, 0.05400072941465086, 0.94853554559409148], [6.0220414084229908e-17, 4.2851040197699128e-18, 0.50070175134818651]], [[0.0, 0.0, 0.8177116892877433], [1.4383630369970196e-14, 4.7416252773873253e-19, 0.50080421355563198], [3.5156811539634758e-17, 2.0412025010190599e-23, 0.50080421355563198], [0.0, 0.0, 0.50080421355563198], [2.8644236324409111e-19, 1.384044686923866e-16, 0.50080421355563198], [1.0119334248219523e-17, 2.4499819279029716e-19, 0.50068983678356282], [0.0, 0.0, 0.50068983678356282], [3.8079214419726836e-17, 1.7529430557026321e-18, 0.50068983678356282], [9.9977758381364799e-18, 1.4193630079604866e-16, 0.50068983678356282], [9.4003828902726783e-12, 4.8999638558059432e-19, 0.50068983678356282], [3.0110207042114954e-17, 2.1425520098849564e-18, 0.50068983678356282]])), ([2.5900293909116993e-15, 0.57757079086437968, 0.4254073383846832], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[1.8478628132871261e-16, 0.27174437119656192, 0.2294743419680903], [0.0, 0.34886364388584984, 0.21744427757140386], [1.4765217224843556e-17, 0.33142254512060643, 0.18281642036297527], [7.5753482923482831e-16, 0.27572510905606828, 0.23574597220160551], [8.7911677773385624e-15, 0.27973842525071979, 0.2265508825187407], [4.7429136442392423e-14, 0.36375087004209627, 0.15586699182911357], [1.2400345355345051e-16, 0.33608759833161811, 0.22804191494317277], [2.7139733016375431e-15, 0.27876896024097048, 0.23096820745779131], [5.2195416110889374e-15, 0.27883388532110698, 0.22609449147541916], [2.5900293909116993e-15, 0.57757079086437968, 0.4254073383846832]], [[0.0, 0.6790416599958826, 0.5290286140740371], [9.2393140664356305e-17, 0.27174437119656192, 0.2294743419680903], [0.0, 0.34886364388584984, 0.21744427757140386], [7.3826086124217779e-18, 0.33142254512060643, 0.18281642036297527], [3.7876741461741416e-16, 0.33142254512060643, 0.18281642036297527], [4.3955838886692812e-15, 0.33142254512060643, 0.18281642036297527], [2.3714568221196212e-14, 0.36375087004209627, 0.15586699182911357], [6.2001726776725255e-17, 0.36375087004209627, 0.15586699182911357], [1.3569866508187716e-15, 0.36375087004209627, 0.15586699182911357], [2.6097708055444687e-15, 0.36375087004209627, 0.15586699182911357], [1.2950146954558496e-15, 0.36375087004209627, 0.15586699182911357]]))], (4, 'rNSGAII_solution', 'DTLZ4'): [([0.10002930687671215, 1.1892333778843424e-166, 0.99498543147807761, 0.0], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.1029450154799528, 0.0, 0.9946876733490625, 9.6128661564211542e-301], [0.1000191959681406, 0.0, 0.99499029004612116, 0.0], [0.10057769430917429, 1.1728614551364937e-251, 0.9949295031323433, 0.0], [0.10162119386245509, 0.0, 0.99482367644883973, 2.1054608359038213e-159], [0.86497885045114964, 0.50351674277390557, 0.0, 0.0], [0.10092801250419721, 0.0, 0.99489427226728433, 7.3825631831268508e-307], [0.10295745483534399, 0.0, 0.9946863098999924, 2.5167998941983214e-168], [0.099424730994242511, 0.0, 0.99504636194486695, 0.0], [0.10171829123053534, 4.8152010478542891e-239, 0.99481552702942322, 0.0], [0.10002930687671215, 1.1892333778843424e-166, 0.99498543147807761, 0.0]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.1029450154799528, 0.0, 0.84419039500398529, 9.6128661564211542e-301], [0.1029450154799528, 0.0, 0.84434170335251468, 0.0], [0.1029450154799528, 5.8643072756824686e-252, 0.84431130989562564, 0.0], [0.1029450154799528, 0.0, 0.84425839655387391, 1.0527304179519106e-159], [0.86497885045114964, 0.50351674277390557, 0.34684655832945405, 0.0], [0.1029450154799528, 0.0, 0.84429369446309621, 3.6912815915634254e-307], [0.10295745483534399, 0.0, 0.84418971327945025, 1.2583999470991607e-168], [0.10295745483534399, 0.0, 0.84436973930188752, 0.0], [0.10295745483534399, 0.0, 0.8442543218441656, 0.0], [0.10295745483534399, 5.9461668894217119e-167, 0.84433927406849285, 0.0]])), ([0.33183220545543529, 0.35894189671603272, 0.54941404220174972, 0.67764453400319569], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.68685207938374226, 0.72680127754738633, 0.0, 0.0], [0.31781007796590621, 0.34515484606241104, 0.55605494061381, 0.68605537470092837], [0.31831989007164857, 0.3832856137561968, 0.60141891304141271, 0.62455505374338116], [0.7875492415682448, 0.0, 0.0, 0.61626983575508953], [0.32033519136064764, 0.36051598621289438, 0.5580684388270698, 0.6752622181225495], [0.31754811108228792, 0.36973655947360268, 0.5581978424763151, 0.67147734292527905], [0.37061515158324043, 0.41112627465468238, 0.51811936780654311, 0.6584776208426607], [0.31535804508974935, 0.37980095032467731, 0.5603834540385717, 0.66504082911854068], [0.85524393450044178, 0.51822957510572776, 0.0, 0.0], [0.33183220545543529, 0.35894189671603272, 0.54941404220174972, 0.67764453400319569]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.68685207938374226, 0.50916160011451106, 0.0, 0.0], [0.68685207938374226, 0.31833838437202344, 0.55605494061381, 0.68605537470092837], [0.68685207938374226, 0.33740376821891627, 0.0, 0.0], [0.68685207938374226, 0.1457609613408179, 0.0, 0.61626983575508953], [0.31831989007164857, 0.32601895444726509, 0.5580684388270698, 0.6752622181225495], [0.31781007796590621, 0.33062924107761926, 0.0, 0.68605537470092837], [0.68685207938374226, 0.35132409866815906, 0.0, 0.0], [0.68685207938374226, 0.72680127754738633, 0.0, 0.0], [0.7875492415682448, 0.51822957510572776, 0.0, 0.0], [0.33183220545543529, 0.32523190969883425, 0.55605494061381, 0.67764453400319569]])), ([0.42896342052642844, 0.25795133913264678, 0.67997966052252679, 0.60043516293077115], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.39048014094271372, 0.30152850496735328, 0.67882469361493425, 0.61051569351935775], [0.43637742720538397, 0.39256528130235935, 0.63369208389144915, 0.6120539277922673], [0.65389692248373688, 0.0, 0.75658844202363229, 0.0], [0.61983319807525306, 0.0, 0.78474352727823837, 0.0], [0.41103052276911367, 0.27804665767237402, 0.66748467812890488, 0.59404813367418996], [0.38204990994474303, 0.0, 0.92414312249825592, 0.0], [0.42887022825591281, 0.25961607588432972, 0.68968092626021837, 0.58191899525885693], [0.41511821530644799, 0.26539485766401583, 0.68392741235844845, 0.58005500861041071], [0.42601178909719101, 0.26306405237364411, 0.65729360124021374, 0.60825529739385031], [0.42896342052642844, 0.25795133913264678, 0.67997966052252679, 0.60043516293077115]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.39048014094271372, 0.30152850496735328, 0.67882469361493425, 0.72027362109031334], [0.43637742720538397, 0.39256528130235935, 0.63369208389144915, 0.72104273822676812], [0.39048014094271372, 0.39256528130235935, 0.67882469361493425, 0.41501577433063441], [0.39048014094271372, 0.30152850496735328, 0.67882469361493425, 0.41501577433063441], [0.41103052276911367, 0.39256528130235935, 0.67882469361493425, 0.71203984116772934], [0.38204990994474303, 0.27804665767237402, 0.92131359267343527, 0.41501577433063441], [0.43637742720538397, 0.39256528130235935, 0.63369208389144915, 0.70597527196006293], [0.41103052276911367, 0.39256528130235935, 0.66748467812890488, 0.70504327863583982], [0.43637742720538397, 0.39256528130235935, 0.63369208389144915, 0.71914342302755951], [0.43637742720538397, 0.39256528130235935, 0.63369208389144915, 0.71523335579601999]])), ([0.02792356784724585, 0.99961192001177779, 0.0, 0.0], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.026976976468302431, 0.99963728908775407, 0.0, 2.4209996796672053e-301], [0.027660346839277136, 0.99961783649774938, 0.0, 8.065156836784708e-285], [0.99999889512325468, 0.0, 0.0, 0.003524440858577916], [0.029187727350980335, 0.99958188475492671, 6.6323979013479934e-122, 0.0], [0.02883647756702095, 0.99958644806448582, 0.0, 0.0], [0.026880153959962703, 0.99963940208135005, 0.0, 6.1518010535722804e-286], [0.028325690627349045, 0.99959954725323696, 1.7889378531397306e-252, 1.5128159725791716e-305], [0.028388925129290685, 0.99959829730872052, 5.0832197413906084e-152, 2.998761701892966e-226], [0.028275524392124968, 0.99960168925022352, 4.9581493781260906e-249, 0.0], [0.02792356784724585, 0.99961192001177779, 0.0, 0.0]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.026976976468302431, 0.99963728908775407, 0.0, 1.2104998398336026e-301], [0.027660346839277136, 0.99961783649774938, 0.0, 4.032578418392354e-285], [0.027660346839277136, 0.99961783649774938, 0.0, 0.001762220429288958], [0.99999889512325468, 0.99958188475492671, 3.3161989506739967e-122, 0.0], [0.99999889512325468, 0.0, 0.0, 0.0], [0.029187727350980335, 0.99958188475492671, 0.0, 3.0759005267861402e-286], [0.029187727350980335, 0.99958188475492671, 8.9446892656986529e-253, 0.0], [0.029187727350980335, 0.99958188475492671, 2.5416098706953042e-152, 1.499380850946483e-226], [0.029187727350980335, 0.99958188475492671, 2.4790746890630453e-249, 0.0], [0.029187727350980335, 0.99958188475492671, 0.0, 0.0]])), ([0.38046460806582694, 0.92830803665681949, 0.0, 0.0], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.37446285865159074, 0.92724509502451657, 0.0, 0.0], [0.35978372367728784, 0.93982625666752351, 6.9499843092364756e-90, 0.0], [0.36125829417523914, 0.93246784069150956, 0.0, 9.6942621750801671e-175], [0.37417317952619406, 0.92739517589965748, 0.0, 0.0], [0.37455405208923898, 0.92721544690743662, 0.0, 0.0], [0.37365126367269846, 0.92758094631339549, 0.0, 0.0], [0.37488048460112916, 0.92707821968121318, 0.0, 0.0], [0.37873445696855751, 0.92668998400583391, 0.0, 0.0], [0.37437664862969816, 0.9273028707466946, 0.0, 0.0], [0.38046460806582694, 0.92830803665681949, 0.0, 0.0]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.38439831092575427, 0.92724509502451657, 0.0, 0.0], [0.37705874343860279, 0.92724509502451657, 3.4749921546182378e-90, 0.0], [0.37779602868757844, 0.92724509502451657, 0.0, 0.0], [0.38425347136305588, 0.92724509502451657, 0.0, 0.0], [0.37455405208923898, 0.92721544690743662, 0.0, 0.0], [0.38399251343630814, 0.92721544690743662, 0.0, 0.0], [0.38460712390052343, 0.92707821968121318, 0.0, 0.0], [0.37873445696855751, 0.92668998400583391, 0.0, 0.0], [0.38435520591480798, 0.92668998400583391, 0.0, 0.0], [0.38739918563287234, 0.92830803665681949, 0.0, 0.0]])), ([1.8718173718209576e-05, 1.6376700979539337e-293, 1.0000012338959481, 0.0], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[7.4918860686215613e-06, 4.625934783081315e-169, 1.0000013216211154, 0.0], [4.5332087908875762e-05, 0.0, 1.0000009955663873, 0.0], [3.0129478872464695e-05, 0.0, 1.0000007942425249, 0.0], [0.00035320815180453377, 0.0, 1.0000019094608874, 0.0], [8.9629342141750009e-05, 3.342199912622469e-273, 1.0000036597130775, 0.0], [4.9573460445253128e-05, 0.0, 1.0000006293435932, 3.7688636598577361e-196], [5.7422501776535105e-06, 0.0, 1.0000007563109612, 9.8136473294952669e-284], [4.6587994875051061e-05, 0.0, 1.0000031278714152, 0.0], [0.61538665087896227, 0.0, 0.0, 0.78822581598697705], [1.8718173718209576e-05, 1.6376700979539337e-293, 1.0000012338959481, 0.0]], [[0.0, 0.0, 0.9612129216540486, 0.0], [3.7459430343107807e-06, 2.3129673915406575e-169, 0.98060712163758201, 0.0], [2.2666043954437881e-05, 0.0, 0.98060695861021796, 0.0], [1.5064739436232347e-05, 0.0, 0.98060685794828673, 0.0], [0.00017660407590226689, 0.0, 0.980607415557468, 0.0], [4.4814671070875005e-05, 1.6710999563112345e-273, 1.0, 0.0], [2.4786730222626564e-05, 0.0, 0.98060677549882092, 1.884431829928868e-196], [5.7422501776535105e-06, 0.0, 0.98060683898250489, 4.9068236647476335e-284], [5.7422501776535105e-06, 0.0, 0.9806080247627319, 0.0], [0.61538665087896227, 0.0, 0.4806064608270243, 0.78822581598697705], [9.3590868591047879e-06, 8.1883504897696684e-294, 0.98060707777499834, 0.0]])), ([0.63042013026878008, 0.0, 0.77625513767016141, 0.0], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[0.50099190741149136, 0.0, 0.0, 0.86545425776210849], [0.83610722409132698, 0.0, 0.54857175034150196, 0.0], [6.4042121289848295e-06, 0.99354835838423383, 0.0, 0.11341742620715142], [0.13361459184735852, 0.99103834847846561, 0.0, 0.0], [0.28060010920956435, 0.95982690462472098, 0.0, 0.0], [3.1064008643266754e-09, 0.99335830301497063, 0.0, 0.11507065058902666], [2.8598524903915915e-06, 0.99300880868628161, 0.0, 0.11809639478612875], [6.0958587639177356e-08, 0.99352820337386749, 6.7913156802646611e-184, 0.11359621328614967], [8.4677761330278176e-08, 0.99364496721930573, 0.0, 0.11259118478805419], [0.63042013026878008, 0.0, 0.77625513767016141, 0.0]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.50099190741149136, 0.0, 0.0, 0.47732969478872372], [0.83610722409132698, 0.0, 0.54857175034150196, 0.044602565907669478], [3.2021060644924148e-06, 0.99354835838423383, 0.0, 0.11341742620715142], [0.83610722409132698, 0.0, 0.0, 0.044602565907669478], [0.50099190741149136, 0.0, 0.0, 0.044602565907669478], [1.5532004321633377e-09, 0.99103834847846561, 0.0, 0.0], [1.4299262451957958e-06, 0.99335830301497063, 0.0, 0.10365076330073386], [3.0479293819588678e-08, 0.99354835838423383, 3.3956578401323305e-184, 0.10140067255074431], [4.2338880665139088e-08, 0.99364496721930573, 0.0, 0.10089815830169657], [0.28060010920956435, 0.95982690462472098, 0.0, 0.044602565907669478]])), ([0.48908154553470412, 0.0, 0.87223864062436951, 0.0], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.47791613163128815, 0.0, 0.87840603945896012, 3.4138865099315262e-185], [0.50035763507673947, 0.0, 0.86582348152877298, 0.0], [0.50319417026119972, 3.286697020979144e-244, 0.86417892658678941, 8.4968166495649402e-204], [0.48707481552318277, 0.0, 0.8733606026866354, 0.0], [0.48657176483572079, 0.0, 0.87364299528602785, 5.2496543810315798e-299], [0.4891774725272518, 2.2469370319353055e-302, 0.87218610646774253, 3.1620201333839779e-322], [0.48508968445258693, 6.189927910052595e-300, 0.87446584292280849, 0.0], [0.46009243062344696, 0.0, 0.88787152670362268, 0.0], [0.50676900004265313, 8.1424868365764117e-82, 0.86210135132307697, 3.0178801278778607e-305], [0.48908154553470412, 0.0, 0.87223864062436951, 0.0]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.3401585857671352, 0.0, 0.87840603945896012, 3.4138865099315262e-185], [0.35137933748986083, 0.0, 0.87840603945896012, 0.0], [0.35279760508209096, 1.643348510489572e-244, 0.87840603945896012, 3.4138865099315262e-185], [0.50319417026119972, 0.0, 0.86417892658678941, 0.0], [0.34448640236935146, 0.0, 0.87840603945896012, 2.6248271905157899e-299], [0.34578925621511702, 1.1234685159676527e-302, 0.87840603945896012, 1.5810100666919889e-322], [0.34374536217778456, 3.0949639550262975e-300, 0.87840603945896012, 0.0], [0.33124673526321458, 0.0, 0.88787152670362268, 0.0], [0.35458501997281766, 4.0712434182882059e-82, 0.88787152670362268, 0.0], [0.34574129271884313, 0.0, 0.88787152670362268, 0.0]])), ([0.028709494870594467, 0.00065751097378512235, 8.5356736122413042e-127, 0.99959258858986821], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[0.059780011041233254, 0.0011176115809453015, 0.0, 0.99822180084429346], [3.7780573496073615e-10, 0.0011264955513821306, 0.051971730919591765, 0.99866511238460753], [1.000000166691609, 0.0, 0.0, 0.0], [0.092519952481019074, 0.99571556216303514, 0.0, 0.0], [0.04956115978855017, 0.00098646498642958628, 0.0, 0.99877261616282775], [0.63551382864820083, 0.0, 0.77209162449784741, 0.0], [0.76282408392478462, 0.6466093160255223, 0.0, 0.0], [0.77272274209452863, 0.0, 0.0, 0.63475553296716425], [3.8916710566268363e-09, 0.052107488002665042, 0.0, 0.99864393160068921], [0.028709494870594467, 0.00065751097378512235, 8.5356736122413042e-127, 0.99959258858986821]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.059780011041233254, 0.0024895993720451494, 0.0, 0.99822180084429346], [1.8890286748036808e-10, 0.002494041357263564, 0.051971730919591765, 0.99866511238460753], [1.0, 0.0019307935815724986, 0.0, 0.99822180084429346], [0.092519952481019074, 0.49978857466309007, 0.0, 0.0], [3.7780573496073615e-10, 0.0024240260747872919, 0.0, 0.99877261616282775], [1.0, 0.0019307935815724986, 0.0, 0.0], [0.76282408392478462, 0.32523545159433365, 0.0, 0.0], [0.76282408392478462, 0.0019307935815724986, 0.0, 0.0], [1.9458355283134181e-09, 0.02798453758290502, 0.0, 0.99864393160068921], [3.7780573496073615e-10, 0.0022595490684650596, 4.2678368061206521e-127, 0.99866511238460753]])), ([2.0851943698416524e-12, 0.81339595323616343, 0.0, 0.58171110731400799], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[1.0880301798864374e-10, 0.81289100421336447, 0.0, 0.58242480244452388], [9.5938325054450327e-10, 0.8107850951670722, 2.8123922629132384e-175, 0.58534748560411076], [4.5046534168163704e-12, 0.81739514241769806, 0.0, 0.57608005572882326], [1.8371610896642768e-13, 0.81014729564140397, 0.0, 0.58623054259569063], [2.0160164461361012e-09, 0.79696130799563403, 0.0, 0.60403300980064956], [1.2967546750754002e-12, 0.79954044159862347, 0.0, 0.60061947277669625], [4.3163903083497628e-09, 0.78967797989876876, 2.9843739252797779e-213, 0.61352690610201732], [4.5359107556915943e-11, 0.80375769999482383, 0.0, 0.59495884335487792], [5.6685388398397067e-11, 0.81382797442788291, 8.2865583556147512e-263, 0.58110771132527417], [2.0851943698416524e-12, 0.81339595323616343, 0.0, 0.58171110731400799]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [5.440150899432187e-11, 0.81289100421336447, 0.0, 0.58242480244452388], [4.7969162527225164e-10, 0.81289100421336447, 1.4061961314566192e-175, 0.58242480244452388], [2.2523267084081852e-12, 0.81739514241769806, 0.0, 0.57608005572882326], [9.1858054483213841e-14, 0.81739514241769806, 0.0, 0.57608005572882326], [1.0080082230680506e-09, 0.81739514241769806, 0.0, 0.57608005572882326], [6.4837733753770012e-13, 0.81739514241769806, 0.0, 0.57608005572882326], [2.1581951541748814e-09, 0.81739514241769806, 0.0, 0.57608005572882326], [2.2679553778457972e-11, 0.81739514241769806, 0.0, 0.57608005572882326], [2.8342694199198533e-11, 0.81739514241769806, 4.1432791778073756e-263, 0.57608005572882326], [1.0425971849208262e-12, 0.81739514241769806, 0.0, 0.57608005572882326]]))], (3, 'rNSGAII_solution', 'DTLZ2'): [([0.59798453137049379, 4.3684776173934741e-06, 0.80150804259731023], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.60666267873718194, 4.5054214968166153e-09, 0.79496018337970586], [0.61306643714667941, 2.0663030996782105e-12, 0.79003241426491677], [0.58021034500424506, 1.141811738941182e-09, 0.81446830821116367], [0.58918179131244752, 4.3588173381959033e-15, 0.80800162004111897], [0.61326760597025576, 4.3612021939120417e-16, 0.78987637063208715], [0.61073893099249488, 2.0695459693260987e-12, 0.79183541744731545], [0.6038083789044999, 1.7130893582801515e-10, 0.79737148792458046], [0.61307341975916474, 2.4483252244282892e-14, 0.79002648135589759], [0.61238373883428499, 2.2846174892560519e-13, 0.79056224162792144], [0.59798453137049379, 4.3684776173934741e-06, 0.80150804259731023]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.60666267873718194, 2.2527107484083076e-09, 0.74685973692687802], [0.61306643714667941, 1.0331515498391053e-12, 0.74439585236948347], [0.61306643714667941, 5.7090586947059102e-10, 0.75661379934260697], [0.61306643714667941, 2.1794086690979516e-15, 0.75338045525758457], [0.61326760597025576, 2.1806010969560209e-16, 0.74431783055306866], [0.61326760597025576, 1.0347729846630493e-12, 0.74529735396068286], [0.61326760597025576, 8.5654467914007574e-11, 0.74806538919931531], [0.61326760597025576, 1.2241626122141446e-14, 0.74439288591497388], [0.61326760597025576, 1.142308744628026e-13, 0.74466076605098586], [0.61326760597025576, 4.3612021939120417e-16, 0.75013366653568014]])), ([1.2851171596505354e-05, 5.5707150936976523e-13, 1.0000002886807191], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[1.3832600933458616e-12, 0.00010778581064852632, 1.0000017740731795], [1.0036546298937416e-05, 9.2520466616211762e-16, 1.0000031698624756], [4.8700064374188653e-13, 5.1387879107780069e-05, 1.0000006722291084], [8.5843004094157208e-05, 8.4945056471095418e-10, 1.0000001798484079], [3.72532058289428e-07, 0.00013802678825158718, 1.0000017820399481], [1.4961959131996703e-13, 0.00038672260611982711, 1.00000202596246], [1.4753209134186948e-13, 5.7688024427045971e-05, 1.0000009555255878], [4.9037694096537006e-05, 4.0831868632738509e-14, 1.0000009326183643], [1.538005857275391e-13, 8.2438485939842902e-05, 1.0000015871828285], [1.2851171596505354e-05, 5.5707150936976523e-13, 1.0000002886807191]], [[0.0, 0.0, 0.9829301429354526], [6.9163004667293082e-13, 5.3892905324263159e-05, 1.0], [5.018273149468708e-06, 4.6260233308105881e-16, 1.0], [2.4350032187094327e-13, 2.5693939553890035e-05, 1.0], [4.2921502047078604e-05, 4.2472528235547709e-10, 1.0], [1.86266029144714e-07, 6.9013394125793591e-05, 1.0], [7.4809795659983517e-14, 0.00019336130305991356, 1.0], [7.3766045670934741e-14, 2.8844012213522986e-05, 1.0], [2.4518847048268503e-05, 2.0415934316369255e-14, 1.0], [7.690029286376955e-14, 4.1219242969921451e-05, 1.0], [6.425585798252677e-06, 2.7853575468488261e-13, 1.0]])), ([0.67303382755080354, 0.27148471031386523, 0.68809132570485698], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.6435284286459606, 0.38338265583970588, 0.66272424732655899], [0.44186273024450939, 0.59864543436856665, 0.66816324738858179], [0.72330470015640524, 1.4590546193267073e-08, 0.6905311631552743], [0.56436461248958736, 0.47065746132652869, 0.67825369685026049], [0.44929529868260459, 0.578717998643836, 0.68065031772234608], [0.24676273904507331, 0.70299925665513996, 0.66709455995008682], [0.39533820214734439, 0.60266498438587057, 0.69318904966146744], [0.46103209088258268, 0.65275664990415116, 0.60134422170762825], [0.56909409816161904, 0.44303702459857819, 0.69275764774052839], [0.67303382755080354, 0.27148471031386523, 0.68809132570485698]], [[0.0, 0.0, 0.0], [0.6435284286459606, 0.38338265583970588, 0.3313621236632795], [0.6435284286459606, 0.38338265583970588, 0.3340816236942909], [0.6435284286459606, 7.2952730966335366e-09, 0.34526558157763715], [0.72330470015640524, 0.38338265583970588, 0.33912684842513025], [0.6435284286459606, 0.38338265583970588, 0.34032515886117304], [0.6435284286459606, 0.38338265583970588, 0.33354727997504341], [0.72330470015640524, 0.38338265583970588, 0.34659452483073372], [0.46103209088258268, 0.65275664990415116, 0.30067211085381412], [0.72330470015640524, 0.65275664990415116, 0.3463788238702642], [0.46103209088258268, 0.65275664990415116, 0.34404566285242849]])), ([0.7763514847620635, 0.63030247863479449, 1.9992583154736876e-13], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[0.76490735322106285, 0.64433707397603679, 1.0562551491598471e-08], [0.7774046262569263, 0.62900131619577138, 1.7087009363601133e-07], [0.77956623189157104, 0.62632121325904466, 2.0155700082663774e-13], [0.76517645094622022, 0.64382161628380741, 5.4877369595965077e-10], [0.76130030013914041, 0.64840028265322547, 9.8630529141967001e-10], [0.76157277477054097, 0.64808008095134484, 1.3402245916322473e-05], [0.7616682377658347, 0.64796947974774777, 9.2080099340811386e-09], [0.77964584528268743, 0.62622398984561811, 4.9871540203893024e-11], [0.78078781813808751, 0.62479700586592413, 5.3083467863471532e-12], [0.7763514847620635, 0.63030247863479449, 1.9992583154736876e-13]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.76490735322106285, 0.64433707397603679, 5.2812757457992357e-09], [0.7204817915380054, 0.64433707397603679, 8.5435046818005663e-08], [0.72156259435532777, 0.64433707397603679, 1.0077850041331887e-13], [0.71436770388265236, 0.64433707397603679, 2.7438684797982539e-10], [0.71242962847911251, 0.64840028265322547, 4.9315264570983501e-10], [0.71256586579481274, 0.64840028265322547, 6.7011229581612367e-06], [0.71261359729245966, 0.64840028265322547, 4.6040049670405693e-09], [0.72160240105088591, 0.64840028265322547, 2.4935770101946512e-11], [0.72217338747858606, 0.64840028265322547, 2.6541733931735766e-12], [0.71995522079057395, 0.64840028265322547, 9.9962915773684379e-14]])), ([0.64342540096296719, 4.5127207191164721e-05, 0.76551122010126615], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.63655599107213912, 1.9469630123997939e-05, 0.77123126988581392], [6.3328268830221736e-07, 0.49888314078174234, 0.86667397868314444], [1.5873123476854587e-05, 0.62297736684038185, 0.78224060949720853], [2.3788343493767611e-07, 0.54736571669609546, 0.83690026809575158], [0.55468361908010844, 0.048967161889516317, 0.83087449877725572], [0.64255047235264662, 0.095031365107948657, 0.76033098638341956], [0.62134587748872261, 2.0223208226460798e-10, 0.78353832787039823], [0.53499137328120705, 2.4126288422079079e-05, 0.84525862726304934], [0.51857082152553169, 0.30107719882724837, 0.80041803701119263], [0.64342540096296719, 4.5127207191164721e-05, 0.76551122010126615]], [[0.0, 0.0, 0.2524577834161418], [0.63655599107213912, 1.9469630123997939e-05, 0.5118445266509779], [3.1664134415110868e-07, 1.9469630123997939e-05, 0.55956588104964311], [7.9365617384272937e-06, 0.49888314078174234, 0.51734919645667521], [1.1894171746883806e-07, 0.54736571669609546, 0.54467902575594662], [0.63655599107213912, 1.9469630123997939e-05, 0.54166614109669875], [0.63655599107213912, 0.095031365107948657, 0.50639438489978073], [0.62134587748872261, 0.62297736684038185, 0.51799805564326995], [0.64255047235264662, 1.206314421103954e-05, 0.54885820533959562], [0.64255047235264662, 0.095031365107948657, 0.52643791021366715], [0.64255047235264662, 2.256360359558236e-05, 0.50898450175870402]])), ([2.3977236086740819e-05, 0.87391951492417874, 0.48607222675250966], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.32084277763635732, 0.77875330955125932, 0.53935692096129795], [0.11894744180244166, 0.96807663292603496, 0.26540284617648063], [0.28639890026960912, 0.81315835313396456, 0.50690592300928672], [0.29897732861863935, 0.77567036915256182, 0.55586223547832903], [0.27827284062494995, 0.78895734498194281, 0.54782776705356973], [0.26640193955017533, 0.80615876534589737, 0.52834595317666133], [0.20675242789357129, 0.84538807223558221, 0.50027999795501732], [0.26282952236614077, 0.84774229632639109, 0.46083457860775179], [1.6245031143776435e-07, 0.79703366575103618, 0.60394926070271326], [2.3977236086740819e-05, 0.87391951492417874, 0.48607222675250966]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.32084277763635732, 0.77875330955125932, 0.28322375692078122], [0.11894744180244166, 0.96807663292603496, 0.14624671952837257], [0.11894744180244166, 0.96807663292603496, 0.26699825794477561], [0.29897732861863935, 0.77567036915256182, 0.55586223547832903], [0.29897732861863935, 0.77875330955125932, 0.54782776705356973], [0.11894744180244166, 0.96807663292603496, 0.27771827302846291], [0.11894744180244166, 0.96807663292603496, 0.26368529541764091], [0.29897732861863935, 0.77567036915256182, 0.24396258574400814], [8.1225155718882177e-08, 0.96807663292603496, 0.31551992679148888], [1.1988618043370409e-05, 0.96807663292603496, 0.25658140981638711]])), ([0.84185332761641829, 4.9631364013569909e-15, 0.53970970686669284], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.84193713753048371, 1.9269745914807753e-15, 0.53957713128050411], [0.84048741572176566, 3.3627884169659633e-14, 0.54183119266633561], [0.8407859572038614, 2.4500084065013972e-14, 0.54136841356816401], [0.84161901708433162, 5.0046606130760111e-13, 0.54007315710981807], [0.84336876410544148, 1.3226600124807892e-10, 0.5380005010679686], [0.84036860163191862, 1.1806429720116404e-11, 0.54201706194334864], [0.84079214393102319, 5.9085486445443486e-13, 0.54135964545129545], [0.84052905981450587, 8.3778301639943442e-15, 0.54176732224059954], [0.84459468590705045, 1.0292673796768474e-15, 0.53540760260154796], [0.84185332761641829, 4.9631364013569909e-15, 0.53970970686669284]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.8373775581071452, 9.6348729574038767e-16, 0.53507590531894378], [0.83665269720278623, 1.6813942084829817e-14, 0.53620293601185964], [0.8368019679438341, 1.2250042032506986e-14, 0.53597154646277378], [0.83721849788406921, 2.5023303065380055e-13, 0.53532391823360082], [0.84336876410544148, 6.6133000624039461e-11, 0.53428759021267602], [0.83659329015786277, 5.9032148600582018e-12, 0.5362958706503661], [0.83680506130741494, 2.9542743222721743e-13, 0.5359671624043395], [0.83667351924915634, 4.1889150819971721e-15, 0.53617100079899149], [0.84459468590705045, 5.146336898384237e-16, 0.53299114097946576], [0.8373356531501126, 2.4815682006784955e-15, 0.53514219311203814]])), ([0.32920082008002033, 0.89815851238800382, 0.29213638278400611], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.16932263883777424, 0.86280021830918918, 0.47640592775517487], [0.41417844885164412, 0.85763925633827875, 0.30483013110501384], [0.32920832336254857, 0.8820401715001126, 0.33736084869282784], [0.15214288750894245, 0.8540632665600294, 0.49986106157754007], [0.10378452846447801, 0.92360273684164973, 0.36912869425538819], [0.23446033157568569, 0.91603646467102806, 0.32557317872896585], [0.22398654579356586, 0.9029092259663607, 0.36713381008142504], [0.0084016223012394614, 0.8683602615274475, 0.49596703120716107], [0.40174584729779972, 0.860543502430551, 0.31332670044327127], [0.32920082008002033, 0.89815851238800382, 0.29213638278400611]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.16932263883777424, 0.86280021830918918, 0.47640592775517487], [0.41417844885164412, 0.85763925633827875, 0.15241506555250692], [0.41417844885164412, 0.85763925633827875, 0.16868042434641392], [0.15214288750894245, 0.85763925633827875, 0.24993053078877003], [0.41417844885164412, 0.85763925633827875, 0.1845643471276941], [0.41417844885164412, 0.85763925633827875, 0.16278658936448293], [0.41417844885164412, 0.85763925633827875, 0.18356690504071252], [0.0075166746889362632, 0.8683602615274475, 0.24798351560358053], [0.15214288750894245, 0.85763925633827875, 0.15666335022163563], [0.15214288750894245, 0.89815851238800382, 0.36713381008142504]])), ([0.32343939616569833, 9.0666618138515216e-05, 0.9462495954895378], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[6.219715713678443e-11, 0.31157614826775148, 0.95022289575117458], [8.0791672506182079e-12, 0.31883601061378747, 0.94781122043992572], [1.0428625005431819e-15, 0.23163786670018588, 0.97702649391600516], [1.8613883715190038e-11, 0.30186382908501797, 0.95335693597997884], [0.32435547324538766, 1.2183266299281361e-10, 0.94593725070418666], [0.32561083959241843, 2.7066771866556729e-15, 0.94550474198134715], [0.31530630843313995, 2.555224829759546e-08, 0.94959584100763106], [0.30933738256163468, 2.4004849798813929e-11, 0.95095372830499936], [8.0419562167801201e-12, 0.31441401078681913, 0.9492889190374868], [0.32343939616569833, 9.0666618138515216e-05, 0.9462495954895378]], [[0.0, 0.0, 0.8177116892877433], [3.1098578568392215e-11, 0.31157614826775148, 0.95022289575117458], [4.0395836253091039e-12, 0.31883601061378747, 0.94781122043992572], [5.2143125027159095e-16, 0.31883601061378747, 0.94781122043992572], [9.3069418575950191e-12, 0.31883601061378747, 0.94781122043992572], [0.16217773662269383, 6.0916331496406805e-11, 0.94593725070418666], [0.16280541979620922, 1.3533385933278364e-15, 0.94593725070418666], [0.15765315421656997, 1.277612414879773e-08, 0.94959584100763106], [0.15466869128081734, 1.2002424899406964e-11, 0.97702649391600516], [4.02097810839006e-12, 1.2183266299281361e-10, 0.95095372830499936], [0.16171969808284917, 4.5333309069257608e-05, 0.95095372830499936]])), ([6.4848665818843432e-06, 0.77119233238831664, 0.63660349472662048], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[1.7959097799141011e-12, 0.76990445522929452, 0.63816164458991331], [3.5944108390846611e-12, 0.77014010833553526, 0.63787706093281216], [3.9764182884475231e-16, 0.78690851200841727, 0.6170711698223732], [6.5836608828098635e-13, 0.78662784417247444, 0.6174285354405401], [3.4999902127013313e-07, 0.79123860006269875, 0.61150874376091546], [4.2881003454027475e-12, 0.77172611500571242, 0.63595635803043538], [8.8312864585894801e-14, 0.77487738644747017, 0.63211235718564829], [5.0107669247287357e-15, 0.77082485557050318, 0.63704797315977002], [7.8742250149987305e-13, 0.77020488446163904, 0.63779775653189663], [6.4848665818843432e-06, 0.77119233238831664, 0.63660349472662048]], [[0.0, 0.6790416599958826, 0.5290286140740371], [8.9795488995705057e-13, 0.76990445522929452, 0.63816164458991331], [1.7972054195423305e-12, 0.77014010833553526, 0.63787706093281216], [1.9882091442237615e-16, 0.78690851200841727, 0.6170711698223732], [3.2918304414049317e-13, 0.78690851200841727, 0.6170711698223732], [1.7499951063506657e-07, 0.78690851200841727, 0.6170711698223732], [2.1440501727013738e-12, 0.79123860006269875, 0.61150874376091546], [4.41564322929474e-14, 0.78690851200841727, 0.6170711698223732], [2.5053834623643679e-15, 0.79123860006269875, 0.6170711698223732], [3.9371125074993652e-13, 0.78690851200841727, 0.6170711698223732], [3.2424332909421716e-06, 0.79123860006269875, 0.61150874376091546]]))], (3, 'rNSGAII_solution', 'DTLZ3'): [([0.010380016783050375, 0.00027475078317115313, 2.1704751011608105], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[9.9319734497315864, 1.9964002952834159, 0.00012313308354160865], [7.0978431013572898, 0.0013558512429677533, 0.0019991080163021613], [5.4465565451089244, 0.42390000769172514, 6.3313923245075836], [3.7832685205623595, 0.17192814754288036, 0.75938095427227514], [1.4183401681164463, 0.023858468986394508, 3.8738000200525535], [1.3140334184986939, 3.15430032506129, 7.7513249863884637], [1.2992448533405738, 4.9953735024323776e-06, 2.0525219854585135], [3.0326748516357238, 2.4071951039609651e-05, 14.793380491979729], [7.6804114218342008, 1.6944320558878267, 0.69475912138080054], [0.010380016783050375, 0.00027475078317115313, 2.1704751011608105]], [[0.4981520234633334, 0.0, 0.6987592904740502], [1.0, 1.0, 0.34944121177879589], [1.0, 0.00067792562148387663, 0.35037919924517619], [1.0, 0.0013558512429677533, 1.0], [1.0, 0.17192814754288036, 0.7290701223731626], [1.0, 0.17192814754288036, 1.0], [1.0, 0.023858468986394508, 1.0], [1.0, 2.4976867512161888e-06, 1.0], [1.0, 1.2035975519804826e-05, 1.0], [1.0, 4.9953735024323776e-06, 0.69675920592742535], [1.0, 0.00013737539158557656, 1.0]])), ([3.4799445391453463e-08, 4.4513620026163599e-07, 4.3076716149046828], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[0.00017309917197288072, 2.8290542903133231e-05, 9.3825261394872772], [0.00075471999158363849, 0.017346071261399275, 2.1815787922474814], [0.0079166649931554124, 0.30464678560210795, 6.1479744103829566], [0.025981465200854009, 0.10222243142459471, 2.1905481099803135], [0.51164339568337192, 0.046407949282133348, 9.4398797687146967], [8.3485828635902595e-07, 1.3121576783942912e-10, 9.8694492575075756], [0.0079427053395792861, 0.10574803278425782, 5.4554746982764968], [0.10733534373855771, 0.0025406215698354209, 6.7964458302268573], [2.5954237029061418e-06, 1.2263304172406136e-06, 5.9186066228820531], [3.4799445391453463e-08, 4.4513620026163599e-07, 4.3076716149046828]], [[0.0, 0.0, 0.9829301429354526], [8.6549585986440362e-05, 1.4145271451566616e-05, 1.0], [0.00037735999579181925, 2.8290542903133231e-05, 1.0], [0.0039583324965777062, 0.017346071261399275, 1.0], [0.012990732600427005, 0.30464678560210795, 1.0], [0.25582169784168596, 0.10222243142459471, 1.0], [4.1742914317951298e-07, 6.5607883919714561e-11, 1.0], [0.0039713526697896431, 0.017346071261399275, 1.0], [0.053667671869278857, 0.0012703107849177104, 1.0], [1.2977118514530709e-06, 1.3121576783942912e-10, 1.0], [1.7399722695726732e-08, 4.4513620026163599e-07, 1.0]])), ([0.7750063248518706, 4.6144647687382472, 0.0020421911945846666], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[2.9847401622289911, 1.6722004309980909, 5.7880224402372486e-05], [0.416936391335332, 9.1788723499526768, 0.11824638024131892], [4.7681059176810309e-05, 0.00073116450703693367, 3.0466397216380274], [4.1719880652551247, 0.25069459380580478, 0.00079998724282852427], [0.89603728121296677, 5.1923383964772274, 0.026323461003197862], [5.1201535784701901, 0.00023349734166648104, 0.76777015455467978], [5.4624340873130111, 0.049539542534358993, 0.65846091077553148], [6.738451772518661, 0.00034522289765126718, 1.0106189962782319e-11], [0.804208758452859, 7.5254274446409095, 8.5703664361336474e-11], [0.7750063248518706, 4.6144647687382472, 0.0020421911945846666]], [[0.0, 0.0, 0.0], [1.0, 1.0, 2.8940112201186243e-05], [1.0, 1.0, 0.059123190120659461], [2.3840529588405155e-05, 0.00036558225351846684, 1.0], [1.0, 1.0, 0.00039999362141426213], [4.7681059176810309e-05, 0.00073116450703693367, 0.013161730501598931], [1.0, 0.00011674867083324052, 0.38388507727733989], [1.0, 1.0, 0.32923045538776574], [1.0, 0.00017261144882563359, 5.0530949813911596e-12], [1.0, 1.0, 4.2851832180668237e-11], [1.0, 1.0, 0.0010210955972923333]])), ([3.5474322606646034, 7.5533955082078705, 0.026018398135831219], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[4.9625952325057394, 0.80954014706249844, 0.00059787675184374796], [4.2708804846293225, 0.50377315717898274, 6.5146829111858407e-11], [4.4253413504076455, 0.81835143733001792, 0.00019123994049795523], [0.078039327977717232, 1.4923322528587393, 5.2579697172731876], [0.7665919386913147, 5.6407743042962952, 0.00065948559062764902], [0.11372979176244238, 3.3706351213485704, 3.1240418216418486e-05], [5.8010323382805255e-09, 5.7894053864039652, 3.865332792997299e-07], [1.6370801927436358, 5.9762299746219538, 0.00042903094685542924], [4.617618560097978, 0.00036717507443043086, 0.53348857169832431], [3.5474322606646034, 7.5533955082078705, 0.026018398135831219]], [[0.6635589568190845, 0.5338175168551759, 0.0], [1.0, 0.80954014706249844, 0.00029893837592187398], [1.0, 0.50377315717898274, 3.2573414555929204e-11], [1.0, 0.50377315717898274, 9.5619970248977617e-05], [0.37079914239840084, 1.0, 1.0], [1.0, 0.50377315717898274, 0.00032974279531382451], [0.38864437429076343, 1.0, 1.5620209108209243e-05], [0.33177948131005841, 0.50377315717898274, 1.9326663964986495e-07], [1.0, 1.0, 0.00021451547342771462], [1.0, 0.50377315717898274, 3.1240418216418486e-05], [1.0, 1.0, 0.00019123994049795523]])), ([4.6865628853606661e-07, 7.5700727615831571e-06, 3.0410100577553001], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.1224550734632838, 0.0081848565826054109, 5.5813298479585525], [0.19876625752138452, 0.00052481228397596139, 8.0674582961644177], [3.8537337948034751, 0.32157094458389546, 0.056102448758914861], [7.3718356422593958, 0.3316538596929321, 0.99550049233508797], [1.0692040094418351, 4.0267081662303132, 6.0636609501987225e-05], [0.0016327033501021056, 8.2389710625458226e-07, 3.1579783265214014], [6.315146564625719, 0.022185330442730081, 0.016175622817735009], [7.2098535642807193e-06, 0.14691330699248395, 3.4173625776400134], [2.4094436464431568, 0.0011116873012802504, 7.6824477974109211e-06], [4.6865628853606661e-07, 7.5700727615831571e-06, 3.0410100577553001]], [[0.0, 0.0, 0.2524577834161418], [0.1224550734632838, 0.0040924282913027054, 1.0], [0.1224550734632838, 0.00026240614198798069, 1.0], [1.0, 0.00052481228397596139, 0.15428011608752831], [0.19876625752138452, 0.32157094458389546, 0.056102448758914861], [0.19876625752138452, 1.0, 0.12625921001282189], [0.0016327033501021056, 4.1194855312729113e-07, 1.0], [0.0016327033501021056, 1.0, 1.0], [3.6049267821403597e-06, 0.14691330699248395, 1.0], [1.0, 0.0005558436506401252, 0.1262327329319696], [2.343281442680333e-07, 3.7850363807915786e-06, 1.0]])), ([0.52560781212351515, 4.2317879366128563, 0.66107291470051666], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.17222726903878455, 4.301867105361656, 0.012432847912452883], [1.4623552618382771e-07, 2.1730330118071515, 2.5505056939853828e-09], [0.00059759230643868766, 0.0056023740806191184, 7.4365678830116311], [0.013781815887903292, 7.8462885759836967, 0.84718686955640066], [0.53801396108433841, 5.8280138868136868, 0.80474687350729079], [2.3753188826097045, 2.3958797323851138, 0.40514303706748073], [0.074485697583368343, 7.6063841415693068, 9.565973637999783e-07], [0.0099963582928083907, 4.3174549406742555, 0.20492816301965266], [0.45939881345282718, 4.2795291681116625, 0.0014533224164320862], [0.52560781212351515, 4.2317879366128563, 0.66107291470051666]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.17222726903878455, 1.0, 0.019761720396358689], [7.3117763091913855e-08, 1.0, 0.013545297715385096], [0.00029879615321934383, 1.0, 1.0], [0.00059759230643868766, 0.0056023740806191184, 0.43713873121833258], [1.4623552618382771e-07, 1.0, 1.0], [0.00059759230643868766, 0.0056023740806191184, 0.21611681497387261], [0.00059759230643868766, 1.0, 0.013545774738814149], [0.00059759230643868766, 1.0, 0.11600937794995858], [0.00059759230643868766, 0.0056023740806191184, 0.014271957648348292], [0.00059759230643868766, 1.0, 1.0]])), ([5.1115970756360296, 5.5004322263455004e-08, 0.31008712340514449], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[6.1966976223701105, 0.76696498679434799, 0.50839400862476491], [1.9949937129350053, 0.32683142788443437, 0.66251075713565211], [7.6583247237188177, 0.034054880424050374, 0.0065160822118237047], [5.2096484090590245, 0.33649814086579521, 0.17939102105384161], [5.0479478189288098, 0.13402869241139484, 3.4277172020677469], [5.8503979169579319, 1.2174225163288104, 0.58227039686303883], [6.6891849480789203, 0.99172729148250283, 2.5974041661735205e-11], [1.3364755102401005, 3.701580909550942, 0.91280897643783165], [4.5346283433910903, 0.013329955534228853, 0.17149073840590809], [5.1115970756360296, 5.5004322263455004e-08, 0.31008712340514449]], [[0.8328179786838068, 0.0, 0.5305746793573836], [1.0, 0.76696498679434799, 0.51948434399107424], [1.0, 0.32683142788443437, 0.59654271824651783], [1.0, 0.32683142788443437, 0.26854538078460366], [1.0, 0.32683142788443437, 0.3549828502056126], [1.0, 0.32683142788443437, 1.0], [1.0, 0.33649814086579521, 0.55642253811021125], [1.0, 0.32683142788443437, 0.66251075713565211], [1.0, 1.0, 0.72169182789760766], [1.0, 1.0, 0.35103270888164584], [1.0, 2.7502161131727502e-08, 0.42033090138126405]])), ([5.4932796103138932e-07, 5.2933307417034454, 1.1343852703956277e-05], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.63099851260852935, 5.7272607776405113, 0.020724575013273176], [0.35945440035577558, 5.3488065082332445, 9.904441416760234e-06], [5.4017247226195746, 0.47873660176018445, 9.1542382013447478e-09], [1.5748660428232335, 7.2087323193294841, 1.7290194849438505e-09], [0.73104801225302241, 11.419408973120435, 0.073376354292307905], [9.4287615331216531e-09, 5.2675018259309665, 0.0062665809042969853], [0.070387844557303417, 5.324852829003035, 0.32739670491804734], [0.12925493815421737, 3.2568087783559174, 0.027876514775247833], [0.12948283040445444, 3.3745309625425559, 0.15770190906719], [5.4932796103138932e-07, 5.2933307417034454, 1.1343852703956277e-05]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.63099851260852935, 1.0, 0.010362287506636588], [0.35945440035577558, 1.0, 4.952220708380117e-06], [1.0, 0.47873660176018445, 4.5771191006723739e-09], [1.0, 1.0, 8.6450974247192527e-10], [0.63099851260852935, 1.0, 0.036688177146153952], [0.0033158682526972989, 1.0, 0.0031332904521484926], [1.0, 1.0, 0.16369835245902367], [0.12925493815421737, 1.0, 0.013938257387623916], [1.0, 1.0, 9.1542382013447478e-09], [0.0033161382022970482, 1.0, 5.6719263519781387e-06]])), ([0.0027272058812358494, 0.08085936137021317, 6.2083669623524882], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[0.0048778543304199909, 0.35932243560418548, 3.4468135013790464], [0.057664910703538332, 0.51263142488337643, 10.254230663577477], [0.35856895567481867, 0.0091180636815957929, 4.0606545683039599], [3.2268429419632012, 2.331937877182531, 1.1708474252415804], [0.00024349757283672482, 0.1210372152147593, 3.2357034026411449], [2.1436554318261906e-06, 1.7995771032202633e-05, 5.703494883356802], [5.560805215230439, 0.8575137350256703, 8.2297809742932515e-06], [0.026219016704772793, 0.32124667545982349, 4.6809171032143118], [4.7054841214091114, 0.46248920439425273, 4.4896837441334263], [0.0027272058812358494, 0.08085936137021317, 6.2083669623524882]], [[0.0, 0.0, 0.8177116892877433], [0.0024389271652099954, 0.35932243560418548, 1.0], [0.028832455351769166, 0.35932243560418548, 1.0], [0.17928447783740933, 0.0045590318407978964, 1.0], [1.0, 0.0091180636815957929, 1.0], [0.00012174878641836241, 0.1210372152147593, 1.0], [1.0718277159130953e-06, 8.9978855161013164e-06, 1.0], [1.0, 0.8575137350256703, 1.0], [2.1436554318261906e-06, 1.7995771032202633e-05, 1.0], [1.0, 0.8575137350256703, 1.0], [0.0013636029406179247, 0.8575137350256703, 1.0]])), ([5.436643580494068, 0.037418441398509826, 0.012560662139537273], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[0.52547945955883479, 0.0010664238014193651, 10.398187259001825], [0.29922231631490792, 0.019337952201961894, 6.3488441421525996], [0.24881107900990748, 4.0787252425694236, 0.21933971834356769], [3.3634570399695326e-11, 4.8938772665466921, 0.043389297838480875], [0.00029929079669917312, 0.0047170742803590232, 3.9342028133476559], [0.095653726768945249, 3.2935174366074089, 0.99442717120333768], [0.029314798356960509, 2.7177701681219748, 1.6749218236030452], [4.9533129606957367, 0.051203382751899229, 1.6302945976102196e-08], [0.23587048843552777, 5.8637325492030383, 0.00032240091963679573], [5.436643580494068, 0.037418441398509826, 0.012560662139537273]], [[0.0, 0.6790416599958826, 0.5290286140740371], [0.2627397297794174, 0.0010664238014193651, 1.0], [0.14961115815745396, 0.019337952201961894, 1.0], [0.12440553950495374, 1.0, 1.0], [1.6817285199847663e-11, 1.0, 0.043389297838480875], [0.00014964539834958656, 1.0, 1.0], [0.047826863384472625, 0.0047170742803590232, 1.0], [0.014657399178480255, 1.0, 1.0], [1.0, 0.051203382751899229, 1.0], [0.11793524421776388, 1.0, 0.99442717120333768], [1.0, 0.051203382751899229, 1.0]]))], (4, 'ACH_solution', 'DTLZ2'): [([0.28646794912001444, 0.21477262302549827, 0.9102391129337533, 0.21654562480827924], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.2869650464929058, 0.21704309398387184, 0.9106552938126412, 0.21700415349100485], [0.2865349586323227, 0.21488123621870173, 0.9103061209601414, 0.21231051585012808], [0.2869290786242151, 0.2165911526571082, 0.9107002426399937, 0.21666122054520126], [0.2859043888131365, 0.21609707924385682, 0.9097901959027794, 0.21609642532662365], [0.28602444738892063, 0.2173770312828188, 0.9110701461937843, 0.21599209228953584], [0.2866780971659791, 0.21687419256536508, 0.9105582673899884, 0.21687414137746908], [0.28617683022233104, 0.21528185946196296, 0.909947985134973, 0.21625486856595008], [0.28584387232548725, 0.2159219208177439, 0.9095663359563287, 0.2158570062276365], [0.2866849346877316, 0.21569798192868767, 0.910456096008914, 0.21675532231247788], [0.28646794912001444, 0.21477262302549827, 0.9102391129337533, 0.21654562480827924]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.28696504649290577, 0.21704309398387184, 0.8021742052357747, 0.21700415349100485], [0.2865349586323227, 0.21488123621870173, 0.8019996188095248, 0.21231051585012808], [0.2865349586323227, 0.21488123621870173, 0.8021966796494509, 0.21231051585012808], [0.28590438881313651, 0.21609707924385682, 0.8017416562808437, 0.21609642532662365], [0.28590438881313651, 0.21488123621870173, 0.8023816314263462, 0.21609642532662365], [0.28590438881313651, 0.21488123621870173, 0.8021256920244482, 0.21609642532662365], [0.28590438881313651, 0.21488123621870173, 0.8018205508969405, 0.21231051585012808], [0.2865349586323227, 0.21488123621870173, 0.8016297263076184, 0.21585700622763651], [0.2865349586323227, 0.21528185946196296, 0.802074606333911, 0.21625486856595008], [0.2865349586323227, 0.21609707924385682, 0.8019661147963306, 0.21585700622763651]])), ([0.45739456873683076, 0.4723681781529837, 0.5481545319813144, 0.5201280482289972], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.45762923624032836, 0.4733311466243086, 0.5486463422304386, 0.5202550736884323], [0.45847828140277963, 0.47395822794308823, 0.5492734216862739, 0.5212393746482942], [0.45741856600932373, 0.47286492154628523, 0.5481801206078991, 0.520151946440784], [0.45662012862270085, 0.4730500702758375, 0.5488879690554425, 0.5208597435800455], [0.4567387067766612, 0.4722007916037348, 0.5474844403082989, 0.5194879718948033], [0.4555597750873927, 0.4723378845686837, 0.547653077596858, 0.5195853961911872], [0.45720869614870474, 0.4731385862174772, 0.5484537775839107, 0.5198870925884427], [0.45647403451574187, 0.4730104212669337, 0.5483254723241286, 0.5202976058637068], [0.457558168093532, 0.47287184928701426, 0.5483196747071973, 0.5201751663884828], [0.45739456873683076, 0.4723681781529837, 0.5481545319813144, 0.5201280482289972]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.45762923624032836, 0.47333114662430859, 0.54864634223043862, 0.52025507368843227], [0.45762923624032836, 0.382740075312362, 0.54864634223043862, 0.52025507368843227], [0.45741856600932373, 0.3821934221139605, 0.54818012060789911, 0.520151946440784], [0.45741856600932373, 0.3822859964787366, 0.54818012060789911, 0.520151946440784], [0.45673870677666117, 0.3818613571426853, 0.5474844403082989, 0.51948797189480334], [0.45673870677666117, 0.3819299036251598, 0.5474844403082989, 0.51948797189480334], [0.45673870677666117, 0.3823302544495565, 0.5474844403082989, 0.51948797189480334], [0.45673870677666117, 0.38226617197428475, 0.5474844403082989, 0.51948797189480334], [0.45673870677666117, 0.382196885984325, 0.5474844403082989, 0.51948797189480334], [0.45673870677666117, 0.38194505041730975, 0.5474844403082989, 0.51948797189480334]])), ([0.3343004747680123, 0.13059541916647663, 0.704057843672915, 0.6156050446694066], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.33443661913546413, 0.13270011578651597, 0.70419375978945, 0.6157414744119487], [0.33492220555695734, 0.12443769125501078, 0.7046795716692473, 0.6162252417097999], [0.3349218799586451, 0.1268731847077733, 0.7046792527498924, 0.6162267272291662], [0.3337627239063198, 0.13032303851452828, 0.7035200949362047, 0.6150673621974805], [0.3341321314209922, 0.13027984072239013, 0.7038895007140481, 0.6153984369827895], [0.3337820417426537, 0.12920867916124953, 0.7035341067842228, 0.6150868973841642], [0.33543181468164057, 0.1286262747254821, 0.7051891450979119, 0.6167366712979134], [0.3352604859271879, 0.1277694037771463, 0.7050259695156713, 0.6165734524903563], [0.33381569540628414, 0.13214430046252296, 0.7035730600165258, 0.6151201931701945], [0.3343004747680123, 0.13059541916647663, 0.704057843672915, 0.6156050446694066]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.33443661913546413, 0.13270011578651597, 0.70419375978944998, 0.7228865115366088], [0.33443661913546413, 0.12443769125501078, 0.70467957166924733, 0.7231283951855343], [0.33443661913546413, 0.13270011578651597, 0.70419375978944998, 0.7231291379452175], [0.33492220555695734, 0.12687318470777331, 0.7046792527498924, 0.7225494554293747], [0.33492220555695734, 0.13270011578651597, 0.70467957166924733, 0.7227149928220291], [0.33492220555695734, 0.12920867916124953, 0.7046792527498924, 0.7225592230227165], [0.33492220555695734, 0.12920867916124953, 0.70352009493620471, 0.61508689738416422], [0.33526048592718788, 0.12920867916124953, 0.70518914509791186, 0.7233025005758126], [0.33376272390631978, 0.13214430046252296, 0.70353410678422279, 0.7225758709157317], [0.33376272390631978, 0.13032303851452828, 0.7038895007140481, 0.7228182966653377]])), ([0.3765694951624022, 0.7689868049103987, 0.3654336168291499, 0.3651205228681438], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.37647922710834936, 0.7688965285423056, 0.36540845317480614, 0.3654400425161305], [0.3793065966332509, 0.7723885500315584, 0.36893219016406, 0.3689322620168852], [0.3796430728054761, 0.7720603819839662, 0.3686039215312269, 0.36860358729015275], [0.37589495100615555, 0.7706164122288801, 0.36681761209669533, 0.36716011907633606], [0.3820595304539434, 0.7744768388730998, 0.3710205572168516, 0.37100390764811814], [0.3786656979907665, 0.7711118495704016, 0.36715893901791424, 0.3676555693732005], [0.3790530364004105, 0.771501271561867, 0.366089873238824, 0.3680449268979146], [0.3779862933428221, 0.7704040143047985, 0.3669477375789984, 0.36694371302156764], [0.37726282169708, 0.7696831751267831, 0.36622690226348537, 0.3662256092296037], [0.3765694951624022, 0.7689868049103987, 0.3654336168291499, 0.3651205228681438]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.37647922710834936, 0.76889652854230561, 0.36540845317480614, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.18446609508203, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.18430196076561345, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.18340880604834767, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.1855102786084258, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.18357946950895712, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.36540845317480614, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.1834738687894992, 0.36544004251613049], [0.37647922710834936, 0.76889652854230561, 0.18311345113174268, 0.36544004251613049], [0.37647922710834936, 0.76898680491039872, 0.18271680841457494, 0.36512052286814378]])), ([0.21340207187229074, 0.9773048861733349, 1.5713187533939424e-06, 1.571318753395881e-06], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.2172244289784722, 0.9773095132055053, 2.4772858236313956e-06, 2.531668159248581e-06], [0.21510669970002033, 0.9773092050285863, 3.3956290045279246e-06, 3.3795487953498857e-06], [0.21488644401613577, 0.9773164569027906, 3.100468160333987e-06, 3.654791403186369e-06], [0.21661139564233753, 0.9773170056524779, 1.964612451142924e-06, 1.8631878093016318e-06], [0.21794242182602203, 0.977315141269599, 2.589586658320348e-06, 3.885334132113208e-06], [0.2147870305307587, 0.9773190579777911, 2.439912357328267e-06, 2.5466374713093068e-06], [0.2148513243096049, 0.9773220617107325, 5.08610114906061e-06, 5.6544171405282245e-06], [0.2152433448140604, 0.977307298447571, 1.5719420036838107e-06, 1.57194200368575e-06], [0.21707928095393475, 0.9773126866486633, 1.6937320192946516e-06, 1.6762454193689735e-06], [0.21340207187229074, 0.9773048861733349, 1.5713187533939424e-06, 1.571318753395881e-06]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.30577909608919496, 0.9773130336910811, 1.2386429118156978e-06, 1.2658340796242905e-06], [0.30472023144996907, 0.9773128796026217, 1.6978145022639623e-06, 2.5316681592485809e-06], [0.30461010360802676, 0.9773165055397238, 1.5502340801669936e-06, 1.8273957015931844e-06], [0.30547257942112765, 0.9773167799145674, 9.82306225571462e-07, 1.8631878093016318e-06], [0.30613809251296986, 0.9773158477231281, 1.294793329160174e-06, 1.942667066056604e-06], [0.3045603968653382, 0.9773178060772241, 1.2199561786641336e-06, 1.2733187356546534e-06], [0.2147870305307587, 0.9773193079436948, 2.543050574530305e-06, 2.8272085702641123e-06], [0.30478855400698907, 0.977311926312114, 7.859710018419053e-07, 7.85971001842875e-07], [0.3057065220769263, 0.9773146204126602, 8.468660096473258e-07, 8.381227096844868e-07], [0.21524334481406041, 0.977310720174996, 7.856593766969712e-07, 7.856593766979405e-07]])), ([0.03589867527695376, 0.03692071918719155, 0.9981336407386341, 0.036290002154249976], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[0.03585627243472798, 0.03703789939855549, 0.9983966246481595, 0.03718369771283421], [0.036537087935715314, 0.037147123773409334, 0.998360044791114, 0.032915545420207463], [0.036968663866546854, 0.03712164281194838, 0.9983345637679957, 0.03497818007459288], [0.03702774175107857, 0.03608271134314213, 0.9982406634022241, 0.03679512576098609], [0.03599949173083897, 0.03591496745710119, 0.9982785888989105, 0.03706565615197311], [0.036985238178374374, 0.0371911794435242, 0.9984041010808781, 0.034965715895459354], [0.03620924858669464, 0.036841085680663695, 0.9980540076953524, 0.03668515723109553], [0.03709321155904061, 0.03678233109597424, 0.9983061319582444, 0.03341510057597568], [0.03302672948783428, 0.03700800439098674, 0.9982209261906603, 0.036277301690885666], [0.03589867527695376, 0.03692071918719155, 0.9981336407386341, 0.036290002154249976]], [[0.0, 0.0, 0.9612129216540486, 0.0], [0.035856272434727977, 0.037037899398555493, 0.979804773151104, 0.03718369771283421], [0.035856272434727977, 0.037037899398555493, 0.9797864832225813, 0.03718369771283421], [0.036968663866546854, 0.03712164281194838, 0.99836004479111395, 0.032915545420207463], [0.03702774175107857, 0.036082711343142129, 0.9797267925281363, 0.036795125760986092], [0.036537087935715314, 0.035914967457101189, 0.9797457552764796, 0.032915545420207463], [0.03702774175107857, 0.037147123773409334, 0.9798085113674633, 0.032915545420207463], [0.035999491730838967, 0.036841085680663695, 0.9796334646747005, 0.036685157231095528], [0.036209248586694637, 0.036841085680663695, 0.9797595268061465, 0.036685157231095528], [0.036537087935715314, 0.037147123773409334, 0.9797169239223544, 0.036685157231095528], [0.036209248586694637, 0.037147123773409334, 0.9796732811963413, 0.036685157231095528]])), ([0.1682509801524947, 0.9389851473947625, 0.1682495594311011, 0.257455666852824], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[0.16770095277274533, 0.9384351185923676, 0.16719036689552724, 0.2567460289739367], [0.16813618765205784, 0.9388703579454261, 0.16502880148241664, 0.25733305473537127], [0.16781149924930047, 0.9385456693460833, 0.16766771340533249, 0.25689679143001787], [0.15891245330408402, 0.9385273922356712, 0.16779296991947845, 0.2569983364610455], [0.1678231113505677, 0.9385572811907602, 0.16627746966431145, 0.2570282406857594], [0.1684579463716413, 0.9391921159800862, 0.16248107077897941, 0.2576615263390606], [0.16779121356839963, 0.9385253823624269, 0.1677763010953549, 0.25689551649080605], [0.16702149306174546, 0.9384468619760786, 0.16776135718781646, 0.25696647928347477], [0.16774350025388662, 0.938477503082777, 0.16774322070265096, 0.2569486320717737], [0.1682509801524947, 0.9389851473947625, 0.1682495594311011, 0.257455666852824]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.16770095277274533, 0.93843511859236761, 0.16719036689552724, 0.17297558039463784], [0.16813618765205784, 0.93843511859236761, 0.16719036689552724, 0.1732690932753551], [0.16770095277274533, 0.93887035794542606, 0.16719036689552724, 0.1730509616226784], [0.16813618765205784, 0.93843511859236761, 0.16719036689552724, 0.25674602897393672], [0.16813618765205784, 0.93887035794542606, 0.16719036689552724, 0.17311668625054918], [0.16770095277274533, 0.93919211598008623, 0.16719036689552724, 0.17343332907719977], [0.16770095277274533, 0.93919211598008623, 0.16248107077897941, 0.1730503241530725], [0.16770095277274533, 0.93843511859236761, 0.16719036689552724, 0.17308580554940686], [0.1684579463716413, 0.93852538236242689, 0.16777630109535491, 0.17307688194355633], [0.16770095277274533, 0.93919211598008623, 0.16777630109535491, 0.17333039933408148]])), ([0.5396526728109766, 0.337252963748538, 0.6973212758758698, 0.33725404419410704], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.5412710821115307, 0.33886611482796003, 0.698937261891341, 0.3384003077538114], [0.5415072494845717, 0.33910620719714085, 0.6991662942853625, 0.3391056046482941], [0.5406205448951897, 0.3374979147528943, 0.6982867136050118, 0.33821950101973036], [0.5383861960504045, 0.33597528740321897, 0.695970756296341, 0.33598515492993675], [0.5383123515353088, 0.3359536615533553, 0.696044481239864, 0.335977265070627], [0.5414642884707108, 0.3310465075104488, 0.6991364998263918, 0.3390692620702395], [0.540019308117436, 0.337591955036909, 0.6976854878068385, 0.3376180823401333], [0.5393379143710626, 0.3369365393245674, 0.6970040888929385, 0.33693644585603966], [0.5411845430468688, 0.33745820388865466, 0.6988507153698664, 0.3387814821447815], [0.5396526728109766, 0.337252963748538, 0.6973212758758698, 0.33725404419410704]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.37183606100725647, 0.33886611482796003, 0.69893726189134098, 0.33840030775381141], [0.37195414469377697, 0.33886611482796003, 0.69893726189134098, 0.33840030775381141], [0.54062054489518974, 0.33749791475289431, 0.69828671360501182, 0.33821950101973036], [0.37039361797669335, 0.33597528740321897, 0.69597075629634098, 0.33598515492993675], [0.3703566957191455, 0.33597528740321897, 0.69597075629634098, 0.33598515492993675], [0.3719326641868465, 0.33595366155335532, 0.69604448123986395, 0.33597726507062697], [0.3712101740102091, 0.33595366155335532, 0.69604448123986395, 0.33597726507062697], [0.3708694771370224, 0.33597528740321897, 0.69604448123986395, 0.33597726507062697], [0.3717927914749255, 0.33595366155335532, 0.69604448123986395, 0.33597726507062697], [0.3710268563569794, 0.33595366155335532, 0.69604448123986395, 0.33597726507062697]])), ([0.4842545190695682, 0.4881132603015269, 0.4842545066393838, 0.5610716604689464], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[0.4811864121315025, 0.4850963959605333, 0.4812348088003576, 0.5580518986691767], [0.4820141190999188, 0.48587569470377456, 0.48201409821979413, 0.5588323066136015], [0.48478610603508937, 0.4886838161348327, 0.4766519511853615, 0.5616404272048955], [0.4807538819507097, 0.48461849469201135, 0.48073911072429376, 0.5575751096535725], [0.47908187062814644, 0.48294345489839535, 0.4790671654737278, 0.5553967578086058], [0.4850059517082947, 0.48872905895087027, 0.485005932922166, 0.5618241348602007], [0.4844579521128844, 0.48273327967418017, 0.48446661902212146, 0.561284810285268], [0.48047775863946235, 0.4841191330538404, 0.48036669278650534, 0.557295956616684], [0.4786873184797121, 0.48722558090734924, 0.4833639890619261, 0.559899830074348], [0.4842545190695682, 0.4881132603015269, 0.4842545066393838, 0.5610716604689464]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.48118641213150248, 0.48509639596053328, 0.48123480880035763, 0.5580518986691767], [0.48118641213150248, 0.24486864093345978, 0.48123480880035763, 0.5580518986691767], [0.48118641213150248, 0.24627270164898885, 0.48123480880035763, 0.5580518986691767], [0.48075388195070973, 0.48868381613483269, 0.48123480880035763, 0.5580518986691767], [0.47908187062814644, 0.24340252103077017, 0.4790671654737278, 0.55539675780860576], [0.47908187062814644, 0.24629532305700763, 0.4790671654737278, 0.55539675780860576], [0.47908187062814644, 0.24329743341866258, 0.4790671654737278, 0.55539675780860576], [0.47908187062814644, 0.2439903601084927, 0.4790671654737278, 0.561284810285268], [0.48445795211288439, 0.24554358403524712, 0.4790671654737278, 0.561284810285268], [0.48445795211288439, 0.24598742373233595, 0.4790671654737278, 0.561284810285268]])), ([0.16908464739613072, 0.7652949668147021, 0.16962694660247604, 0.598830562465113], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[0.16912929122908987, 0.764931904534164, 0.16815913810268127, 0.5983326728331553], [0.16911106318507535, 0.7649138696771369, 0.16911125437040564, 0.5983148001846536], [0.16354233122357195, 0.7668958184010906, 0.1710682854435549, 0.6002968658478974], [0.16997694527575255, 0.7658647306980881, 0.17006199871785063, 0.5992660561301743], [0.16881729795889042, 0.7648578515650443, 0.16905517275266985, 0.5982588380254836], [0.16999023010729414, 0.7672537002076839, 0.16939251856442455, 0.6006546953422903], [0.1701916965035544, 0.7664161223655653, 0.17061399882757602, 0.5998175883903194], [0.17001318361532083, 0.765815798073337, 0.16904608819299569, 0.5991799502714357], [0.170312534221667, 0.7661145852217052, 0.1641898985000148, 0.5995161520208757], [0.16908464739613072, 0.7652949668147021, 0.16962694660247604, 0.598830562465113]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.08456464561454494, 0.76493190453416404, 0.16815913810268127, 0.59833267283315528], [0.08455553159253767, 0.76493190453416404, 0.16815913810268127, 0.59831480018465355], [0.08177116561178598, 0.76491386967713693, 0.1710682854435549, 0.59831480018465355], [0.08498847263787628, 0.76689581840109056, 0.1710682854435549, 0.60029686584789743], [0.08440864897944521, 0.76493190453416404, 0.1710682854435549, 0.60029686584789743], [0.08499511505364707, 0.76689581840109056, 0.1710682854435549, 0.60029686584789743], [0.0850958482517772, 0.76493190453416404, 0.16905517275266985, 0.60029686584789743], [0.08500659180766042, 0.76689581840109056, 0.16815913810268127, 0.59833267283315528], [0.0851562671108335, 0.76485785156504427, 0.16905517275266985, 0.59825883802548363], [0.08454232369806536, 0.76485785156504427, 0.16905517275266985, 0.60029686584789743]]))], (3, 'ACH_solution', 'DTLZ1'): [([0.38201114957739046, 3.82011531599907e-07, 0.6177352940388894], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.44367542250133973, 4.4367586618996413e-07, 0.5560709590765491], [0.374881373483999, 3.7488174837652734e-07, 0.6248650767339089], [0.41728206330104023, 4.1728248059552005e-07, 0.5824643445373185], [0.02228148051633827, 2.2281502798481787e-08, 0.4777184972931084], [0.7629797421978257, 0.2687655576148471, 0.9675248480746376], [0.3401412860738154, 3.4014162622522256e-07, 0.6596051991769137], [0.5990167279217403, 0.10086990420570517, 0.7996291931419137], [0.4034535427196068, 4.034539461851546e-07, 0.5962928796648012], [0.0004943160835131052, 4.943165778438975e-10, 0.4995056836710379], [0.38201114957739046, 3.82011531599907e-07, 0.6177352940388894]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.44367542250133973, 2.2183793309498207e-07, 0.6274151247752997], [0.37488137348399903, 1.8744087418826367e-07, 0.6618121836039795], [0.37488137348399903, 3.7488174837652734e-07, 0.6406118175056843], [0.41728206330104023, 1.1140751399240894e-08, 0.5882388938835793], [0.76297974219782572, 0.26876555761484711, 0.8331420692743439], [0.34014128607381539, 1.7007081311261128e-07, 0.679182244825482], [0.02228148051633827, 3.4014162622522256e-07, 0.749194241807982], [0.34014128607381539, 2.017269730925773e-07, 0.6475260850694257], [0.00049431608351310525, 2.4715828892194873e-10, 0.5991324870725441], [0.34014128607381539, 1.910057657999535e-07, 0.6582472922564697]])), ([0.17222577966229174, 0.1722257807244458, 1.1551558907489825], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[0.005605519748182468, 0.005620117127523093, 0.9885319318677919], [0.005547308492101731, 0.005634720978328769, 0.9885648650186976], [4.0711995465000647e-07, 9.288004561000774e-08, 0.49999950026001394], [2.39297494428458e-07, 2.607025056188375e-07, 0.49999950004729543], [0.005607980741671453, 0.00560070878916081, 0.9885381373223102], [0.0055861772595841575, 0.005616595205281417, 0.9885467432608681], [2.466971394157754e-07, 2.533028607435638e-07, 0.499999500159339], [3.1703689877923655e-07, 1.8296310154095176e-07, 0.499999500320188], [3.1425799261834944e-07, 1.8574200750190995e-07, 0.49999950012025923], [0.17222577966229174, 0.1722257807244458, 1.1551558907489825]], [[0.0, 0.0, 0.9829301429354526], [0.002802759874091234, 0.0028100585637615464, 0.9857310374016222], [0.0027736542460508654, 0.0028173604891643844, 0.9857475039770751], [2.0355997732500323e-07, 9.2880045610007744e-08, 0.49999950026001394], [1.19648747214229e-07, 2.6070250561883749e-07, 0.49999950004729543], [0.0028039903708357264, 0.002800354394580405, 0.9857341401288814], [0.0027930886297920787, 0.0028082976026407086, 0.9857384430981604], [1.233485697078877e-07, 1.266514303717819e-07, 0.49999950026001394], [1.5851844938961827e-07, 9.148155077047588e-08, 0.49999950004729543], [1.5712899630917472e-07, 9.287100375095498e-08, 0.49999950004729543], [0.08611288983114587, 0.0056201171275230928, 0.98853813732231022]])), ([0.3332484925990307, 0.33324917527940917, 0.3332491791367967], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.4988959376135403, 0.5002989126797819, 0.500298920149805], [0.1666667132115165, 0.16666671550027376, 0.16666668175122623], [0.4998310295539401, 0.49983139977829305, 0.4998314018919188], [0.333248937384443, 0.33324894478427175, 0.33324894863251403], [0.1663045817849323, 0.16684930367647627, 0.16684930465165407], [0.1666669833750251, 0.1666662851101282, 0.16666698473611286], [0.4998312128006642, 0.4998312292460879, 0.4998312320347009], [0.16666776564515848, 0.16666750004997558, 0.16666776633331645], [0.33326560768597635, 0.3332656077561835, 0.3332606917202903], [0.3332484925990307, 0.33324917527940917, 0.3332491791367967]], [[0.0, 0.0, 0.0], [0.49889593761354029, 0.50029891267978188, 0.2501494600749025], [0.16666671321151649, 0.16666671550027376, 0.08333334087561312], [0.16666671321151649, 0.16666671550027376, 0.2499157009459594], [0.16666671321151649, 0.16666671550027376, 0.16662447431625702], [0.16666671321151649, 0.16666671550027376, 0.08342465232582703], [0.16666698337502511, 0.16666671550027376, 0.08333349236805643], [0.16630458178493229, 0.16684930367647627, 0.24991561601735046], [0.16666671321151649, 0.16666671550027376, 0.08333388316665823], [0.16666671321151649, 0.16666671550027376, 0.16663034586014516], [0.16666671321151649, 0.16666671550027376, 0.16662458956839835]])), ([0.6388403370667414, 0.3609054883264573, 9.997468251687722e-07], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[1.0974294659118273, 0.9676874436094395, 0.43387051081355205], [0.6508857673450624, 0.34886005814072885, 9.997468252613648e-07], [0.31820581266909925, 0.1817936874405018, 5.00000000123979e-07], [0.3324878910742122, 0.16751160915129604, 5.000000002398864e-07], [0.7642627776727839, 0.6345261185767528, 0.10070860172449674], [0.764268448377024, 0.6345179410403362, 0.10070949332250453], [1.430612061949741, 1.3008706219464783, 0.7670143367720589], [0.5481804425906516, 0.4515653827623571, 9.997468251285822e-07], [0.7642546058554205, 0.6345291649242533, 0.10071164889913495], [0.6388403370667414, 0.3609054883264573, 9.997468251687722e-07]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.8804942113654559, 0.96768744360943948, 0.43387051081355205], [0.6572223620820734, 0.34886005814072885, 4.998734126306824e-07], [0.49088238474409185, 0.18179368744050181, 2.500000000619895e-07], [0.49802342394664834, 0.34886005814072885, 2.500000001199432e-07], [0.7139108672459342, 0.16751160915129604, 0.10070860172449674], [0.7139137025980542, 0.16751160915129604, 5.0000000023988638e-07], [1.0, 0.34886005814072885, 0.10070949332250453], [0.605869699704868, 0.16751160915129604, 5.0000000023988638e-07], [0.7139067813372525, 0.16751160915129604, 5.0000000023988638e-07], [0.6511996469429129, 0.34886005814072885, 4.998734125843861e-07]])), ([0.2490954526073135, 0.249096863380102, 0.5015546485224129], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.2490956057659496, 0.24909681933058037, 0.5015546015111808], [0.24909640581129033, 0.2490963200973928, 0.5015541900514922], [0.24909099071530416, 0.24910060352599464, 0.5015583891427914], [0.24915662784286058, 0.2491740710118981, 0.5016318589800401], [0.24909619289134213, 0.24909642321386247, 0.5015542120520946], [0.5822608872569718, 0.5822609048149655, 0.8347186897574518], [0.2475271550210709, 0.249881416434525, 0.5023392003304341], [0.24945199013322744, 0.2484053220460063, 0.5019097736227383], [0.24923016206301848, 0.2489476222423502, 0.5016879546400355], [0.2490954526073135, 0.249096863380102, 0.5015546485224129]], [[0.0, 0.0, 0.2524577834161418], [0.24909560576594961, 0.24909681933058037, 0.3770061924636613], [0.24909640581129033, 0.2490963200973928, 0.37700598673381697], [0.24909640581129033, 0.2490963200973928, 0.3770080862794666], [0.24909640581129033, 0.2490963200973928, 0.37704482119809096], [0.24909619289134213, 0.24909642321386247, 0.3770059977341182], [0.24915662784286058, 0.24917407101189809, 0.5435882365867968], [0.24909619289134213, 0.24909642321386247, 0.37739849187328794], [0.24909619289134213, 0.24909642321386247, 0.37718377851944], [0.24945199013322744, 0.2490963200973928, 0.3770728690280886], [0.24909619289134213, 0.24909642321386247, 0.37700621596927736]])), ([0.20546787568465896, 0.5619569911222604, 0.232556577143573], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.20571015812042703, 0.5621992637377059, 0.2323669364314424], [0.2053901385414952, 0.5618762984426359, 0.2324807320223393], [0.5385536003856034, 0.8950426954922218, 0.5656441959075554], [0.2053796888092501, 0.5618830783837002, 0.23248457455574056], [0.20539015985578835, 0.5618792639894553, 0.2324779542417573], [0.37209876141089043, 0.7285878627157168, 0.39916544010360366], [0.038807464417703816, 0.39529574505222953, 0.0658979880001453], [0.2056220615278386, 0.5621111649357105, 0.23254805502676223], [0.20535046106144544, 0.562178598310406, 0.23278008706068007], [0.20546787568465896, 0.5619569911222604, 0.232556577143573]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.20571015812042703, 0.5621992637377059, 0.12972876465585345], [0.20571015812042703, 0.5621992637377059, 0.1297856624513019], [0.20539013854149521, 0.56187629844263587, 0.29636739439390997], [0.20571015812042703, 0.5621992637377059, 0.12978758371800253], [0.20539015985578835, 0.56187926398945531, 0.1297842735610109], [0.20539013854149521, 0.5621992637377059, 0.21312801649193408], [0.038807464417703816, 0.39529574505222953, 0.0464942904402049], [0.038807464417703816, 0.39529574505222953, 0.12981932395351337], [0.038807464417703816, 0.39529574505222953, 0.1299353399704723], [0.038807464417703816, 0.39529574505222953, 0.12982358501191876]])), ([0.5737754648637374, 5.737760386562754e-07, 0.4259707864587255], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.2614619372616376, 2.614621987313548e-07, 0.2385378012868218], [1.0448238359828366, 0.21200585995320362, 0.742427214146729], [1.0447668391138092, 0.21194979539852316, 0.7425244753008995], [0.6008682871758975, 6.00868888062064e-07, 0.39887793705580327], [0.8781684292178425, 0.04537839228171393, 0.5759530612091089], [0.6302229807443472, 6.302236109860807e-07, 0.36952321422775447], [0.13145842965820928, 1.3145856112055058e-07, 0.3685414389628992], [3.2483208711057145e-05, 3.640183179925847e-09, 0.4999680880181042], [0.0012930902285254223, 1.2930915216541276e-09, 0.49870690850680477], [0.5737754648637374, 5.737760386562754e-07, 0.4259707864587255]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.26146193726163758, 1.307310993656774e-07, 0.23853780128682181], [0.26146193726163758, 2.6146219873135481e-07, 0.6365009467520563], [0.26146193726163758, 2.6146219873135481e-07, 0.6365495773291415], [0.26146193726163758, 3.00434444031032e-07, 0.23853780128682181], [0.26146193726163758, 2.6146219873135481e-07, 0.23853780128682181], [0.26146193726163758, 3.1511180549304036e-07, 0.23853780128682181], [0.13145842965820928, 6.572928056027529e-08, 0.44955805916014135], [3.2483208711057145e-05, 1.8200915899629234e-09, 0.5152713836877438], [3.2483208711057145e-05, 6.465457608270638e-10, 0.5146407939320942], [0.0012930902285254223, 2.868880193281377e-07, 0.4782727329080545]])), ([0.15315038723992366, 0.7016738004227047, 0.14504556411049055], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.15282114351318832, 0.7013445541734246, 0.14618757185393036], [0.0011358316508682388, 0.49886366838967705, 5.000000000549232e-07], [0.3191850802418067, 0.8677319431378563, 0.31257680567866597], [0.15262254555788235, 0.7011459592799155, 0.1459898097892322], [0.15266531512222747, 0.7011887306755011, 0.14602734450175503], [0.3191922896029436, 0.8677302640686788, 0.3125751260700787], [0.00168555798257357, 0.4983139420323922, 5.000000000293436e-07], [0.1526186370627601, 0.7011423396701081, 0.14598721331485368], [0.15315038723992366, 0.7016738004227047, 0.14504556411049055]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.15282114351318832, 0.70134455417342456, 0.07309378592696518], [0.003883779363750652, 0.49886366838967705, 2.500000000274616e-07], [0.0011358316508682388, 0.49886366838967705, 5.0000000005492318e-07], [0.0011358316508682388, 0.49886366838967705, 0.0729949048946161], [0.15262254555788235, 0.70114595927991552, 0.07301367225087751], [0.15266531512222747, 0.70118873067550114, 0.15628756303503935], [0.0041586425296033175, 0.49831394203239221, 2.500000000146718e-07], [0.0016855579825735701, 0.49831394203239221, 5.0000000002934363e-07]])), ([0.06062063561699689, 0.060707245242586166, 0.8784189443363961], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[0.22727906587066418, 0.22727906964626482, 1.0449356834167876], [3.0370573835283423e-07, 1.9629426170702898e-07, 0.49999950005986316], [0.06057076030973772, 0.06073233504206014, 0.8784440251943699], [0.39384294220764704, 0.3938429426702676, 1.2115546370219104], [0.06087019031009199, 0.060503779769262696, 0.8785818815749443], [0.06067981297035381, 0.06067815066445259, 0.8783915023853542], [1.5673357975414e-07, 3.432664202765021e-07, 0.4999995000306421], [1.878667636657002e-07, 3.121332363484662e-07, 0.49999950001416643], [0.060678478868052256, 0.06067838366745593, 0.8783901691628417], [0.06062063561699689, 0.060707245242586166, 0.8784189443363961]], [[0.0, 0.0, 0.8177116892877433], [0.11363953293533209, 0.22727906964626482, 1.0], [1.5185286917641712e-07, 9.814713085351449e-08, 0.49999950005986316], [3.0370573835283423e-07, 1.9629426170702898e-07, 0.49999950005986316], [0.19692147110382352, 1.9629426170702898e-07, 0.49999950005986316], [0.030435095155045996, 1.9629426170702898e-07, 0.87858188157494432], [3.0370573835283423e-07, 1.9629426170702898e-07, 0.49999950005986316], [7.836678987707e-08, 1.7163321013825106e-07, 0.49999950005986316], [9.39333818328501e-08, 1.560666181742331e-07, 0.49999950001416643], [0.030339239434026128, 1.9629426170702898e-07, 0.49999950005986316], [3.0370573835283423e-07, 1.9629426170702898e-07, 0.49999950005986316]])), ([2.4368674800478324e-07, 0.24368650431803524, 0.2563132523897456], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[0.2637232601952275, 0.9427660817044652, 0.79275303736888], [6.140900224310133e-07, 0.5102432077660858, 0.4895030033341865], [0.0972137840111412, 0.7762554452854358, 0.6261376198768408], [0.2637233677301513, 0.9427650790825423, 0.7927520337692936], [0.0972176011735768, 0.7762551443667217, 0.6262462158589334], [4.892684523491193e-08, 0.04892679630806669, 0.4510731549243829], [0.09714232896496303, 0.7761839466487815, 0.6261680045119898], [6.165338214408813e-07, 0.6165332049070599, 0.3832130038777553], [3.524844749040186e-08, 0.035248412241954366, 0.4647515525293601], [2.4368674800478324e-07, 0.24368650431803524, 0.2563132523897456]], [[0.0, 0.6790416599958826, 0.5290286140740371], [0.13186163009761376, 0.94276608170446519, 0.79275303736888003], [3.0704501121550667e-07, 0.51024320776608578, 0.4895030033341865], [0.0486068920055706, 0.51024320776608578, 0.4895030033341865], [0.13186168386507566, 0.51024320776608578, 0.6261376198768408], [0.0486088005867884, 0.51024320776608578, 0.4895030033341865], [2.4463422617455964e-08, 0.048926796308066692, 0.45107315492438288], [0.04857116448248151, 0.51024320776608578, 0.4895030033341865], [3.0826691072044065e-07, 0.61653320490705987, 0.38321300387775531], [1.762422374520093e-08, 0.61653320490705987, 0.45107315492438288], [3.524844749040186e-08, 0.24368650431803524, 0.45107315492438288]]))], (4, 'ACH_solution', 'DTLZ3'): [([2.8624475935955647, 2.793976803641279, 3.4876216670402846, 2.7939767918291847], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[0.8352881158588482, 0.7652646203604027, 1.459059276838017, 0.765366094666692], [2.8624475935955647, 2.793976803641279, 3.4876216670402846, 2.7939767918291847]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.83528811585884821, 0.76526462036040266, 1.0, 0.76536609466669203]])), ([1.9566752923312705, 1.9721215825338518, 2.0474367354160017, 2.019408627644731], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[1.456902652700088, 1.4723350270725257, 1.5475205979724067, 1.5196361473193218], [1.9566752923312705, 1.9721215825338518, 2.0474367354160017, 2.019408627644731]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [1.0, 0.8819284748770807, 1.0, 1.0]])), ([2.3762079012135384, 2.175014381678613, 2.7459654289494124, 2.657512537914718], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[2.3762079012135384, 2.175014381678613, 2.7459654289494124, 2.657512537914718]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688]])), ([0.8920505813622472, 1.2844678896412316, 0.8810115915728781, 0.8810116065327377], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.3764705520389708, 0.7688882405870511, 0.3654319666810017, 0.36543172705757127], [1.899202729189024, 2.2914668512880225, 1.8880948718531854, 1.8881637609253068], [1.8997114102196977, 2.2925438931569606, 1.8887454132508903, 1.8890876378244035], [1.3930487868425423, 1.7910563513480318, 1.387573518100531, 1.3876000928575012], [1.3964607049965836, 1.7894208381221541, 1.3859645619202603, 1.385964550047104], [3.4016078546503254, 3.7940251551967767, 3.3905687431206246, 3.390568824596367], [1.3968737797015691, 1.7892976792500361, 1.3858335209672652, 1.385841206631073], [1.8991440985004309, 2.2915565547161365, 1.8881051320059279, 1.8881047048180222], [1.9008327468256347, 2.292979814950777, 1.8897860575287653, 1.8897937802695821], [0.8920505813622472, 1.2844678896412316, 0.8810115915728781, 0.8810116065327377]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.37647055203897078, 0.76888824058705108, 0.18271598334050085, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.9440474359265927, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.9443727066254451, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.6937867590502655, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.6929822809601301, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 1.0, 1.0], [0.37647055203897078, 0.76888824058705108, 0.6929167604836326, 0.36543172705757127], [0.37647055203897078, 1.0, 0.9440525660029639, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.3654319666810017, 0.36543172705757127], [0.37647055203897078, 0.76888824058705108, 0.44050579578643906, 0.36543172705757127]])), ([2.010220081886251, 2.593202226251779, 1.6158861968036486, 1.6158863080172057], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[2.010220081886251, 2.593202226251779, 1.6158861968036486, 1.6158863080172057]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0]])), ([1.7151318055887106, 1.7151318201385528, 2.6763446398752913, 1.7151317891283144], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[1.7151318055887106, 1.7151318201385528, 2.6763446398752913, 1.7151317891283144]], [[0.0, 0.0, 0.9612129216540486, 0.0]])), ([3.2685610153174682, 4.039294919635037, 3.2685609615479785, 3.3577656975715353], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[1.24854463176969, 2.0204151928278185, 1.249571213568609, 1.3388861406963837], [3.2685610153174682, 4.039294919635037, 3.2685609615479785, 3.3577656975715353]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [1.0, 1.0, 1.0, 0.7140456362558613]])), ([2.05550808912783, 1.8531040541709785, 2.213174257731879, 1.8531067584705616], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[2.05550808912783, 1.8531040541709785, 2.213174257731879, 1.8531067584705616]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0]])), ([2.4810283387422323, 2.484889858706089, 2.4803927927769163, 2.5552103428461455], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[2.4810283387422323, 2.484889858706089, 2.4803927927769163, 2.5552103428461455]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874]])), ([3.232335237565485, 3.828136759063669, 3.2323346145279293, 3.6615388387085694], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[0.1691417775379749, 0.7649881161352293, 0.1690824346594075, 0.5983891171590171], [1.2211268761893657, 1.8168969517883717, 1.2211134295963941, 1.6503304912695744], [2.2288650601437543, 2.8246667905730303, 2.228865048014011, 2.6580686328609064], [3.232335237565485, 3.828136759063669, 3.2323346145279293, 3.6615388387085694]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.08457088876898745, 0.76498811613522932, 0.1690824346594075, 0.5983891171590171], [0.6105634380946828, 0.76498811613522932, 0.1690824346594075, 0.5983891171590171], [0.16914177753797491, 0.76498811613522932, 0.1690824346594075, 0.5983891171590171]]))], (3, 'ACH_solution', 'DTLZ3'): [([4.706582040660076, 4.208429984977035, 4.907189262981643], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[4.706582040660076, 4.208429984977035, 4.907189262981643]], [[0.4981520234633334, 0.0, 0.6987592904740502]])), ([3.6853927169227956, 3.685392655441289, 4.668322831099613], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[3.6853927169227956, 3.685392655441289, 4.668322831099613]], [[0.0, 0.0, 0.9829301429354526]])), ([2.308524076160192, 2.308524045821012, 2.3085240427051876], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[2.308524076160192, 2.308524045821012, 2.3085240427051876]], [[0.0, 0.0, 0.0]])), ([2.5550288724621497, 2.425287418863653, 1.8914699165589375], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[2.5550288724621497, 2.425287418863653, 1.8914699165589375]], [[0.6635589568190845, 0.5338175168551759, 0.0]])), ([1.0637218448696084, 1.0643414058926532, 1.3167991902899503], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[1.0637218448696084, 1.0643414058926532, 1.3167991902899503]], [[0.0, 0.0, 0.2524577834161418]])), ([4.48605128946316, 4.842540366049841, 4.513141750342488], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[1.0151180594108804, 1.3716085258998105, 1.0422100146213422], [4.48605128946316, 4.842540366049841, 4.513141750342488]], [[0.0, 0.35648910413162704, 0.027090592880264497], [1.0, 1.0, 0.5346503037508034]])), ([3.8238414773339704, 2.991022391605085, 3.5215981767552655], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[3.8238414773339704, 2.991022391605085, 3.5215981767552655]], [[0.8328179786838068, 0.0, 0.5305746793573836]])), ([3.2722226557982936, 3.820746068860175, 3.2655908798198476], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[3.2722226557982936, 3.820746068860175, 3.2655908798198476]], [[0.006631727076633065, 0.5551551402005014, 0.0]])), ([2.0035469490484394, 2.0035388119734256, 2.821258636995158], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[2.0035469490484394, 2.0035388119734256, 2.821258636995158]], [[0.0, 0.0, 0.8177116892877433]])), ([4.782950824753862, 5.46199246581955, 5.311979320390754], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[4.782950824753862, 5.46199246581955, 5.311979320390754]], [[0.0, 0.6790416599958826, 0.5290286140740371]]))], (4, 'ACH_solution', 'DTLZ1'): [([0.0017146957002465509, 3.989586690670535e-07, 0.49828440535374086, 5.000000000270344e-07], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[1.9962714437279106e-06, 3.121405712429675e-07, 0.49999719160033074, 5.000000000267235e-07], [0.12853827453133332, 0.05940090530178059, 0.7530940222537003, 0.05872114176681708], [3.6702724119081424e-06, 6.68490789377315e-08, 0.49999576299850196, 5.000000001343707e-07], [0.2544492511560526, 0.18239266411330465, 0.8782204234264471, 0.18451293824434115], [0.0006748511980397327, 4.337398788692765e-07, 0.49932419871570294, 5.163980704693117e-07], [0.25389169746463913, 0.18396968991912704, 0.8776628742435603, 0.18396975264635657], [0.1289203660933793, 0.059104057748348704, 0.7527971694406363, 0.05906151832457142], [0.3788666245117216, 0.308794894265195, 1.0026377902589665, 0.30894327600080335], [0.12898621366328397, 0.05893922458033564, 0.7527579865037399, 0.05906498901825093], [0.0017146957002465509, 3.989586690670535e-07, 0.49828440535374086, 5.000000000270344e-07]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [1.9962714437279106e-06, 1.5607028562148374e-07, 0.5968451541296194, 5.0000000002672355e-07], [0.12853827453133332, 3.1214057124296748e-07, 0.7233935694563042, 5.0000000002672355e-07], [1.9962714437279106e-06, 3.342453946886575e-08, 0.596844439828705, 2.5000000006718537e-07], [3.6702724119081424e-06, 6.6849078937731498e-08, 0.7859567700426776, 5.0000000013437074e-07], [1.9962714437279106e-06, 2.1686993943463825e-07, 0.5965086576873055, 2.5819903523465583e-07], [3.6702724119081424e-06, 6.6849078937731498e-08, 0.7856779954512342, 5.0000000013437074e-07], [3.6702724119081424e-06, 6.6849078937731498e-08, 0.7232451430497722, 5.0000000013437074e-07], [1.9962714437279106e-06, 3.1214057124296748e-07, 0.8481654534589373, 5.0000000002672355e-07], [0.00067485119803973269, 4.3373987886927651e-07, 0.49932419871570294, 5.1639807046931166e-07], [1.9962714437279106e-06, 1.9947933453352676e-07, 0.5959887610063245, 2.500000000135172e-07]])), ([0.2077686268646839, 0.22294824520685558, 0.2985300669297339, 0.27050212416153985], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[0.08263832122786839, 0.09825309838081768, 0.17356829552650507, 0.14554028691206009], [0.20773225482545343, 0.22323511204327323, 0.298550310778353, 0.27049412957433255], [0.33269411862754594, 0.34791429481868463, 0.4234566173097102, 0.39542861977981364], [0.33263905261484833, 0.348084753939845, 0.42340055990440695, 0.395371482082831], [0.08280668075870389, 0.0981686368851393, 0.17358809655433166, 0.14556008563169895], [0.08227293320871036, 0.0983841471886767, 0.1736993444842679, 0.1456452506944229], [0.2078656663887795, 0.2228752429697471, 0.2986271763827466, 0.2703852567777428], [0.0827655211010204, 0.09820824715280019, 0.1735274442694876, 0.14549943526265], [0.3327723705609458, 0.3476785572655847, 0.4235371485648953, 0.3955091395449698], [0.2077686268646839, 0.22294824520685558, 0.2985300669297339, 0.27050212416153985]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.082638321227868392, 0.19488751053122674, 0.17356829552650507, 0.14554028691206009], [0.082638321227868392, 0.25737851736245454, 0.29855031077835298, 0.27049412957433255], [0.082638321227868392, 0.3197181087501602, 0.29855031077835298, 0.27049412957433255], [0.082638321227868392, 0.31980333831074037, 0.29855031077835298, 0.14554028691206009], [0.082638321227868392, 0.09825309838081768, 0.17356829552650507, 0.14554028691206009], [0.082272933208710361, 0.19495303493515626, 0.17356829552650507, 0.14554028691206009], [0.082272933208710361, 0.2571985828256914, 0.17369934448426791, 0.14564525069442291], [0.082765521101020395, 0.194865084917218, 0.1735274442694876, 0.14549943526265], [0.082765521101020395, 0.098208247152800188, 0.1735274442694876, 0.14549943526265], [0.082765521101020395, 0.25723508394424566, 0.1735274442694876, 0.14554028691206009]])), ([0.38734307182754013, 0.1861494030853309, 0.757100263198016, 0.6686479310891101], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.2624061778160931, 0.0612126228738764, 0.6321636998115691, 0.5437111847679872], [0.14183319219130963, 1.4183333402872216e-07, 0.5001488957813071, 0.35776459614644557], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.3018578930158145, 0.1981418051385583], [0.6372154224873464, 0.43602326678661607, 1.0069743248950724, 0.9185212127159673], [0.512194252379859, 0.3111156054609034, 0.8820641896677913, 0.7936141328973382], [0.0908538395350044, 9.085393039154736e-08, 0.5107509930802825, 0.3981422918764077], [0.2624079781779214, 0.0612072741116757, 0.6321655475923855, 0.5437130371265201], [0.17036856230266897, 1.7036873267630071e-07, 0.44153925444627107, 0.38783883798185026], [0.013095810464669896, 1.3095823560870036e-08, 0.3411838334108085, 0.14572034304117698], [0.38734307182754013, 0.1861494030853309, 0.757100263198016, 0.6686479310891101]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.2624061778160931, 0.061212622873876403, 0.63216369981156906, 0.686871366714628], [0.14183319219130963, 0.061212622873876403, 0.63216369981156906, 0.5938980724038572], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.63216369981156906, 0.5140866768999135], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.50014889578130706, 0.874276380688618], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.3018578930158145, 0.19814180513855831], [0.090853839535004394, 0.061212622873876403, 0.63216369981156906, 0.6140869202688383], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.51075099308028249, 0.6868722928938944], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.3018578930158145, 0.19814180513855831], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.3018578930158145, 0.4878759458512229], [2.7523776836342115e-07, 2.6620426510588198e-08, 0.3018578930158145, 0.7493397398751895]])), ([0.03245232765220528, 0.42486963672582934, 0.02140093590985521, 0.02140438965656173], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[0.03245305578523296, 0.42487036611194573, 0.02141193809014498, 0.021355036626559513], [0.2822870079252471, 0.674700409427553, 0.27125358045517844, 0.2712535804159688], [0.15735232368258475, 0.549769819013399, 0.1463135453428201, 0.14631311846407158], [0.03244583514964621, 0.4248631452303591, 0.021320930815186285, 0.021401622138663604], [0.2822891710763848, 0.6746874634058723, 0.27125881227752374, 0.2712588124420688], [0.40724009070735395, 0.7996571922360416, 0.39616028951876814, 0.39620113046105937], [0.15750954836450642, 0.5499409917831036, 0.1464846974150659, 0.1464831341769212], [0.03241301803008485, 0.424839443870622, 0.021383173894511127, 0.021374968652457586], [0.03248654042290944, 0.42490384928650415, 0.02143786446266485, 0.021202865809046952], [0.03245232765220528, 0.42486963672582934, 0.02140093590985521, 0.02140438965656173]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.032453055785232957, 0.42487036611194573, 0.01070596904507249, 0.021355036626559513], [0.032453055785232957, 0.42487036611194573, 0.13562679022758922, 0.021355036626559513], [0.032453055785232957, 0.42487036611194573, 0.07315677267141005, 0.021355036626559513], [0.032445835149646211, 0.42487036611194573, 0.02141193809014498, 0.021355036626559513], [0.032445835149646211, 0.42486314523035912, 0.021320930815186285, 0.021401622138663604], [0.032445835149646211, 0.54976981901339905, 0.19808014475938407, 0.021355036626559513], [0.032453055785232957, 0.42487036611194573, 0.07324234870753295, 0.021355036626559513], [0.032445835149646211, 0.42487036611194573, 0.010691586947255563, 0.021401622138663604], [0.032445835149646211, 0.42486314523035912, 0.010718932231332424, 0.021401622138663604], [0.032486540422909438, 0.42490384928650415, 0.021437864462664848, 0.021202865809046952]])), ([0.2334084123759629, 0.766336413230954, 9.997458253814905e-07, 9.997468251283158e-07], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.3758565091452036, 0.12414224331231619, 7.477300633736225e-07, 5.000000002019611e-07], [0.4263435439648869, 1.009035439621241, 0.032057787134533125, 0.032057787127185294], [0.42628170169149765, 1.0092949606457475, 0.031964924409482856, 0.0319784064804324], [0.15620997102434736, 0.8435348545809178, 9.99745834037386e-07, 9.997468251266726e-07], [0.4262951846637261, 1.009277317302507, 0.03196114785183314, 0.03196142355571332], [0.2387678518005757, 0.261230281290194, 9.317772546757509e-07, 9.356332635822742e-07], [0.426293987963137, 1.0092784796683942, 0.031959554611089555, 0.031961925844050575], [0.4262942551072643, 1.0092907687019328, 0.03197421452455621, 0.03197106746438174], [0.1426866122860365, 0.35731214889096924, 7.389371250175667e-07, 5.000000001285087e-07], [0.2334084123759629, 0.766336413230954, 9.997458253814905e-07, 9.997468251283158e-07]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.37585650914520358, 0.12414224331231619, 3.7386503168681124e-07, 2.5000000010098056e-07], [0.37585650914520358, 0.12414224331231619, 7.4773006337362248e-07, 5.0000000020196111e-07], [0.41030773244570773, 0.12414224331231619, 0.032057787134533125, 0.032057787127185294], [0.2752718671121326, 0.12414224331231619, 4.99872917018693e-07, 4.998734125633363e-07], [0.41031447393182197, 0.12414224331231619, 0.031964924409482856, 0.032057787127185294], [0.37585650914520358, 0.12414224331231619, 4.6588862733787545e-07, 4.678166317911371e-07], [0.4103138755815274, 0.12414224331231619, 7.4773006337362248e-07, 5.0000000020196111e-07], [0.41031400915359106, 0.12414224331231619, 7.4773006337362248e-07, 5.0000000020196111e-07], [0.26851018774297714, 0.26123028129019399, 3.6946856250878337e-07, 2.5000000006425437e-07], [0.3138710877879403, 0.12414224331231619, 4.998729126907453e-07, 4.998734125641579e-07]])), ([0.2599548597812756, 0.2590312661554314, 1.2211677833856478, 0.259674494043822], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[0.38444380524278254, 0.3844435325167176, 1.3456567644424235, 0.38444385062894254], [0.13317108652815013, 0.13539019444887188, 1.0966031166621772, 0.13506791398076495], [0.009579793437479407, 0.009651495103538965, 0.9708648900820193, 0.009651977977927741], [0.13472972346158094, 0.1340882527463856, 1.095944481701222, 0.13473156034002548], [0.00966633542457139, 0.00954647411005747, 0.9708766018164293, 0.009666337340285906], [0.25981159995158626, 0.2589509455907985, 1.2210245228367682, 0.25952809177695896], [0.009654386204398806, 0.009659732804413568, 0.9708726458151056, 0.009642488331230482], [1.4422800737349975e-07, 3.5577149263884593e-07, 0.49999900001284564, 5.000000000267235e-07], [0.009619198531601513, 0.009674730437357133, 0.9708876566884819, 0.009590253109752932], [0.2599548597812756, 0.2590312661554314, 1.2211677833856478, 0.259674494043822]], [[0.0, 0.0, 0.9612129216540486, 0.0], [0.38444380524278254, 0.38444353251671759, 1.0, 0.38444385062894254], [0.13317108652815013, 0.13539019444887188, 1.0, 0.13506791398076495], [0.004789896718739703, 0.0048257475517694826, 0.9660389058680339, 0.004825988988963871], [0.0095797934374794065, 0.0096514951035389651, 1.0, 0.0096519779779277413], [0.004833167712285695, 0.004773237055028735, 0.966044761735239, 0.004833168670142953], [0.0096663354245713901, 0.0095464741100574693, 1.0, 0.0096663373402859059], [0.004827193102199403, 0.004829866402206784, 0.9660427837345771, 0.004821244165615241], [7.211400368674987e-08, 1.7788574631942297e-07, 0.7306059608334471, 2.500000000133618e-07], [0.004809599265800757, 0.004837365218678566, 0.9660502891712652, 0.004795126554876466], [1.4422800737349975e-07, 3.5577149263884593e-07, 1.0, 5.0000000002672355e-07]])), ([0.03493805414894137, 0.8056910867876216, 0.034958056098527035, 0.1241631879052108], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[0.15989063890544064, 0.9306248312317836, 0.1598823947277185, 0.2490958269264328], [0.034974322188020227, 0.8057078195175575, 0.034956353653196603, 0.1241794554723197], [4.1968567160000636e-07, 0.41968525191433476, 4.1968609129816606e-07, 0.08031390872620359], [4.4148357503353833e-07, 0.4414831335499633, 4.4148401653025005e-07, 0.05851598349487964], [0.03491301487428884, 0.8056854744980909, 0.03497460978998292, 0.12417974182938912], [0.03495860975964235, 0.8056927781047084, 0.03495273175499452, 0.12415351623531322], [4.4537319940654043e-07, 0.44537275403334103, 4.4537364479299225e-07, 0.054626355231005806], [7.755704589126249e-07, 0.47203189191515205, 4.720331395323241e-07, 0.02796686060142008], [4.779811581274697e-07, 0.4779806801463116, 4.779816361228506e-07, 0.02201836391136662], [0.03493805414894137, 0.8056910867876216, 0.034958056098527035, 0.1241631879052108]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.15989063890544064, 0.93062483123178363, 0.15988239472771851, 0.1691504793708859], [0.034974322188020227, 0.80570781951755754, 0.034956353653196603, 0.10669229364382933], [2.0984283580000318e-07, 0.41968525191433476, 2.0984304564908303e-07, 0.08475952027077127], [2.2074178751676916e-07, 0.41968525191433476, 2.2074200826512503e-07, 0.0738605576551093], [4.1968567160000636e-07, 0.41968525191433476, 4.1968609129816606e-07, 0.10669243682236404], [0.034913014874288842, 0.41968525191433476, 0.034974609789982923, 0.1066793240253261], [4.1968567160000636e-07, 0.41968525191433476, 2.2268682239649613e-07, 0.07191574352317238], [3.8778522945631244e-07, 0.44148313354996332, 2.3601656976616204e-07, 0.058585996208379516], [2.3899057906373487e-07, 0.44537275403334103, 2.389908180614253e-07, 0.05561174786335279], [4.1968567160000636e-07, 0.41968525191433476, 4.1968609129816606e-07, 0.10668415986027488]])), ([0.31171417735652673, 0.10932279199195086, 0.469389182629487, 0.10932279283183229], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[0.4366566152458972, 0.23425674059905738, 0.5943239595893172, 0.2342564712491091], [0.43665922428744686, 0.23424310558730288, 0.594329388217922, 0.23426218314996], [0.3116018072136629, 0.10935894035057984, 0.46942920289403833, 0.10936198746549472], [0.4366585236098302, 0.23425544572661267, 0.5943250078592023, 0.23425778827641816], [0.3114941645428169, 0.10940844112751352, 0.46947566342284375, 0.10940633323902416], [0.5615941295342018, 0.35919308176677267, 0.7192602411123956, 0.3591931094949848], [0.1672834724648291, 1.672836397532792e-07, 0.33271586026387684, 5.000000000267235e-07], [0.3107809436450607, 0.10963445566728815, 0.4697204342108722, 0.10965321620067671], [0.4366721483000797, 0.2342640331025053, 0.5943383325322503, 0.2342443341192576], [0.31171417735652673, 0.10932279199195086, 0.469389182629487, 0.10932279283183229]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.3195288275744397, 0.23425674059905738, 0.59432395958931716, 0.23425647124910909], [0.3195301320952145, 0.23425674059905738, 0.59432395958931716, 0.23425647124910909], [0.25700142355832256, 0.10935894035057984, 0.46942920289403833, 0.10936198746549472], [0.31952978175640623, 0.10935894035057984, 0.46942920289403833, 0.10936198746549472], [0.2569476022228996, 0.10940844112751352, 0.46942920289403833, 0.10940633323902416], [0.381997584718592, 0.10935894035057984, 0.46942920289403833, 0.10936198746549472], [0.18484225618390565, 8.36418198766396e-08, 0.33271586026387684, 2.500000000133618e-07], [0.25659099177402145, 1.672836397532792e-07, 0.33271586026387684, 0.10936198746549472], [0.31953659410153096, 0.10935894035057984, 0.33271586026387684, 0.10936198746549472], [0.25705760862975446, 1.672836397532792e-07, 0.46938918262948698, 0.10965321620067671]])), ([0.35469497792588967, 0.35856798819924324, 0.35470640097443756, 0.4315245561298736], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[0.2297647660813352, 0.23363451697244098, 0.22975742055650222, 0.3065911290982631], [0.10483176338535306, 0.10869335066312064, 0.10483138108571016, 0.18164987326566806], [0.1048241532372313, 0.10869709652356897, 0.10483550964695935, 0.1816452303627665], [0.2297203934698725, 0.233643894545268, 0.22978244409718157, 0.3066006451964571], [0.230260574691852, 0.23412216810459155, 0.22828789196990626, 0.30707759990065814], [0.47961997819047536, 0.4835085707191934, 0.4796474021514737, 0.5564656003709644], [0.35470792686310837, 0.3585695182409501, 0.35469048714706225, 0.43152575509256813], [0.35477247478080604, 0.35863407557329025, 0.3547169575583708, 0.4314601106395241], [0.2298883367893012, 0.23347019573774874, 0.22987748420263998, 0.3067065358368225], [0.35469497792588967, 0.35856798819924324, 0.35470640097443756, 0.4315245561298736]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.2297647660813352, 0.11874805206779299, 0.22975742055650222, 0.30659112909826308], [0.10483176338535306, 0.056277468913132817, 0.10483138108571016, 0.18164987326566806], [0.10482415323723129, 0.056279341843356985, 0.10483550964695935, 0.1816452303627665], [0.10482415323723129, 0.10869709652356897, 0.10483550964695935, 0.1816452303627665], [0.10482415323723129, 0.11899187763386827, 0.10483550964695935, 0.1816452303627665], [0.2297647660813352, 0.2436850789411692, 0.10483550964695935, 0.30659112909826308], [0.10482415323723129, 0.18121555270204756, 0.10483550964695935, 0.1816452303627665], [0.10483176338535306, 0.18124783136821762, 0.10483550964695935, 0.18164987326566806], [0.10482415323723129, 0.11866589145044687, 0.10483550964695935, 0.1816452303627665], [0.10483176338535306, 0.18121478768119412, 0.10483138108571016, 0.18164987326566806]])), ([5.746652463213041e-07, 0.5746646716560577, 6.039492650863914e-07, 0.4250809748830883], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[8.403999542453696e-08, 0.08403991138454155, 8.404007946703305e-08, 0.41595992054772923], [5.745142835760329e-07, 0.5745137090617494, 1.51586261124434e-06, 0.42523102602121277], [0.2433386519996088, 0.8394102433357575, 0.2436453652696339, 0.6728489842601446], [5.911883320913307e-07, 0.5911877409029986, 1.2745785466026681e-06, 0.40855721868513023], [2.700732365652246e-07, 0.0797056379095312, 1.7763353032532188e-07, 0.42029391505658587], [5.719319259091706e-07, 0.5719313539772447, 1.0724825020986585e-06, 0.4278138269294062], [5.857209574282318e-07, 0.5857203717072744, 7.356411198462495e-07, 0.41402513202941643], [9.657377728203849e-08, 0.09657368070826122, 9.657387385868941e-08, 0.40342612615594486], [0.24333292921813743, 0.8396536355506881, 0.24389579742444384, 0.6730994168345011], [5.746652463213041e-07, 0.5746646716560577, 6.039492650863914e-07, 0.4250809748830883]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [4.201999771226848e-08, 0.084039911384541546, 4.202003973351653e-08, 0.41595992054772923], [2.8725714178801645e-07, 0.57451370906174937, 7.5793130562217e-07, 0.42721732188889316], [0.1216693259998044, 0.084039911384541546, 8.4040079467033055e-08, 0.41595992054772923], [2.9559416604566533e-07, 0.593495177724914, 6.372892733013341e-07, 0.41595992054772923], [1.350366182826123e-07, 0.57451370906174937, 8.881676516266094e-08, 0.4247487664065797], [2.859659629545853e-07, 0.079705637909531196, 5.362412510493293e-07, 0.42850872234298987], [2.928604787141159e-07, 0.079705637909531196, 3.6782055992312476e-07, 0.42029391505658587], [4.8286888641019246e-08, 0.096573680708261223, 4.8286936929344704e-08, 0.40342612615594486], [0.12166646460906871, 0.079705637909531196, 9.6573873858689409e-08, 0.42029391505658587], [2.8733262316065203e-07, 0.079705637909531196, 3.019746325431957e-07, 0.4271422963198309]]))], (3, 'ACH_solution', 'DTLZ2'): [([0.5967338479587603, 0.09858147972057549, 0.7973410998995], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.5969507824044534, 0.09724723153916398, 0.7975580465266644], [0.5969953743855955, 0.09547868668460226, 0.7976026413609927], [0.5973015688530093, 0.09084085335627973, 0.7979088337435358], [0.5966103990929449, 0.09800606455546367, 0.7972176633351664], [0.597200772933602, 0.09515112675879388, 0.7978080286520685], [0.5968171618602842, 0.09622577161524423, 0.797424424396937], [0.5970927473355049, 0.09662685517934749, 0.7977000052046713], [0.5968645780087087, 0.09559938030891031, 0.7974718203326289], [0.5967716387937235, 0.09722229030250082, 0.7973788989850283], [0.5967338479587603, 0.09858147972057549, 0.7973410998995]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.59695078240445343, 0.097247231539163981, 0.7481586685003573], [0.59695078240445343, 0.097247231539163981, 0.7481809659175214], [0.59730156885300933, 0.095478686684602257, 0.748334062108793], [0.59730156885300933, 0.098006064555463665, 0.7479884769046083], [0.59661039909294489, 0.098006064555463665, 0.7482836595630593], [0.59730156885300933, 0.095151126758793877, 0.7480918574354936], [0.59730156885300933, 0.098006064555463665, 0.7482296478393607], [0.59730156885300933, 0.098006064555463665, 0.7481155554033395], [0.59730156885300933, 0.098006064555463665, 0.7480690947295392], [0.59661039909294489, 0.098006064555463665, 0.7480501951867751]])), ([0.015734930653678748, 0.013897383173970695, 0.999889950746737], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[0.013828946987069156, 0.016909162611708148, 0.9998607967047671], [0.016897034543589918, 0.01620190924214241, 0.9998271777106625], [0.01692486083747808, 0.012586627545754661, 0.9998550042207184], [0.005265070770644929, 0.01693733709500703, 0.999867640396051], [0.0169133987965996, 0.013373293297448855, 0.9998435415641114], [0.016091772280966588, 0.01684480732726626, 0.9997752773298815], [0.009390735852302236, 0.017006474927590554, 0.9999368251552081], [0.0105885218449634, 0.015301623822696096, 0.999987797647161], [0.01698429361184861, 0.0030670387720347467, 0.9999145264785776], [0.015734930653678748, 0.013897383173970695, 0.999889950746737]], [[0.0, 0.0, 0.9829301429354526], [0.006914473493534578, 0.016909162611708148, 0.99986079670476713], [0.016897034543589918, 0.016201909242142411, 0.99982717771066254], [0.00846243041873904, 0.016201909242142411, 0.99986079670476713], [0.0026325353853224646, 0.01693733709500703, 0.99986764039605103], [0.0084566993982998, 0.016201909242142411, 0.99982717771066254], [0.008045886140483294, 0.016844807327266259, 0.99977527732988147], [0.004695367926151118, 0.01693733709500703, 0.99986764039605103], [0.0052942609224817, 0.016844807327266259, 0.99986764039605103], [0.008492146805924305, 0.0015335193860173733, 0.99991452647857759], [0.007867465326839374, 0.017006474927590554, 0.9999368251552081]])), ([0.5804064111699403, 0.580357028589837, 0.5804064110016761], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.5773503856619595, 0.5773503843097959, 0.5773503850729637], [0.5773502673698846, 0.577350290350606, 0.5773502798822152], [0.5826156692499683, 0.5826156684009839, 0.5826149014359305], [0.5801680097235249, 0.5801680085004447, 0.5801279874331386], [0.5810068857445115, 0.5810069271751672, 0.581006922947744], [0.5773502764717939, 0.5773502734120037, 0.5773502758215565], [0.5821097160157181, 0.58210970681676, 0.5821097033732295], [0.5773502966823063, 0.5773502940179644, 0.5773502955895976], [0.5811063103713501, 0.5811062592915925, 0.5811063101645253], [0.5804064111699403, 0.580357028589837, 0.5804064110016761]], [[0.0, 0.0, 0.0], [0.57735038566195951, 0.57735038430979591, 0.28867519253648183], [0.5773502673698846, 0.57735029035060603, 0.2886751399411076], [0.5773502673698846, 0.57735029035060603, 0.29130745071796527], [0.5773502673698846, 0.57735029035060603, 0.2900639937165693], [0.5773502673698846, 0.57735029035060603, 0.290503461473872], [0.5773502673698846, 0.57735027341200373, 0.28867513791077826], [0.5773502673698846, 0.57735027341200373, 0.29105485168661477], [0.5773502673698846, 0.57735027341200373, 0.2886751477947988], [0.5773502673698846, 0.57735027341200373, 0.57735027582155651], [0.5773502673698846, 0.57735027341200373, 0.29020320550083806]])), ([0.766450043585801, 0.636713129402526, 0.10289559830444872], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[0.7655725761034963, 0.6358306311180668, 0.10201361836759525], [0.765571991922961, 0.6358321707134678, 0.10201465481748656], [0.7659029141231432, 0.636161501958547, 0.10234398047131571], [0.7652291276939044, 0.6356534012766881, 0.10183586524729345], [0.7660685074518585, 0.6363260521516707, 0.10250955066860512], [0.7660577464338605, 0.6363163015238086, 0.10106244318706456], [0.7661002542700057, 0.6363588036856402, 0.10027701693160944], [0.76591532960686, 0.6361738824953711, 0.10126755088748857], [0.7662380386677367, 0.6364965901979512, 0.09791468265588123], [0.766450043585801, 0.636713129402526, 0.10289559830444872]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.7145657664612903, 0.63583063111806681, 0.10201361836759525], [0.76557257610349627, 0.63583063111806681, 0.10201361836759525], [0.7147309354711138, 0.63583217071346776, 0.10201465481748656], [0.7143940422564945, 0.63565340127668812, 0.10183586524729345], [0.7148137321354715, 0.63565340127668812, 0.10183586524729345], [0.76522912769390439, 0.63565340127668812, 0.10183586524729345], [0.7148296055445451, 0.63635880368564024, 0.10183586524729345], [0.7147371432129723, 0.63635880368564024, 0.10201465481748656], [0.7148984977434105, 0.63565340127668812, 0.10183586524729345], [0.76623803866773665, 0.63649659019795124, 0.097914682655881227]])), ([0.4825161026183028, 0.48232509821156816, 0.7349738833567132], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.4820759811396723, 0.48204237385813786, 0.7345337612838678], [0.4829208235498333, 0.4829208142563574, 0.7353786054939286], [0.4852336631509128, 0.4852244782023799, 0.7376914440518829], [0.48184762388849944, 0.4818476201728523, 0.7343053999004839], [0.48186811925085576, 0.4818668696822021, 0.7343259020200665], [0.48249197492956897, 0.48246731410736493, 0.7349497527797784], [0.4825757285426203, 0.48264095190086814, 0.7350987327140937], [0.4830369728089958, 0.4820276686112868, 0.7354947531134974], [0.4847408028723165, 0.484738308961341, 0.7371985859231915], [0.4825161026183028, 0.48232509821156816, 0.7349738833567132]], [[0.0, 0.0, 0.2524577834161418], [0.48207598113967232, 0.48204237385813786, 0.4934957723500048], [0.48207598113967232, 0.48204237385813786, 0.73453376128386783], [0.48207598113967232, 0.48204237385813786, 0.49507461373401235], [0.48184762388849944, 0.48184762017285232, 0.49338159165831286], [0.48184762388849944, 0.48184762017285232, 0.49339184271810416], [0.48184762388849944, 0.48184762017285232, 0.49370376809796007], [0.48184762388849944, 0.48184762017285232, 0.73430539990048393], [0.48184762388849944, 0.48184762017285232, 0.4939762682648196], [0.48184762388849944, 0.48184762017285232, 0.49482818466966666], [0.48184762388849944, 0.48184762017285232, 0.4937158333864275]])), ([0.4276425901662693, 0.7841316908403974, 0.45473238731438415], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.42855449894471154, 0.785043548305284, 0.45564506330279797], [0.42869646274693074, 0.7851854842761307, 0.4557870464091039], [0.42557255616885986, 0.7830238823979998, 0.4536253670875261], [0.4281512916676253, 0.7836170658310143, 0.45524188452097386], [0.4262288867605392, 0.7827976826640768, 0.45339917331250756], [0.42628497868698656, 0.7827740789710269, 0.45337300737488], [0.42815919009567044, 0.7846482949491096, 0.4550483505276389], [0.42894866370682755, 0.7854379474842039, 0.45603943474508074], [0.43056692725576673, 0.7871114159188147, 0.45771288668785354], [0.4276425901662693, 0.7841316908403974, 0.45473238731438415]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.42855449894471154, 0.785043548305284, 0.24136782809153123], [0.42855449894471154, 0.785043548305284, 0.2414388196446842], [0.42557255616885986, 0.7830238823979998, 0.2403579799838953], [0.42557255616885986, 0.7830238823979998, 0.24116623870061918], [0.42622888676053922, 0.78279768266407679, 0.24024488309638603], [0.42628497868698656, 0.78277407897102691, 0.24023180012757225], [0.42628497868698656, 0.78277407897102691, 0.2410694717039517], [0.42628497868698656, 0.78277407897102691, 0.24156501381267262], [0.42628497868698656, 0.78277407897102691, 0.24240173978405902], [0.42628497868698656, 0.78277407897102691, 0.24091149009732432]])), ([0.8419705822843523, 0.004047315138623232, 0.5397272779994531], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.8419177087469647, 0.007693638919789375, 0.5396743971541194], [0.8419198541170524, 0.007537065128522955, 0.5396765438291136], [0.8419501676415118, 0.004960987445254405, 0.5397068553542591], [0.8419471208741479, 0.002310560944511279, 0.5397038083957543], [0.8419262150671042, 0.00343153592427987, 0.5396829028060569], [0.8419111845844643, 0.005367355016887179, 0.5396678783832523], [0.841978080751614, 0.008800962886330926, 0.5397347682143976], [0.8419377411016591, 0.003182431866857108, 0.5396944341714568], [0.8419228785626047, 0.005879120349622639, 0.5396795733153332], [0.8419705822843523, 0.004047315138623232, 0.5397272779994531]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.8373678437153858, 0.0038468194598946875, 0.5351245382557515], [0.8373689164004297, 0.0076936389197893749, 0.5351256115932486], [0.8373840731626593, 0.0024804937226272027, 0.5351407673558213], [0.8373825497789773, 0.0076936389197893749, 0.535139243876569], [0.8373720968754554, 0.001715767962139935, 0.5351287910817202], [0.8373645816341355, 0.0026836775084435895, 0.5351212788703179], [0.8373980297177104, 0.004400481443165463, 0.5351547237858906], [0.837377859892733, 0.001591215933428554, 0.5351345567644201], [0.8373704286232058, 0.0029395601748113194, 0.5351271263363584], [0.8373942804840795, 0.002023657569311616, 0.5351509786784183]])), ([0.3383953665995133, 0.8869187791292169, 0.3309143690378071], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.3368651627681044, 0.8853885753993886, 0.32950463002764485], [0.33479137817717125, 0.8833147908024278, 0.3281318568452048], [0.33578274647302364, 0.8843071101618607, 0.32915195437459094], [0.3365983749010468, 0.8851217843010223, 0.32996617787035104], [0.3347846265262561, 0.8833091634687636, 0.3281533738687573], [0.3368424913714865, 0.8854402261432527, 0.3302850713043681], [0.33477959156036985, 0.8833108688149093, 0.32815562367640394], [0.3370696411519958, 0.885593097177703, 0.32892458807804764], [0.3371910269884006, 0.8868895780621261, 0.3317344360866072], [0.3383953665995133, 0.8869187791292169, 0.3309143690378071]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.33686516276810441, 0.88538857539938864, 0.32950463002764485], [0.33479137817717125, 0.88331479080242781, 0.1640659284226024], [0.33479137817717125, 0.88331479080242781, 0.16457597718729547], [0.33479137817717125, 0.88331479080242781, 0.16498308893517552], [0.33479137817717125, 0.88331479080242781, 0.16407668693437866], [0.33478462652625612, 0.88330916346876365, 0.16514253565218406], [0.33479137817717125, 0.88331479080242781, 0.16407781183820197], [0.33479137817717125, 0.88331479080242781, 0.16446229403902382], [0.33478462652625612, 0.88331479080242781, 0.1658672180433036], [0.33479137817717125, 0.88331479080242781, 0.16545718451890354]])), ([0.15914002559069873, 0.15296719419892044, 0.9768517141232249], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[0.15890452755863163, 0.15792267194627393, 0.9766162146852237], [0.15599995825551974, 0.158165983506968, 0.975877670544446], [0.15885808654981598, 0.1570804325836528, 0.9765697749389938], [0.1578536831051743, 0.15798578699506596, 0.9756974760912583], [0.15676913883393026, 0.1596813677644111, 0.9773930569860391], [0.1585378192994061, 0.15522588150041214, 0.9762495080064112], [0.1585593426422577, 0.15785403426738445, 0.9762710266627683], [0.15875459640320264, 0.15568915587032364, 0.9764662850169794], [0.15846688721547106, 0.15643178635902538, 0.9761785761761236], [0.15914002559069873, 0.15296719419892044, 0.9768517141232249]], [[0.0, 0.0, 0.8177116892877433], [0.07945226377931582, 0.15792267194627393, 0.97661621468522375], [0.07799997912775987, 0.15816598350696801, 0.97587767054444596], [0.15599995825551974, 0.15816598350696801, 0.97587767054444596], [0.07892684155258715, 0.15798578699506596, 0.97587767054444596], [0.07838456941696513, 0.15816598350696801, 0.97587767054444596], [0.07926890964970305, 0.15798578699506596, 0.97587767054444596], [0.07927967132112886, 0.15798578699506596, 0.97569747609125834], [0.07937729820160132, 0.15798578699506596, 0.97587767054444596], [0.15785368310517431, 0.15798578699506596, 0.97569747609125834], [0.07957001279534937, 0.15798578699506596, 0.97587767054444596]])), ([0.09513243222714331, 0.7750592818376308, 0.6250462312321434], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[0.09136512753474897, 0.7755098672786175, 0.6254968124957921], [0.09620714953539099, 0.7754943257596363, 0.625481276572633], [0.09498937003943514, 0.7762662703975023, 0.626253223747061], [0.0964188187581736, 0.775460478755707, 0.6254427297937535], [0.09581275756435911, 0.7748544174549116, 0.6248401592036751], [0.09609090229520019, 0.7753561138403527, 0.6253430608021832], [0.09181244206156751, 0.7756162340172815, 0.6256031848153263], [0.0961249162949141, 0.775416705920197, 0.625403585991912], [0.0963671828341473, 0.7754052975973992, 0.6253957655039691], [0.09513243222714331, 0.7750592818376308, 0.6250462312321434]], [[0.0, 0.6790416599958826, 0.5290286140740371], [0.045682563767374486, 0.77550986727861748, 0.62549681249579214], [0.048103574767695494, 0.77550986727861748, 0.62549681249579214], [0.04749468501971757, 0.77549432575963628, 0.62548127657263297], [0.0482094093790868, 0.77546047875570701, 0.62549681249579214], [0.04790637878217956, 0.77550986727861748, 0.62549681249579214], [0.048045451147600095, 0.77485441745491157, 0.62484015920367508], [0.095812757564359113, 0.77485441745491157, 0.62484015920367508], [0.04806245814745705, 0.77485441745491157, 0.62484015920367508], [0.04818359141707365, 0.77485441745491157, 0.62549681249579214], [0.047566216113571654, 0.77485441745491157, 0.62549681249579214]]))], (2, 'ACH_solution', 'DTLZ1'): [([0.7927795656854529, 0.20696746650561396], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[1.0426527498304075, 0.45684117186612955], [0.4999995003502528, 5.00000000364631e-07], [0.7927796246532672, 0.20696752611619115], [0.7927827563175558, 0.20697065666007688], [0.49999950010151867, 5.000000001158966e-07], [0.7927794734106733, 0.206967389756532], [1.2925314412892777, 0.7067193437157575], [0.4999994198537613, 5.801576074125699e-07], [0.7927808563758952, 0.20696877175934242], [0.7927795656854529, 0.20696746650561396]], [[0.5858121009812931, 0.0], [0.8142324254058503, 0.45684117186612955], [0.542905800665773, 2.500000001823155e-07], [0.49999950035025281, 5.0000000036463104e-07], [0.6892974286494244, 0.20696752611619115], [0.5429058005414059, 2.500000000579483e-07], [0.6892957871959833, 0.20696738975653201], [0.9391717711352854, 0.45684117186612955], [0.5429057604175271, 2.9007880370628494e-07], [0.49999941985376128, 5.0000000011589664e-07], [0.689295833333373, 0.20696738975653201]])), ([1.1614360623384163, 0.3380579325992228], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.911564705888031, 0.0881865725868494], [0.911565054517237, 0.08818691971034323], [0.49999950005488936, 5.000000000692673e-07], [1.9110561149819476, 1.087678009399671], [0.49999950001980636, 5.000000000341842e-07], [0.4999995001702192, 5.000000001845973e-07], [0.49999929622769007, 7.039340472249059e-07], [0.911563122608058, 0.08818499242318195], [0.9115630605997547, 0.08818492832222895], [1.1614360623384163, 0.3380579325992228]], [[0.8233781347997915, 0.0], [0.91156470588803096, 0.0440932862934247], [0.91156470588803096, 0.044093459855171616], [0.49999950005488936, 2.5000000003463363e-07], [0.91156505451723702, 0.5438390046998355], [0.49999950001980636, 2.500000000170921e-07], [0.49999950001980636, 2.5000000009229863e-07], [0.49999950001980636, 3.5196702361245294e-07], [0.49999929622769007, 0.044092496211590976], [0.49999929622769007, 0.044092464161114474], [0.91156505451723702, 0.1690289662996114]])), ([0.7497468243730719, 0.7497468262078019], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[0.49987340780577394, 0.4998734173521023], [0.7497468243404278, 0.7497468259334915], [0.25000000412193124, 0.2500000076385725], [0.7497468175501079, 0.7497468326394788], [0.4998734383005445, 0.49987344270146106], [0.7497468237633429, 0.7497468301975827], [0.2501058119989988, 0.25010581374037816], [0.4998734271602389, 0.49987342724513073], [0.4998734105005212, 0.4998734146120135], [0.7497468243730719, 0.7497468262078019]], [[0.0, 0.0], [0.49987340780577394, 0.24993670867605114], [0.49987340780577394, 0.37487341296674576], [0.25000000412193124, 0.12500000381928625], [0.25000000412193124, 0.3748734163197394], [0.25000000412193124, 0.24993672135073053], [0.25000000412193124, 0.37487341509879135], [0.25000000412193124, 0.2500000076385725], [0.25000000412193124, 0.24993671362256537], [0.25000000412193124, 0.24993670730600676], [0.25010581199899878, 0.37487341310390093]])), ([0.7693460511033303, 0.23040078904944014], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[1.0192271587508093, 0.480281906462213], [0.769346051228444, 0.2304007940889063], [0.7693460456996728, 0.23040078192542987], [0.7693460396566054, 0.23040078597511407], [0.7693488312689516, 0.23040356708749402], [1.518966275021838, 0.9800210302740694], [1.2690929878416564, 0.7301477608441364], [0.7693460441201034, 0.23040079341320685], [2.018713412781966, 1.4797681635876723], [0.7693460511033303, 0.23040078904944014]], [[0.5389452641563056, 0.0], [0.7790862114535575, 0.48028190646221303], [0.6541456576923748, 0.23040079408890629], [0.6541456549279892, 0.23040078192542987], [0.6541456519064555, 0.23040078192542987], [0.6541470477126285, 0.23040078192542987], [1.0, 0.48028190646221303], [0.9040191259989809, 0.48028190646221303], [0.6541456541382045, 0.23040078192542987], [1.0, 0.48028190646221303], [0.6541456576298179, 0.23040078192542987]])), ([0.7497468239146101, 0.7497468263407462], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.49987340866545416, 0.49987341902111043], [0.4998734270694593, 0.49987342960644965], [0.4998734083471704, 0.4998734175617051], [0.7497468294284941, 0.7497468317671332], [0.49987341930953366, 0.4998734227083671], [0.24999999937287315, 0.25000000212859247], [0.9996202491884844, 0.9996202711694628], [0.24999999886244437, 0.2500000034259917], [0.24999999939959774, 0.2500000014864935], [0.7497468239146101, 0.7497468263407462]], [[0.0, 0.0], [0.24993670433272708, 0.49987341902111043], [0.24993671353472965, 0.49987341902111043], [0.49987340834717042, 0.49987341756170511], [0.37487341471424707, 0.49987341756170511], [0.24993670965476683, 0.49987341756170511], [0.12499999968643657, 0.25000000212859247], [0.4998101245942422, 0.49987341756170511], [0.12499999943122218, 0.25000000212859247], [0.12499999969979887, 0.25000000148649348], [0.37487341195730506, 0.25000000148649348]])), ([0.4998734080473232, 0.49987341719088874], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.7497468178242814, 0.7497468324458187], [0.4998734110814571, 0.4998734142514348], [0.4998734073449125, 0.49987341777992356], [0.4998734080653328, 0.4998734170368102], [0.24999999903462303, 0.2500000024005845], [0.7497468321752816, 0.7497468420583608], [0.7497468257916245, 0.749746827666059], [0.7497468183472564, 0.7497468335074429], [0.7497468246226873, 0.7497468294606737], [0.4998734080473232, 0.49987341719088874]], [[0.0, 0.0], [0.74974681782428143, 0.3748734162229094], [0.49987341108145711, 0.2499367071257174], [0.49987340734491248, 0.24993670888996178], [0.49987340734491248, 0.49987341425143478], [0.24999999903462303, 0.12500000120029225], [0.24999999903462303, 0.3748734210291804], [0.24999999903462303, 0.3748734138330295], [0.24999999903462303, 0.37487341675372143], [0.24999999903462303, 0.37487341473033686], [0.24999999903462303, 0.24993670859544437]])), ([1.1540411934317738, 0.8451992842582153], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.904167775598474, 0.5953258745896473], [0.904167777347719, 0.5953258741065894], [0.904167778429011, 0.5953258754151192], [0.6542944318642682, 0.34545251711123093], [0.404420974038144, 0.0955790611787077], [1.1540411909124166, 0.845199284437711], [1.1540411857376682, 0.8451992895808403], [1.1540411934317738, 0.8451992842582153]], [[0.8521054652075405, 0.5432635491282871], [0.90416777559847405, 0.5692947118589672], [0.90416777559847405, 0.5692947116174383], [0.90416777559847405, 0.5692947122717031], [0.65429443186426817, 0.444358033119759], [0.65429443186426817, 0.3194213051534974], [0.90416777559847405, 0.694231416782999], [0.40442097403814398, 0.095579061178707694]])), ([0.2788827346560621, 0.22111726631045364], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[1.5282498013675252, 1.4704843353805568], [0.2788827342492024, 0.22111726729489578], [0.528756141826603, 0.47099068327163207], [0.528756142070335, 0.47099068314807063], [0.7786295525673455, 0.7208640976678493], [0.7786295526536539, 0.7208640990900675], [0.27888273419241816, 0.22111726718488015], [0.5287561856027966, 0.4709907194761717], [0.5287561420493431, 0.47099068314068526], [0.2788827346560621, 0.22111726631045364]], [[0.2295261844565868, 0.17176071551877525], [1.0, 0.821122525449666], [0.2788827342492024, 0.19643899140683552], [0.2788827342492024, 0.32137569939520366], [0.2788827342492024, 0.3213756993334229], [0.52875614182660302, 0.4463124065933123], [0.52875614182660302, 0.4463124073044214], [0.27888273419241816, 0.19643899135182769], [0.27888273419241816, 0.3213757174974735], [0.27888273419241816, 0.32137569932973026], [0.27888273419241816, 0.19643899091461445]])), ([0.5049431398152788, 1.4942980474287704], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[0.25511015942235804, 1.2444650656046914], [0.2550699974527107, 1.2444249149492108], [0.005195963772157568, 0.99455086358849], [6.448601424642522e-07, 0.49999935527885747], [5.00000000231104e-07, 0.4999995002311038], [0.005196587979669269, 0.9945514991258645], [5.495939747247808e-07, 0.49999945055030987], [0.005196704493260329, 0.9945514623093225], [0.005195959322170766, 0.99455086746085], [0.5049431398152788, 1.4942980474287704]], [[0.0, 0.9893548997434237], [0.12755507971117902, 1.0], [0.12753499872635535, 1.0], [0.002597981886078784, 0.9919528816659569], [3.224300712321261e-07, 0.49999935527885747], [2.50000000115552e-07, 0.49999950023110379], [0.0025982939898346346, 0.9919531994346441], [2.747969873623904e-07, 0.49999935527885747], [0.0025983522466301646, 0.9919531810263731], [0.002597979661085383, 0.9919528836021368], [6.448601424642522e-07, 0.49999935527885747]])), ([0.6697961537808642, 1.3294443219744947], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.6697961499592228, 1.3294443259240782], [0.17004989230018563, 0.8296980625160132], [5.000000005158096e-07, 0.4999995005158091], [0.4199227444765613, 1.0795709132930509], [0.17004941574902224, 0.829697581205209], [0.6698269788358009, 1.3294751441377772], [0.6697961537808642, 1.3294443219744947]], [[0.21536704667589168, 0.875015209332337], [0.66979614995922276, 1.0], [0.17004989230018563, 0.8523566359241751], [0.17004989230018563, 0.6875073549240731], [0.4199227444765613, 0.977293061312694], [0.17004989230018563, 0.8523563952687729], [0.4199227444765613, 1.0]]))], (2, 'ACH_solution', 'DTLZ2'): [([0.9383357862855604, 0.3525236755475735], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[0.9383924119846541, 0.3525803099335024], [0.9385175863488967, 0.35270547378026107], [0.9390347417461865, 0.3532226292956424], [0.9364944741193366, 0.3506823617398285], [0.9385102432688399, 0.3526981281063552], [0.939320673410923, 0.35350856232925315], [0.938064718375643, 0.3522526069037894], [0.9381579137362642, 0.35234578086169], [0.9381050034461696, 0.3522928892887689], [0.9383357862855604, 0.3525236755475735]], [[0.5858121009812931, 0.0], [0.7621022564829736, 0.35258030993350242], [0.762164843665095, 0.35258030993350242], [0.7624234213637398, 0.35258030993350242], [0.7611532875503149, 0.35068236173982847], [0.7621611721250665, 0.35068236173982847], [0.762566387196108, 0.35068236173982847], [0.761938409678468, 0.35068236173982847], [0.7619850073587786, 0.35068236173982847], [0.7619585522137313, 0.35068236173982847], [0.7620739436334267, 0.35068236173982847]])), ([0.9875088201459925, 0.1641306707884412], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.9876749921154016, 0.1642968392832003], [0.986998747995888, 0.16362059966065973], [0.9878363452026212, 0.16445819757240074], [0.9881253816030792, 0.164747232341452], [0.9871848969589032, 0.16380676151214316], [0.9878290315411423, 0.1644508941593693], [0.9873005439011433, 0.16392239308136633], [0.9872839438137156, 0.16390579130834887], [0.9875516647709314, 0.16417351498760585], [0.9875088201459925, 0.1641306707884412]], [[0.8233781347997915, 0.0], [0.98767499211540155, 0.08214841964160015], [0.98699874799588805, 0.08181029983032986], [0.98699874799588805, 0.08222909878620037], [0.98699874799588805, 0.082373616170726], [0.98699874799588805, 0.08190338075607158], [0.98699874799588805, 0.08222544707968465], [0.98699874799588805, 0.08196119654068316], [0.98699874799588805, 0.08195289565417443], [0.98699874799588805, 0.08208675749380293], [0.98699874799588805, 0.0820653353942206]])), ([0.7071067853693606, 0.7071067770038211], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[0.7096345958871271, 0.7096345944612352], [0.7108563686258925, 0.7108563654346907], [0.7071067873087943, 0.7071067758347354], [0.7105240715188483, 0.7105240689312433], [0.7071067825083387, 0.7071067798916844], [0.7110442861497231, 0.711044281747082], [0.7122866101764975, 0.7122866053686716], [0.7109491947649036, 0.7109491895731641], [0.7093930023822337, 0.7093929948214952], [0.7071067853693606, 0.7071067770038211]], [[0.0, 0.0], [0.70963459588712707, 0.3548172972306176], [0.70963459588712707, 0.35542818271734533], [0.70710678730879428, 0.3535533879173677], [0.70710678730879428, 0.35526203446562166], [0.70710678250833869, 0.3535533899458422], [0.70710678250833869, 0.355522140873541], [0.70710678250833869, 0.3561433026843358], [0.70710678250833869, 0.35547459478658205], [0.70710678250833869, 0.3546964974107476], [0.70710678250833869, 0.35355338850191054]])), ([0.9254404005718032, 0.3864951255248149], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[0.9257746481516929, 0.38682937046408655], [0.9252434903773262, 0.3862982133766733], [0.924532949210848, 0.38558768355704914], [0.9233481580265666, 0.3844028915145292], [0.9252266836497045, 0.38628140716740667], [0.925180326078843, 0.38623505270494213], [0.9239946630185365, 0.385049384581237], [0.9239889752118522, 0.3850436985614115], [0.92435505880916, 0.38540978359732153], [0.9254404005718032, 0.3864951255248149]], [[0.5389452641563056, 0.0], [0.7323599561539993, 0.38682937046408655], [0.7320943772668159, 0.38629821337667331], [0.7317391066835768, 0.38558768355704914], [0.731146711091436, 0.38440289151452922], [0.732085973903005, 0.38440289151452922], [0.7320627951175743, 0.38440289151452922], [0.7314699635874211, 0.38440289151452922], [0.731467119684079, 0.38440289151452922], [0.7316501614827329, 0.38440289151452922], [0.7321928323640544, 0.38440289151452922]])), ([0.7132398657524427, 0.713239856688859], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.707106786738076, 0.7071067756350312], [0.7125182913832613, 0.7125182877239793], [0.7103392748422394, 0.7103392727825723], [0.7071067823824693, 0.707106780755426], [0.7071067858077745, 0.7071067829144291], [0.7100004485184916, 0.710000444284281], [0.7124485464560523, 0.7124485383202545], [0.7123432383641385, 0.7123432370579564], [0.7102706289956854, 0.7102706180191369], [0.7132398657524427, 0.713239856688859]], [[0.0, 0.0], [0.353553393369038, 0.70710677563503122], [0.35625914569163064, 0.70710677563503122], [0.3551696374211197, 0.70710677563503122], [0.35355339119123463, 0.70710677563503122], [0.35355339290388726, 0.70710677563503122], [0.70710678238246927, 0.70710677563503122], [0.35622427322802613, 0.70710677563503122], [0.35617161918206924, 0.70710677563503122], [0.3551353144978427, 0.70710677563503122], [0.35661993287622135, 0.70710677563503122]])), ([0.7110996266303049, 0.7110996233439526], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.7086233560237564, 0.7086233502706532], [0.7071067862941667, 0.7071067760866431], [0.7096317112384541, 0.709631708138423], [0.7103467135101726, 0.710346711214391], [0.7127966273029522, 0.712796622798969], [0.7096022633202922, 0.7096022598270422], [0.7105553249236924, 0.7105553203257312], [0.7108200288419225, 0.7108200280851977], [0.7130121986359333, 0.7130121954850226], [0.7110996266303049, 0.7110996233439526]], [[0.0, 0.0], [0.70862335602375637, 0.70862335027065315], [0.70710678629416668, 0.35355338804332154], [0.70710678629416668, 0.3548158540692115], [0.70710678629416668, 0.70710677608664307], [0.70710678629416668, 0.3563983113994845], [0.70710678629416668, 0.3548011299135211], [0.70710678629416668, 0.3552776601628656], [0.70710678629416668, 0.70710677608664307], [0.70710678629416668, 0.3565060977425113], [0.70710678629416668, 0.3555498116719763]])), ([0.8444990445781453, 0.5356571180715234], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.8445070395476596, 0.5356651102269546], [0.844513214129225, 0.5356712852172175], [0.8444900552854867, 0.5356481259427941], [0.8445089021164046, 0.5356669735033401], [0.8445105046047993, 0.5356685752660313], [0.8444602139008881, 0.5356182860457642], [0.8444947226003034, 0.53565280266213], [0.8445002884493437, 0.5356583591096672], [0.8444913888006771, 0.5356494594526446], [0.8444990445781453, 0.5356571180715234]], [[0.8521054652075405, 0.5432635491282871], [0.8483062523776, 0.5394643296776209], [0.8483093396683827, 0.5394674171727523], [0.8482977602465136, 0.5394558375355406], [0.8483071836619726, 0.5394652613158136], [0.8483079849061699, 0.5394660621971592], [0.8482828395542144, 0.5394409175870256], [0.848300093903922, 0.5394581758952086], [0.8483028768284421, 0.5394609541189772], [0.8482984270041087, 0.5394565042904659], [0.8483022548928429, 0.5394603335999053]])), ([0.7353993961765674, 0.6776339213154676], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[0.7353993939386434, 0.6776339214128072], [0.7381569275981844, 0.680391456386112], [0.7379979874902087, 0.6802325149501257], [0.7365771364145846, 0.6788116602833946], [0.7402540647901241, 0.6824885940371943], [0.7373491799275231, 0.6795837067960689], [0.7353993980788678, 0.6776339269118865], [0.7353993986091097, 0.6776339187131559], [0.7353993976056752, 0.677633917546669], [0.7353993961765674, 0.6776339213154676]], [[0.2295261844565868, 0.17176071551877525], [0.7353993939386434, 0.42469731846579123], [0.7353993939386434, 0.4260760859524436], [0.7353993939386434, 0.4259966152344505], [0.7353993939386434, 0.4252861879010849], [0.7353993939386434, 0.4271246547779848], [0.7353993939386434, 0.4256722111574221], [0.7353993939386434, 0.4246973212153309], [0.7353993939386434, 0.4246973171159656], [0.7353993939386434, 0.42469731653272214], [0.7353993939386434, 0.42469731841712144]])), ([0.01060854072572496, 0.9999659719655557], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[0.010587089261399093, 0.9999440119587628], [0.009061693942488172, 0.9999817262011313], [0.009730430963294002, 0.9999725770857045], [0.010116054079876218, 0.9999491313763613], [0.008867850051487694, 0.9999809487153487], [0.010274384961944638, 0.9999477007873603], [0.010588453935624105, 0.9999495400492103], [0.01062334390566723, 0.9999782444468195], [0.01062374003477335, 0.9999808717164602], [0.01060854072572496, 0.9999659719655557]], [[0.0, 0.9893548997434237], [0.005293544630699546, 0.99994401195876281], [0.004530846971244086, 0.99998172620113135], [0.004865215481647001, 0.99998172620113135], [0.005058027039938109, 0.99998172620113135], [0.004433925025743847, 0.99998094871534871], [0.005137192480972319, 0.99998094871534871], [0.010587089261399093, 0.99994401195876281], [0.005311671952833615, 0.99998094871534871], [0.005311870017386675, 0.99998094871534871], [0.00530427036286248, 0.99998094871534871]])), ([0.2960326086811327, 0.9556807655440175], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.29603218240290685, 0.9556803424976889], [0.2961606036812752, 0.9558087619958912], [0.29595031001323807, 0.9555984642793807], [0.2961458693887118, 0.9557940320198354], [0.29615638489865515, 0.9558045443653991], [0.2962035007352513, 0.9558516589522863], [0.29614301319176933, 0.9557911749725679], [0.2958854468929679, 0.9555336050761889], [0.29583712082821806, 0.9554852814371424], [0.2960326086811327, 0.9556807655440175]], [[0.21536704667589168, 0.875015209332337], [0.29603218240290685, 0.9153477759150129], [0.29603218240290685, 0.9154119856641141], [0.29595031001323807, 0.9153068368058588], [0.29595031001323807, 0.95559846427938067], [0.29595031001323807, 0.9154098768488681], [0.29595031001323807, 0.9154334341423116], [0.29595031001323807, 0.9154031921524525], [0.29588544689296792, 0.9152744072042629], [0.29583712082821806, 0.9152502453847398], [0.29583712082821806, 0.9153479874381772]]))], (3, 'ACH_solution', 'DTLZ4'): [([0.6001764846020227, 1.2271835246434515e-20, 0.8007836231417029], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.5964905541272276, 0.09782721352031037, 0.7970978192352985], [0.6001688902092733, 1.1160200417704388e-35, 0.8007757158128291], [0.6002826647997668, 1.6712330845315135e-22, 0.8008896167090904], [0.5999737816939985, 1.0067488217999583e-13, 0.8005809615568353], [0.6001350782559899, 7.881823736133413e-06, 0.800742030071384], [0.6003130015286083, 5.2821447301152e-29, 0.8009202680280779], [0.6003781925675458, 1.8638046713457318e-61, 0.8009849252543684], [0.5967468527791251, 0.09827208487349648, 0.7973535624762286], [0.6000012766868713, 3.4522721409481264e-55, 0.8006082118658737], [0.6001764846020227, 1.2271835246434515e-20, 0.8007836231417029]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.59649055412722762, 0.097827213520310374, 0.7479285548546744], [0.59649055412722762, 5.580100208852194e-36, 0.7497675031434397], [0.59649055412722762, 8.356165422657567e-23, 0.7498244535915704], [0.59649055412722762, 5.033744108999791e-14, 0.7496701260154428], [0.59997378169399851, 3.9409118680667065e-06, 0.80058096155683534], [0.59997378169399851, 2.6410723650576e-29, 0.80058096155683534], [0.59997378169399851, 9.319023356728659e-62, 0.7498721078642092], [0.59997378169399851, 1.0067488217999583e-13, 0.7480564264751394], [0.59649055412722762, 1.7261360704740632e-55, 0.7496837511699619], [0.59649055412722762, 6.135917623217258e-21, 0.7497714568078766]])), ([0.01697986631157926, 5.312298290518463e-13, 0.9999131247469203], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[0.01671399289302551, 3.1117707182399168e-49, 1.0000075231041328], [0.016439458983796274, 2.406247328515773e-24, 0.9999021894612582], [0.017063879586527192, 4.557437456874743e-52, 0.9999975521106226], [0.01618856878747736, 4.5932525105953874e-10, 0.9999671333957125], [0.0158790692502925, 5.461113469493899e-55, 0.9999382688091994], [0.016982224376373752, 6.664726394757193e-55, 0.9999145795210205], [0.016436741735970373, 1.1268857820073413e-103, 0.9999043747171948], [0.0029817168999502733, 1.188783100119052e-06, 1.0000767575260063], [0.016722620578425853, 1.1044155225677661e-149, 1.0000040207572778], [0.01697986631157926, 5.312298290518463e-13, 0.9999131247469203]], [[0.0, 0.0, 0.9829301429354526], [0.008356996446512756, 1.5558853591199584e-49, 1.0], [0.016439458983796274, 1.2031236642578865e-24, 0.9999021894612582], [0.008531939793263596, 2.2787187284373716e-52, 0.9999021894612582], [0.016439458983796274, 2.2966262552976937e-10, 0.9999021894612582], [0.00793953462514625, 2.7305567347469496e-55, 0.99993826880919945], [0.008491112188186876, 3.3323631973785964e-55, 0.99993826880919945], [0.008218370867985186, 2.4062473285157731e-24, 0.99993826880919945], [0.0014908584499751366, 5.94391550059526e-07, 1.0], [0.008361310289212926, 5.522077612838831e-150, 1.0], [0.00848993315578963, 2.6561491452592317e-13, 0.99993826880919945]])), ([0.5823631866209241, 0.5823628316552252, 0.5823629361434723], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.5807621965361991, 0.5807621958388649, 0.5807617933187055], [0.577350478165764, 0.5773499734570143, 0.5773503559498336], [0.5788721789988672, 0.5788721750516269, 0.5788721564178345], [0.5823668313288809, 0.5823666903598208, 0.5823664344114827], [0.5814224534676885, 0.581524285202587, 0.5815241270555456], [0.5816014044955269, 0.5816010642172353, 0.5816006268737456], [0.5802860154699692, 0.5802860696396989, 0.5771122273733483], [0.5773509484259648, 0.57735060628319, 0.5773509454077915], [0.5796359072549305, 0.5796358460584184, 0.5796357272358136], [0.5823631866209241, 0.5823628316552252, 0.5823629361434723]], [[0.0, 0.0, 0.0], [0.58076219653619909, 0.5807621958388649, 0.29038089665935274], [0.57735047816576401, 0.57734997345701433, 0.2886751779749168], [0.57735047816576401, 0.57734997345701433, 0.28943607820891726], [0.57735047816576401, 0.57734997345701433, 0.57735035594983364], [0.57735047816576401, 0.57734997345701433, 0.2907620635277728], [0.57735047816576401, 0.57734997345701433, 0.2908003134368728], [0.58028601546996916, 0.58028606963969886, 0.28855611368667416], [0.58028601546996916, 0.58028606963969886, 0.28867547270389576], [0.57735047816576401, 0.58028606963969886, 0.2898178636179068], [0.58028601546996916, 0.58028606963969886, 0.29118146807173617]])), ([0.7689957535302876, 0.6392538876609979, 2.777727532916319e-27], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[0.7696419490531962, 0.6399001505437933, 7.11945073501973e-44], [0.7689957490836282, 0.6392539802625198, 6.574619855278782e-44], [0.7673400637885319, 0.6376101750883308, 0.1037931121770458], [0.7697326327266747, 0.6399908939495002, 9.538100433076033e-81], [0.7701287842994751, 0.6403871291943332, 8.862153098702496e-12], [0.7694417196973745, 0.6397000953921945, 6.130508607737481e-91], [0.7700386720954892, 0.6402967610922874, 0.0014420811965722676], [0.769426025043071, 0.6396841018839194, 1.036932217158551e-35], [0.7662530407183316, 0.6365112966916666, 0.1026940063399963], [0.7689957535302876, 0.6392538876609979, 2.777727532916319e-27]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.76964194905319616, 0.63990015054379334, 3.559725367509865e-44], [0.7162773529513564, 0.63925398026251978, 3.287309927639391e-44], [0.7154495103038082, 0.63761017508833084, 0.1037931121770458], [0.7166457947728796, 0.63761017508833084, 0.1037931121770458], [0.7168438705592798, 0.63761017508833084, 0.1037931121770458], [0.7165003382582296, 0.63761017508833084, 3.0652543038687404e-91], [0.7167988144572868, 0.63761017508833084, 0.0007210405982861338], [0.7164924909310777, 0.63925398026251978, 5.184661085792755e-36], [0.76899574908362822, 0.63925398026251978, 6.5746198552787817e-44], [0.716277355174686, 0.63651129669166662, 1.3888637664581594e-27]])), ([0.4843566761195059, 0.4843565491403383, 0.736807474437618], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.48416788864873644, 0.48416996723331607, 0.7366275722599077], [0.48354009769474215, 0.4835400976606346, 0.7359393532931879], [0.48493294855021246, 0.4847349797697134, 0.7373906015345334], [0.4846309021636071, 0.4846308036220274, 0.7370882655646815], [0.4858175631631372, 0.4858172884928096, 0.7382751369895945], [0.4843566761195059, 0.4843565491403383, 0.736807474437618]], [[0.0, 0.0, 0.2524577834161418], [0.48416788864873644, 0.48416996723331607, 0.49454267783802475], [0.48354009769474215, 0.4835400976606346, 0.49419856835466486], [0.48354009769474215, 0.4835400976606346, 0.4949241924753376], [0.48354009769474215, 0.4835400976606346, 0.49477302449041166], [0.48354009769474215, 0.4835400976606346, 0.73593935329318794]])), ([0.4281369766752247, 0.7846259679848134, 0.4537001496296078], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.42894967187035393, 0.7854385333023288, 0.4560389648830087], [0.4279809233588685, 0.7844698593217543, 0.4550661349603716], [0.42944036583626827, 0.7859293391418154, 0.4553020191318486], [0.4275746381909331, 0.784063740322661, 0.4541056320995318], [0.42895953438453227, 0.7854484610704264, 0.45556322080650263], [0.4298978431724102, 0.78638681793452, 0.456957287996667], [0.429092419556846, 0.7855813197516546, 0.456180778409406], [0.4287794335147252, 0.7852683514402224, 0.45586193390943824], [0.4282158099660483, 0.7847047648490817, 0.45530559472632043], [0.4281369766752247, 0.7846259679848134, 0.4537001496296078]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.42894967187035393, 0.78543853330232882, 0.2415647788816366], [0.4279809233588685, 0.78446985932175428, 0.24107836392031806], [0.4279809233588685, 0.78446985932175428, 0.24119630600605654], [0.42757463819093311, 0.78406374032266102, 0.24059811248989815], [0.42757463819093311, 0.78406374032266102, 0.24132690684338357], [0.42757463819093311, 0.78406374032266102, 0.24202394043846576], [0.42757463819093311, 0.78406374032266102, 0.24163568564483526], [0.42757463819093311, 0.78406374032266102, 0.24147626339485137], [0.42757463819093311, 0.78406374032266102, 0.24119809380329246], [0.42757463819093311, 0.78462596798481343, 0.24039537125493615]])), ([0.841970446658453, 3.742739345152077e-18, 0.5397267416329765], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.8419402702648081, 1.868375264573677e-40, 0.539696625375905], [0.8419774291806548, 3.169349323036806e-26, 0.5397336875531921], [0.8419556881094636, 2.7172751262090907e-35, 0.5397121131646074], [0.8419725217869968, 4.289268552112291e-29, 0.5397288815094381], [0.8419692439196914, 7.584387712442422e-15, 0.5397259033893582], [0.841983929740953, 2.3078013908389335e-25, 0.5397406291050416], [0.8419395647093015, 8.066424096500504e-38, 0.5396957881615552], [0.8418910433054735, 1.9678933645370818e-31, 0.5396475606020482], [0.8419977192738416, 1.6755692370285486e-06, 0.5397541799178394], [0.841970446658453, 3.742739345152077e-18, 0.5397267416329765]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.8373791244743074, 9.341876322868386e-41, 0.5351356523666443], [0.8373977039322308, 1.584674661518403e-26, 0.5351541834552878], [0.8373868333966352, 1.3586375631045453e-35, 0.5351433962609955], [0.8373952502354018, 2.1446342760561456e-29, 0.5351517804334108], [0.837393611301749, 3.792193856221211e-15, 0.5351502913733709], [0.8374009542123799, 1.1539006954194668e-25, 0.5351576542312126], [0.8373787716965542, 4.033212048250252e-38, 0.5351352337594694], [0.8373545109946401, 1.9678933645370818e-31, 0.5351111199797158], [0.8374078489788241, 8.377846185142743e-07, 0.5351644296376115], [0.8373942126711299, 1.8713696725760386e-18, 0.53515071049518]])), ([0.33630411535946186, 0.8848271601120774, 0.3295120295113347], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.33585104994545467, 0.8843743759589218, 0.3285884617103505], [0.3356860677551407, 0.8842094446405662, 0.3290452961586179], [0.3383365728338807, 0.8868599346176824, 0.3293287723674397], [0.33756214282921443, 0.8860853655212746, 0.33092867945450927], [0.33676906552775276, 0.8852923839881974, 0.3283113963645275], [0.33867641604119236, 0.8871996268282466, 0.3276314852235209], [0.33691683200620653, 0.8854400138943818, 0.32916276868194183], [0.3354699362674374, 0.8839933493703866, 0.3285243569867428], [0.3367517044547754, 0.8852750407173605, 0.330119836302094], [0.33630411535946186, 0.8848271601120774, 0.3295120295113347]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.33585104994545467, 0.88437437595892177, 0.16429423085517525], [0.33585104994545467, 0.88437437595892177, 0.16452264807930894], [0.3356860677551407, 0.88420944464056617, 0.16466438618371984], [0.3356860677551407, 0.88437437595892177, 0.16546433972725463], [0.3356860677551407, 0.88529238398819743, 0.16415569818226375], [0.3356860677551407, 0.88420944464056617, 0.16381574261176046], [0.3356860677551407, 0.88420944464056617, 0.16458138434097092], [0.33546993626743737, 0.8871996268282466, 0.1642621784933714], [0.33867641604119236, 0.8871996268282466, 0.165059918151047], [0.33867641604119236, 0.8871996268282466, 0.16475601475566734]])), ([0.1690828670692777, 1.1098024462506754e-70, 0.9867946364873275], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[0.16907427633933342, 1.2821944661776656e-26, 0.9867861165995567], [0.16839201492709294, 5.9286681864943e-32, 0.9861034832779693], [0.16888735725697018, 3.434133230605304e-52, 0.986599046559163], [0.16806448924525647, 7.555558477022022e-11, 0.9857760781821715], [0.16879872025896184, 2.3783628498620203e-63, 0.9865101739488084], [0.16991825034690078, 1.248750239687487e-43, 0.9876297827255603], [0.16912119667758377, 1.0054986514899837e-56, 0.9868328851661631], [0.15721148400879797, 0.15858221991035543, 0.9762937281816962], [0.16898991819006498, 6.572399655247683e-45, 0.9867013728647889], [0.1690828670692777, 1.1098024462506754e-70, 0.9867946364873275]], [[0.0, 0.0, 0.8177116892877433], [0.16907427633933342, 6.410972330888328e-27, 0.98678611659955673], [0.08419600746354647, 2.96433409324715e-32, 0.98610348327796926], [0.08444367862848509, 1.717066615302652e-52, 0.98610348327796926], [0.08403224462262823, 3.777779238511011e-11, 0.9857760781821715], [0.08439936012948092, 1.1891814249310102e-63, 0.9857760781821715], [0.08495912517345039, 6.243751198437435e-44, 0.9857760781821715], [0.16806448924525647, 5.027493257449919e-57, 0.9857760781821715], [0.07860574200439899, 0.15858221991035543, 0.97629372818169624], [0.08449495909503249, 3.2861998276238416e-45, 0.9857760781821715], [0.08454143353463885, 5.549012231253377e-71, 0.97629372818169624]])), ([0.09652398437487172, 0.7755655199431082, 0.625427264549011], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[0.09627111260450479, 0.775674279297593, 0.624871780702527], [0.09648952319334808, 0.7755436997536808, 0.6254751305960392], [0.09631765077698771, 0.7753597359509123, 0.6253465420859958], [0.08914467361922114, 0.775726039872334, 0.6257126607668032], [0.09621307475213571, 0.7752153883689279, 0.6252416885022737], [0.09608679390802574, 0.775132146956828, 0.6251190121696072], [0.09641954252887257, 0.7754678193869268, 0.6254547679513066], [0.09628117294489863, 0.775703092783132, 0.6247101403648361], [0.09652708245028008, 0.7755954083942417, 0.6255821774060628], [0.09652398437487172, 0.7755655199431082, 0.625427264549011]], [[0.0, 0.6790416599958826, 0.5290286140740371], [0.04813555630225239, 0.77567427929759303, 0.62487178070252702], [0.04824476159667404, 0.77567427929759303, 0.62487178070252702], [0.04815882538849386, 0.77567427929759303, 0.62487178070252702], [0.04457233680961057, 0.77567427929759303, 0.62571266076680321], [0.048106537376067855, 0.77572603987233402, 0.62571266076680321], [0.04804339695401287, 0.77513214695682797, 0.62511901216960719], [0.048209771264436284, 0.77572603987233402, 0.62571266076680321], [0.048140586472449315, 0.77570309278313199, 0.62571266076680321], [0.096086793908025742, 0.77570309278313199, 0.62511901216960719], [0.04826199218743586, 0.77572603987233402, 0.62571266076680321]]))], (2, 'ACH_solution', 'DTLZ3'): [([3.105046004282511, 2.5192338698749404], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[3.105046004282511, 2.5192338698749404]], [[0.5858121009812931, 0.0]])), ([3.208908598385482, 2.385530453303583], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[2.491947370447163, 1.6685691950169559], [3.208908598385482, 2.385530453303583]], [[0.8233781347997915, 0.0], [1.0, 0.8342845975084779]])), ([2.1206043673942236, 2.120604362476712], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[2.1206043673942236, 2.120604362476712]], [[0.0, 0.0]])), ([2.372885812346776, 1.8339405010211434], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[2.372885812346776, 1.8339405010211434]], [[0.5389452641563056, 0.0]])), ([7.06784544922201, 7.067845424960873], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[7.06784544922201, 7.067845424960873]], [[0.0, 0.0]])), ([3.5341017538721107, 3.5341017125942114], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[1.4138555299107027, 1.4138555082023914], [3.5341017538721107, 3.5341017125942114]], [[0.0, 0.0], [1.0, 0.7069277541011957]])), ([11.46170445733314, 11.152862428190607], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[11.46170445733314, 11.152862428190607]], [[0.8521054652075405, 0.5432635491282871]])), ([4.2696350467852575, 4.2118695564309565], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[4.2696350467852575, 4.2118695564309565]], [[0.2295261844565868, 0.17176071551877525]])), ([3.0046324371584343, 3.993987307516072], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[3.0046324371584343, 3.993987307516072]], [[0.0, 0.9893548997434237]])), ([1.045027180025936, 1.7046753407562854], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[1.045027180025936, 1.7046753407562854]], [[0.21536704667589168, 0.875015209332337]]))], (2, 'ACH_solution', 'DTLZ4'): [([0.936494549857057, 0.35068213461577263], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[0.9382165006779207, 0.3524040824093527], [0.9364948047775989, 0.35068267791120333], [0.9387359983692022, 0.35292364209988764], [0.9364945640504897, 0.35068217395455475], [0.9364945005154461, 0.3506823034933859], [0.9375532004919123, 0.35174076756357286], [1.000000000000001, 1.3456661511672638e-22], [0.9385320065684821, 0.3527195803259141], [0.9364949021305387, 0.3506826985282905], [0.936494549857057, 0.35068213461577263]], [[0.5858121009812931, 0.0], [0.762014300829607, 0.35240408240935273], [0.761153452879446, 0.35068267791120333], [0.7622740496752476, 0.35068267791120333], [0.7611533325158915, 0.35068217395455475], [0.9364945640504897, 0.35068217395455475], [0.7616826507366027, 0.35068217395455475], [0.7929060504906471, 6.728330755836319e-23], [0.7621720537748876, 0.35068217395455475], [0.7611535015559159, 0.35068217395455475], [0.7611533254191751, 0.35068213461577263]])), ([0.9875146802624674, 0.16413649038922634], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.987734747059366, 0.1643565914978669], [0.9872310027915685, 0.16385276657492032], [0.9865909231221276, 0.16321262535522768], [0.9870358903026497, 0.16365761200223858], [0.9874777955330478, 0.16409955554600347], [0.9869036884578726, 0.1635253877868771], [0.9870239932657736, 0.1636457569114797], [0.9869726572832072, 0.16359435742806597], [0.9867894420625699, 0.16341127675140188], [0.9875146802624674, 0.16413649038922634]], [[0.8233781347997915, 0.0], [0.98773474705936604, 0.08217829574893346], [0.9872310027915685, 0.08192638328746016], [0.98659092312212759, 0.08160631267761384], [0.98659092312212759, 0.08182880600111929], [0.98659092312212759, 0.08204977777300174], [0.98659092312212759, 0.08176269389343856], [0.98659092312212759, 0.08182287845573985], [0.98659092312212759, 0.08179717871403298], [0.98659092312212759, 0.08170563837570094], [0.98659092312212759, 0.16321262535522768]])), ([0.711545907455756, 0.711545751455658], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[0.7119578050176029, 0.7119576493336041], [0.7106964938757376, 0.710696483749872], [0.7119913143752673, 0.7119911267027784], [0.7104656608256641, 0.7104654621171509], [0.7071069826241648, 0.707106608558392], [0.7106952099957314, 0.7106949412984771], [0.7099410136169013, 0.7099408534475237], [0.709050365397611, 0.7090498196565301], [0.707107105146209, 0.7071066167191912], [0.711545907455756, 0.711545751455658]], [[0.0, 0.0], [0.71195780501760286, 0.35597882466680203], [0.71069649387573763, 0.355348241874936], [0.71069649387573763, 0.71069648374987204], [0.71046566082566409, 0.35523273105857545], [0.70710698262416483, 0.353553304279196], [0.70710698262416483, 0.35534747064923855], [0.70710698262416483, 0.35497042672376183], [0.70710698262416483, 0.35452490982826507], [0.70710698262416483, 0.3535533083595956], [0.70710698262416483, 0.355772875727829]])), ([0.9245948296665367, 0.3856495170165257], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[0.9253571529661901, 0.38641156802589527], [0.9255662864713644, 0.3866207268076213], [0.9242008016035109, 0.38525524522460497], [0.9262289914669, 0.3872834018648356], [0.924039083756503, 0.3850935753655866], [0.9260067772430156, 0.3870612399034867], [0.9252324047832926, 0.38628698417296026], [0.925372059645059, 0.3864267568633869], [0.9258936316829185, 0.3869480535160691], [0.9245948296665367, 0.3856495170165257]], [[0.5389452641563056, 0.0], [0.7321512085612478, 0.38641156802589527], [0.92535715296619014, 0.38641156802589527], [0.7315730328799083, 0.38525524522460497], [0.7325871278116027, 0.38525524522460497], [0.7314921739564043, 0.38509357536558658], [0.92403908375650301, 0.38509357536558658], [0.7320888344697991, 0.38509357536558658], [0.7321586619006824, 0.38509357536558658], [0.732419447919612, 0.38509357536558658], [0.7317700469114212, 0.38509357536558658]])), ([0.7122408878269827, 0.7122404293631169], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.7106946144294679, 0.7106941955216406], [0.7105826953165533, 0.7105825185601833], [0.7071070603647409, 0.7071065020082445], [0.7116525683726617, 0.7116523951341505], [0.7092309306471265, 0.7092308130614005], [0.7082573800438403, 0.7082571034974222], [0.7144650326081685, 0.7144645727733001], [0.7097236087449523, 0.7097231471867144], [0.7071070607980962, 0.7071065015748886], [0.7122408878269827, 0.7122404293631169]], [[0.0, 0.0], [0.35534730721473395, 0.71069419552164059], [0.35529134765827664, 0.71058251856018328], [0.35355353018237046, 0.70710650200824454], [0.35582628418633083, 0.70710650200824454], [0.35461546532356325, 0.70710650200824454], [0.35412869002192016, 0.70710650200824454], [0.35723251630408426, 0.70710650200824454], [0.35486180437247616, 0.70710650200824454], [0.3535535303990481, 0.70710650157488864], [0.35612044391349135, 0.70710650157488864]])), ([0.7071070578329169, 0.7071065050210673], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.7116636929949667, 0.7116631387946659], [0.7071067830780401, 0.7071067815829688], [0.7071069310814903, 0.7071066333370346], [0.7123238900642355, 0.7123236020369766], [0.7090242496220457, 0.709024045034792], [0.7112583948096645, 0.7112580333480545], [0.7132812879547594, 0.7132809812894255], [0.7099385715469752, 0.7099381613741331], [0.7115994851399462, 0.7115992146001955], [0.7071070578329169, 0.7071065050210673]], [[0.0, 0.0], [0.71166369299496668, 0.71166313879466592], [0.70710678307804009, 0.3535533907914844], [0.70710678307804009, 0.3535533166685173], [0.70710678307804009, 0.3561618010184883], [0.70710678307804009, 0.7071067815829688], [0.70710678307804009, 0.35562901667402724], [0.70710678307804009, 0.35664049064471276], [0.70710678307804009, 0.7071067815829688], [0.70710678307804009, 0.35579960730009774], [0.70710678307804009, 0.35355325251053366]])), ([0.8444790549233188, 0.5356367866956921], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.8444603168742566, 0.5356181453486711], [0.8444602497315631, 0.53561825909974], [0.8445061655052827, 0.5356638927813426], [0.8444980531518846, 0.5356556735365499], [0.8444603214189559, 0.5356181341927994], [0.8444860851890326, 0.5356438665274509], [0.8444947475443867, 0.5356526212090391], [0.8444903427616248, 0.5356482027055802], [0.8444733210795637, 0.535631312849931], [0.8444790549233188, 0.5356367866956921]], [[0.8521054652075405, 0.5432635491282871], [0.8482828910408986, 0.5394408472384791], [0.8482828574695518, 0.5394409041140136], [0.8483058153564116, 0.5394637209548149], [0.8483017591797126, 0.5394596113324185], [0.8482828933132482, 0.5394408416605432], [0.8482957751982865, 0.539453707827869], [0.8483001063759636, 0.5394580851686631], [0.8482979039845826, 0.5394558759169337], [0.8482893931435521, 0.5394474309891091], [0.84450616550528268, 0.5394501679119896]])), ([0.7371511163684322, 0.6793851115301479], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[0.7353996194402801, 0.6776336785738059], [0.7353995464224914, 0.6776337570802725], [0.7401523625932285, 0.6823865554237017], [0.7381071061222434, 0.6803412354992667], [1.0000000000000002, 1.370030890664658e-42], [0.7373894284230244, 0.6796235475209698], [0.7394505288719848, 0.6816845338845694], [0.7389197462875728, 0.6811540649553053], [0.7408376754627785, 0.6830720264186702], [0.7371511163684322, 0.6793851115301479]], [[0.2295261844565868, 0.17176071551877525], [0.73539961944028009, 0.42469719704629055], [0.73539954642249139, 0.4246972362995239], [0.73539954642249139, 0.4270736354712385], [0.73539954642249139, 0.426050975509021], [1.0, 0.08588035775938763], [0.73539954642249139, 0.4256921315198725], [0.73539954642249139, 0.4267226247016723], [1.0, 1.3700308906646581e-42], [0.73539954642249139, 0.4274163709687227], [0.73539954642249139, 0.4255729135244616]])), ([0.010577731203939142, 0.9999954071076802], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[0.010575428967249388, 0.9999965933010846], [0.006773509373403989, 1.0000159619260425], [0.010555312667395574, 0.9999921380216029], [0.005329047811955803, 1.0000595551110334], [0.010559231074327463, 0.999989486908533], [0.0105682022736598, 1.0000206964006018], [0.010571873781496188, 0.999996368805379], [0.01063440659777738, 0.9999895332064392], [0.010617152116261781, 1.0000133727615805], [0.010577731203939142, 0.9999954071076802]], [[0.0, 0.9893548997434237], [0.005287714483624694, 0.99999659330108459], [0.0033867546867019944, 1.0], [0.005277656333697787, 1.0], [0.0026645239059779014, 1.0], [0.0052796155371637316, 1.0], [0.0052841011368299, 1.0], [0.005285936890748094, 1.0], [0.010559231074327463, 0.99998948690853295], [0.0053085760581308905, 1.0], [0.005288865601969571, 1.0]])), ([0.2961481803620168, 0.9557963019220228], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.29596317399066124, 0.9556110931792785], [0.2960126276373887, 0.9556604874577965], [0.2961983563830218, 0.9558463535014441], [0.29565865118190165, 0.9553065571661017], [0.2961677486787834, 0.9558155451795821], [0.29604996812311285, 0.955697844435616], [0.29585149694482865, 0.9554993162278369], [0.2961813956165109, 0.9558294951039947], [0.2961357596275124, 0.955783670794803], [0.2961481803620168, 0.9557963019220228]], [[0.21536704667589168, 0.875015209332337], [0.29596317399066124, 0.9153131512558077], [0.29596317399066124, 0.9153378483950667], [0.29596317399066124, 0.9556110931792785], [0.29565865118190165, 0.9151608832492193], [0.29565865118190165, 0.9154153772559596], [0.29565865118190165, 0.9153565268839765], [0.29565865118190165, 0.915257262780087], [0.29565865118190165, 0.9154223522181659], [0.29565865118190165, 0.91539944006357], [0.29565865118190165, 0.91540575562718]]))], (2, 'rNSGAII_solution', 'DTLZ1'): [([0.50087276512317114, 7.7333039234323655e-09], ([0.5858121009812931, 0.0], [0.41630079584248725, 0.04518209215991853]), ([[0.50041584570278963, 3.7175757747201327e-09], [0.5009442836430209, 1.4803179037012942e-09], [0.50005005115415513, 4.2534414902519555e-09], [0.50259456690839255, 1.2009090959895921e-07], [0.50013185671477545, 3.9353122913751189e-09], [0.5025856179520638, 3.1614199330227588e-07], [0.50326525022788582, 9.7555346511273359e-09], [0.50209624851415346, 1.6726137347272974e-09], [0.50116366608858121, 2.2766488875958088e-06], [0.50087276512317114, 7.7333039234323655e-09]], [[0.5858121009812931, 0.0], [0.54311397334204137, 1.8587878873600664e-09], [0.543378192312157, 7.4015895185064708e-10], [0.54293107606772417, 2.1267207451259777e-09], [0.54420333394484288, 6.0045454799479603e-08], [0.54297197884803428, 1.2009090959895921e-07], [0.54419885946667845, 1.5807099665113794e-07], [0.54453867560458946, 4.8777673255636679e-09], [0.54395417474772323, 8.3630686736364872e-10], [0.54348788353493716, 1.1383244437979044e-06], [0.54334243305223207, 3.8666519617161828e-09]])), ([0.50120373810994068, 4.9069236056602292e-09], ([0.8233781347997915, 0.0], [0.17405614049749618, 0.637269401225692]), ([[0.50180658892512076, 3.0180628093570065e-10], [0.50167598231836197, 1.4503408540717359e-08], [0.50040582682968626, 4.2700920117416191e-08], [0.5036121788204907, 6.9509355765651361e-11], [0.50680526653939673, 5.709690263106851e-10], [0.50062933780643437, 2.926219630934119e-10], [0.50303799370291247, 1.2914319807359196e-10], [0.50275559149703808, 7.8590453348230703e-14], [0.50406040417386966, 5.0312420256275275e-11], [0.50120373810994068, 4.9069236056602292e-09]], [[0.8233781347997915, 0.0], [0.50180658892512076, 1.5090314046785032e-10], [0.50167598231836197, 7.2517042703586795e-09], [0.50040582682968626, 2.1350460058708095e-08], [0.50040582682968626, 3.4754677882825681e-11], [0.50040582682968626, 2.8548451315534255e-10], [0.50040582682968626, 1.4631098154670595e-10], [0.50040582682968626, 6.4571599036795981e-11], [0.50040582682968626, 3.9295226674115352e-14], [0.50040582682968626, 2.5156210128137637e-11], [0.50040582682968626, 2.4534618028301146e-09]])), ([0.48322563100580668, 0.020513339402988665], ([0.0, 0.0], [0.3601022277283289, 0.7184816156875563]), ([[0.2505013419667203, 0.25025260876219529], [0.24980348423616816, 0.25058901949912327], [0.038461561229045461, 0.46421918055715305], [0.25509214321936324, 0.25077411465743327], [0.48904134698141433, 0.013037348185515055], [0.24861598086455433, 0.25271572052141045], [0.2497684140734385, 0.25032254710235302], [0.25114722012767721, 0.25126503979862158], [0.40836631259510331, 0.092405361178666198], [0.48322563100580668, 0.020513339402988665]], [[0.0, 0.0], [0.2505013419667203, 0.12512630438109765], [0.2505013419667203, 0.12529450974956163], [0.2505013419667203, 0.23210959027857653], [0.2505013419667203, 0.12538705732871663], [0.48904134698141433, 0.0065186740927575277], [0.48904134698141433, 0.12635786026070522], [0.48904134698141433, 0.12516127355117651], [0.48904134698141433, 0.12563251989931079], [0.48904134698141433, 0.046202680589333099], [0.48904134698141433, 0.010256669701494333]])), ([0.50694147330047323, 1.960772847954662e-07], ([0.5389452641563056, 0.0], [0.27099914253402557, 0.18549056986582946]), ([[0.50547730068989216, 0.00022280746607831103], [0.50054569207197552, 1.0152305856261749e-09], [0.5015954839184199, 1.0912247294494644e-07], [0.50103019916983604, 3.1899125592012232e-10], [0.50028043721507043, 1.216047444061167e-08], [0.50051503695474353, 2.5369025795627909e-08], [0.50035827524608012, 4.0484187711052896e-09], [0.50203741768221544, 7.9485213206717602e-08], [0.50107636328556071, 5.5272084257836017e-10], [0.50694147330047323, 1.960772847954662e-07]], [[0.5389452641563056, 0.0], [0.52221128242309889, 0.00011140373303915552], [0.51974547811414062, 5.0761529281308747e-10], [0.52027037403736276, 5.4561236472473221e-08], [0.51998773166307077, 1.5949562796006116e-10], [0.51961285068568808, 6.0802372203058348e-09], [0.51973015055552452, 1.2684512897813955e-08], [0.51965176970119287, 2.0242093855526448e-09], [0.50028043721507043, 3.9742606603358801e-08], [0.52001081372093316, 2.7636042128918009e-10], [0.52294336872838942, 9.8038642397733102e-08]])), ([0.023067456923691724, 0.47751631406354744], ([0.0, 0.0], [0.5416568755010757, 0.014742228829903836]), ([[0.25358542482666158, 0.24648193537990656], [0.0029628751761500917, 0.49742408117387904], [0.25226653117094189, 0.25079158575883531], [0.34903062531616175, 0.1543919515295957], [0.15665396587437741, 0.34539023151866999], [0.25108840502648666, 0.24917607522059418], [0.25361673086859882, 0.2502494856592159], [0.1754959820481993, 0.32796147818840071], [0.05900126522362259, 0.44201694737345887], [0.023067456923691724, 0.47751631406354744]], [[0.0, 0.0], [0.12679271241333079, 0.24648193537990656], [0.0014814375880750459, 0.49742408117387904], [0.12613326558547094, 0.49742408117387904], [0.17451531265808087, 0.25079158575883531], [0.078326982937188705, 0.49742408117387904], [0.12554420251324333, 0.34539023151866999], [0.12680836543429941, 0.34539023151866999], [0.08774799102409965, 0.34539023151866999], [0.029500632611811295, 0.49742408117387904], [0.011533728461845862, 0.49742408117387904]])), ([0.10998699289380238, 0.39247604508581407], ([0.0, 0.0], [0.27998496187645416, 0.6354816715136465]), ([[0.0033910741136679396, 0.49663966586422853], [0.25082086252648278, 0.24983705093220759], [0.40696939306532737, 0.093207813369487511], [0.25031798196834998, 0.251216787650555], [0.47444717275142023, 0.030342315065149741], [0.25109358778026364, 0.24919423984280525], [0.28688630501045775, 0.21325306538116973], [0.02214938155510067, 0.48066402067288511], [0.3556954620009794, 0.14818706618943375], [0.10998699289380238, 0.39247604508581407]], [[0.0, 0.0], [0.0016955370568339698, 0.24831983293211427], [0.25082086252648278, 0.1249185254661038], [0.40696939306532737, 0.046603906684743755], [0.40696939306532737, 0.1256083938252775], [0.47444717275142023, 0.030342315065149741], [0.40696939306532737, 0.12459711992140263], [0.40696939306532737, 0.10662653269058486], [0.47444717275142023, 0.030342315065149741], [0.40696939306532737, 0.074093533094716876], [0.28688630501045775, 0.19623802254290704]])), ([0.31424014133159495, 0.19658831847531585], ([0.8521054652075405, 0.5432635491282871], [0.42767271874282864, 0.7936297375374362]), ([[0.29537396504642921, 0.20648579041480178], [0.30686495229084465, 0.19889748124518267], [0.92402401175639104, 0.081165282617717757], [0.28714795669037407, 0.21653119836503976], [0.31424014133159495, 0.19658831847531585]], [[0.8521054652075405, 0.5432635491282871], [0.29537396504642921, 0.20648579041480178], [0.29537396504642921, 0.3710805151867349], [0.29537396504642921, 0.31221441587300247], [0.92402401175639104, 0.081165282617717757]])), ([0.2790909043520064, 0.22170167411427349], ([0.2295261844565868, 0.17176071551877525], [0.08450556232576112, 0.4839103516658896]), ([[0.27911056904373438, 0.22260673144740975], [0.29005774158055819, 0.2109612443953065], [0.27872498980152188, 0.222041472562233], [0.28403620531942708, 0.21650709114328012], [0.27964775808260128, 0.22450954836684694], [0.2877596220292391, 0.21237039067967076], [0.28094164068527722, 0.21928058345545626], [0.27942837748238719, 0.2217385947588528], [0.29191058850207813, 0.20982398623594339], [0.2790909043520064, 0.22170167411427349]], [[0.2295261844565868, 0.17176071551877525], [0.27911056904373438, 0.19718372348309249], [0.29005774158055819, 0.19136097995704088], [0.29005774158055819, 0.19690109404050413], [0.29005774158055819, 0.2109612443953065], [0.29005774158055819, 0.1981351319428111], [0.29005774158055819, 0.19206555309922302], [0.29005774158055819, 0.19552064948711576], [0.29005774158055819, 0.19674965513881404], [0.29191058850207813, 0.20982398623594339], [0.29191058850207813, 0.19673119481652437]])), ([7.4356562129297999e-09, 0.50013234618902602], ([0.0, 0.9893548997434237], [0.6816948434270595, 0.44279385986131736]), ([[5.4575024719771964e-16, 0.50389117818069218], [3.6776871719949849e-08, 0.50082220116899157], [1.9191135533070891e-08, 0.50583019639761795], [1.1022568844756448e-09, 0.50005744761885662], [1.386699447062071e-09, 0.50013003708284853], [3.4905638479517626e-08, 0.50084992831976372], [9.384789391659839e-09, 0.50240338342531254], [3.3929010509702647e-14, 0.50023389911248883], [1.3302657902945029e-10, 0.50452435104409987], [7.4356562129297999e-09, 0.50013234618902602]], [[0.0, 0.9893548997434237], [2.7287512359885982e-16, 0.50389117818069218], [3.6776871719949849e-08, 0.50082220116899157], [9.5955677665354454e-09, 0.50082220116899157], [5.5112844223782239e-10, 0.50005744761885662], [6.9334972353103549e-10, 0.50005744761885662], [1.7452819239758813e-08, 0.50005744761885662], [4.6923946958299195e-09, 0.50005744761885662], [1.6964505254851323e-14, 0.50005744761885662], [6.6513289514725147e-11, 0.50005744761885662], [3.7178281064649e-09, 0.50005744761885662]])), ([0.13659679838671693, 0.36452169290522352], ([0.21536704667589168, 0.875015209332337], [0.7692750757807316, 0.8142107572247883]), ([[0.13762813970773108, 0.36494882270643803], [0.099866609663894726, 0.40078994717548982], [0.12461828254027704, 0.37564744279941059], [0.0023715189721161987, 0.50599159932967452], [0.13193352204700989, 0.36847428605418941], [0.060113171076150135, 0.44301922934534044], [0.12714642450769426, 0.37790474113956873], [0.13665111534423022, 0.36526736613700317], [0.1277596742093092, 0.37399155393727557], [0.13659679838671693, 0.36452169290522352]], [[0.21536704667589168, 0.875015209332337], [0.13762813970773108, 0.61998201601938752], [0.099866609663894726, 0.63790257825391339], [0.099866609663894726, 0.62533132606587383], [0.0023715189721161987, 0.69050340433100577], [0.0023715189721161987, 0.62174474769326316], [0.13762813970773108, 0.36494882270643803], [0.0023715189721161987, 0.62645997523595287], [0.0023715189721161987, 0.62014128773467014], [0.0023715189721161987, 0.62450338163480623], [0.0023715189721161987, 0.61976845111878021]]))], (3, 'rNSGAII_solution', 'DTLZ4'): [([0.5817738291398401, 0.0, 0.81335127810695684], ([0.4981520234633334, 0.0, 0.6987592904740502], [0.2687599231193156, 0.5874227599950453, 0.849032094613711]), ([[0.58024450411455952, 8.0361095060064288e-313, 0.81444273404431544], [0.57733808817187426, 0.0, 0.81650661284548465], [0.57896925110815234, 0.0, 0.81535014672673223], [0.58079743180437859, 3.9927018012215411e-187, 0.81404904168263348], [0.57991440450998311, 0.0, 0.81467934020640131], [0.57789847060037969, 1.7079056019297989e-298, 0.81610938244372122], [0.58149165722736595, 0.0, 0.81355361992250796], [0.58067581303788784, 0.0, 0.81413544449285424], [0.58061700710698039, 1.4667846743608071e-204, 0.8141778677695467], [0.5817738291398401, 0.0, 0.81335127810695684]], [[0.4981520234633334, 0.0, 0.6987592904740502], [0.58024450411455952, 4.0180547530032144e-313, 0.75660101225918286], [0.58024450411455952, 0.0, 0.75763295165976741], [0.58024450411455952, 0.0, 0.7570547186003912], [0.58079743180437859, 1.9963509006107705e-187, 0.75640416607834182], [0.58079743180437859, 0.0, 0.75671931534022574], [0.58079743180437859, 8.5395280096489944e-299, 0.75743433645888569], [0.58149165722736595, 0.0, 0.75615645519827912], [0.58149165722736595, 0.0, 0.7564473674834522], [0.58149165722736595, 7.3339233718040354e-205, 0.75646857912179843], [0.5817738291398401, 0.0, 0.7560552842905035]])), ([3.9826563079261665e-05, 1.1351834416257644e-117, 1.0000020580335927], ([0.0, 0.0, 0.9829301429354526], [0.9425149515857719, 0.88945072845414, 0.8556143026989477]), ([[5.4524513403680844e-05, 0.0, 1.0000005054713772], [0.00012375018286132943, 0.0, 1.0000006356070077], [5.8372062143802531e-06, 2.8334320089475567e-173, 1.0000007351294984], [4.7747626311243626e-05, 0.0, 1.000002188433851], [9.5576420829258167e-05, 4.9406564584124654e-324, 1.0000006825753383], [0.00011105961503419872, 0.0, 1.0000020366721964], [5.1888851793268521e-08, 5.9356395366356397e-167, 1.0000011482274693], [6.4602356148474779e-06, 0.0, 1.0000021737996001], [0.00010506550704508325, 5.7692789275771517e-314, 1.0000029091166547], [3.9826563079261665e-05, 1.1351834416257644e-117, 1.0000020580335927]], [[0.0, 0.0, 0.9829301429354526], [2.7262256701840422e-05, 0.0, 1.0], [6.1875091430664715e-05, 0.0, 1.0], [2.9186031071901265e-06, 2.8334320089475567e-173, 1.0], [2.3873813155621813e-05, 2.8334320089475567e-173, 1.0], [4.7788210414629084e-05, 0.0, 1.0], [5.5529807517099361e-05, 0.0, 1.0], [2.5944425896634261e-08, 2.9678197683178199e-167, 1.0], [3.2301178074237389e-06, 0.0, 1.0], [5.2532753522541623e-05, 0.0, 1.0], [1.9913281539630832e-05, 5.6759172081288218e-118, 1.0]])), ([0.18579589368513613, 0.98259042362343996, 0.0], ([0.0, 0.0, 0.0], [0.3476628807788893, 0.2580358841179641, 0.9741421664144768]), ([[0.30287413618348025, 0.0, 0.95303180421575695], [0.19984287326101338, 0.0, 0.97983125918637526], [0.024895333804731867, 0.99969085962273707, 5.8902225646089654e-270], [0.22876231395966715, 5.7613718607838971e-180, 0.97348715291654031], [0.28279995726458562, 4.0540000743092897e-260, 0.95918088179363226], [0.25316565412474162, 0.96742431810099527, 0.0], [0.18579589368513613, 0.98259042362343996, 0.0]], [[0.0, 0.0, 0.0], [0.30287413618348025, 0.0, 0.47651590210787848], [0.30287413618348025, 0.0, 0.48991562959318763], [0.024895333804731867, 0.99969085962273707, 2.9451112823044827e-270], [0.30287413618348025, 2.8806859303919485e-180, 0.48674357645827016], [0.30287413618348025, 2.0270000371546449e-260, 0.47959044089681613], [0.30287413618348025, 0.96742431810099527, 0.0]])), ([0.77507521660820222, 0.63187009210686429, 3.4557406721173415e-310], ([0.6635589568190845, 0.5338175168551759, 0.0], [0.8704559541553162, 0.18504480075912777, 0.8589391277482235]), ([[0.77444462940906023, 0.63264368775670166, 1.4292617033530572e-248], [0.87408979764478467, 0.0, 0.48576760660351237], [0.7614345810000126, 0.64824266608712255, 0.0], [1.0000018744103765, 0.0, 0.0018665381088999108], [0.76724675507576434, 0.64136608315331689, 2.34255362160966e-132], [0.79006608538361567, 0.61302208492847388, 6.4313913162728633e-225], [0.75690652248880885, 0.65352364581286027, 0.0], [0.90578789362944734, 0.0, 0.43496916345072562], [1.0000001064818729, 4.0019373382880011e-262, 0.0], [0.77507521660820222, 0.63187009210686429, 3.4557406721173415e-310]], [[0.6635589568190845, 0.5338175168551759, 0.0], [0.71900179311407242, 0.63264368775670166, 7.1463085167652859e-249], [0.76882437723193453, 0.63264368775670166, 1.4292617033530572e-248], [0.77444462940906023, 0.63264368775670166, 0.0], [0.83178041561473048, 0.0, 0.48576760660351237], [0.71540285594742437, 0.64824266608712255, 1.17127681080483e-132], [0.72681252110135008, 0.64824266608712255, 3.2156956581364316e-225], [0.71023273965394673, 0.65352364581286027, 0.0], [0.78467342522426597, 0.61302208492847388, 6.4313913162728633e-225], [0.83177953165047869, 0.61302208492847388, 0.0], [0.71931708671364336, 0.65352364581286027, 1.7278703360586954e-310]])), ([1.0000000982332176, 0.0, 0.0], ([0.0, 0.0, 0.2524577834161418], [0.8243632471779416, 0.30876649829826963, 0.9095031967019501]), ([[0.00015279432279922432, 0.0, 1.0000026045903847], [0.00053927993236317997, 0.0, 1.0000004261958075], [0.0017746927482904073, 0.0, 0.99999961705020279], [0.00065583505356602478, 1.2217924897532153e-315, 1.0000048470877587], [0.80142508380152866, 0.59809635721026611, 0.0], [0.0017889128390616528, 1.5960930796075693e-227, 1.0000002881302146], [0.00029034305986072823, 0.0, 1.0000033595393782], [0.00020780589952572686, 9.3387857941110171e-258, 1.0000014371635142], [0.0033359400980513165, 0.0, 0.99999856521339736], [1.0000000982332176, 0.0, 0.0]], [[0.0, 0.0, 0.2524577834161418], [7.6397161399612161e-05, 0.0, 0.6262301940032633], [0.00026963996618158999, 0.0, 1.0], [0.00088734637414520365, 0.0, 0.62622870023317234], [0.00032791752678301239, 6.1089624240627943e-316, 0.6262313152519503], [0.80142508380152866, 0.59809635721026611, 0.0], [0.00089445641953082638, 7.9804653980378467e-228, 0.62622903577317812], [0.00014517152993036411, 0.0, 0.62623057147776007], [0.00010390294976286343, 4.6693928970555085e-258, 0.62622961028982793], [0.0016679700490256583, 0.0, 0.62622817431476951], [1.0, 0.0, 0.12622889170807089]])), ([0.049526905297417524, 0.99877804378048374, 0.0], ([0.0, 0.35648910413162704, 0.027090592880264497], [0.3156326515491916, 0.1106635552700761, 0.5168407789080482]), ([[0.0011823822116216332, 0.99944451376091858, 0.033456238138702819], [0.40791228572103427, 0.91302681361967852, 0.0], [0.049526905297417524, 0.99877804378048374, 0.0]], [[0.0, 0.35648910413162704, 0.027090592880264497], [0.00059119110581081662, 0.99944451376091858, 0.030273415509483658], [0.40791228572103427, 0.91302681361967852, 0.013545296440132248]])), ([0.8405567598255077, 0.0, 0.54172592785038509], ([0.8328179786838068, 0.0, 0.5305746793573836], [0.6341368207852265, 0.07636543423036812, 0.82238215799998]), ([[0.84146257244063838, 0.0, 0.54031929755482822], [0.84055209910029138, 0.0, 0.54174394847669249], [0.84156233266826097, 5.7180707565806363e-272, 0.54016206698461922], [0.84120997765040539, 0.0, 0.5407105059141879], [0.84116739080279279, 0.0, 0.54077797883775625], [0.84110387103746698, 3.2083062106042854e-163, 0.54087414115877142], [0.84137257094524742, 0.0, 0.54045716659070364], [0.84089440014605521, 0.0, 0.54120032852796118], [0.84278977347943007, 0.0, 0.53824471041708999], [0.8405567598255077, 0.0, 0.54172592785038509]], [[0.8328179786838068, 0.0, 0.5305746793573836], [0.84146257244063838, 0.0, 0.53544698845610594], [0.83668503889204904, 0.0, 0.53615931391703797], [0.83719015567603394, 2.8590353782903182e-272, 0.53536837317100139], [0.8370139781671061, 0.0, 0.53564259263578573], [0.8369926847432998, 0.0, 0.5356763290975699], [0.83696092486063689, 1.6041531053021427e-163, 0.53572441025807749], [0.84156233266826097, 0.0, 0.5355159229740436], [0.84156233266826097, 0.0, 0.53588750394267237], [0.83780387608161844, 0.0, 0.53440969488723677], [0.83668736925465725, 0.0, 0.53615030360388438]])), ([0.012112423613261425, 0.99992742561915893, 0.0], ([0.006631727076633065, 0.5551551402005014, 0.0], [0.37980684763043315, 0.05989220652418403, 0.7355834179488067]), ([[0.012022038429080957, 0.99994771179082176, 4.2436467149874535e-81], [0.66776680167850466, 0.0, 0.7443718286797798], [0.012303729558507568, 0.99992470544632339, 0.0], [0.01161379690718657, 0.99993565997650102, 0.0], [0.012023461399647224, 0.99992843933810538, 0.0], [0.012197734849829851, 0.99992699962950049, 0.0], [0.012239994876448486, 0.99992825701507515, 0.0], [0.012112423613261425, 0.99992742561915893, 0.0]], [[0.006631727076633065, 0.5551551402005014, 0.0], [0.009326882752857011, 0.99994771179082176, 2.1218233574937268e-81], [0.66776680167850466, 0.99994771179082176, 0.3721859143398899], [0.0094677283175703154, 0.99994771179082176, 0.0], [0.0091227619919098182, 0.99993565997650102, 0.0], [0.0093275942381401446, 0.99993565997650102, 0.0], [0.66776680167850466, 0.99992470544632339, 0.0], [0.012303729558507568, 0.99992470544632339, 0.0]])), ([0.00045631046010141571, 1.2980999434799788e-210, 1.0000019681346177], ([0.0, 0.0, 0.8177116892877433], [0.6358123042782293, 0.5201175297773143, 0.5798741732217337]), ([[4.3940650135908637e-11, 0.00030852406438040695, 1.0000046145523986], [0.00046505744713496523, 0.0, 1.000002073374515], [0.00042652719479138854, 1.6820969126520869e-276, 1.0000020492421557], [6.6425388838969662e-07, 0.0, 1.0000039197290438], [0.00023820214878407839, 0.0, 1.0000012749325236], [0.0001331385068759404, 4.7602872677364568e-176, 1.0000015795239601], [0.89061080953536442, 0.45477046693574202, 0.0], [0.00040039112200082524, 0.0, 1.0000016303018722], [0.00010204729504401714, 0.0, 1.0000013435276678], [0.00045631046010141571, 1.2980999434799788e-210, 1.0000019681346177]], [[0.0, 0.0, 0.8177116892877433], [2.1970325067954318e-11, 0.00015426203219020347, 1.0], [0.00023252872356748262, 0.0, 1.0], [0.00021326359739569427, 8.4104845632604343e-277, 1.0], [3.3212694419484831e-07, 0.0, 1.0], [0.0001191010743920392, 0.0, 1.0], [6.6569253437970201e-05, 2.3801436338682284e-176, 1.0], [0.44530540476768221, 0.45477046693574202, 0.0], [0.00020019556100041262, 0.0, 1.0], [5.1023647522008569e-05, 0.45477046693574202, 0.0], [0.00022815523005070785, 6.4904997173998938e-211, 1.0]])), ([1.1131637125883072e-11, 0.78899918527276014, 0.61439654167606506], ([0.0, 0.6790416599958826, 0.5290286140740371], [0.24925336613995874, 0.01742467458724206, 0.21027689679733552]), ([[1.4280390887988529e-11, 0.79040575558717041, 0.6125852624070226], [9.4498822878132727e-12, 0.78304085398511636, 0.62198260045500486], [9.6289026009193926e-14, 0.78805436926958139, 0.6156073389567267], [7.720749741644366e-12, 0.7891547635660231, 0.61419492743076565], [0.82826554635866434, 0.0, 0.56034102627950222], [6.7009599331022481e-13, 0.78420681087405342, 0.6205052636651619], [2.0625658259954898e-12, 0.78320022811388013, 0.62177479957516235], [1.7736155761358979e-10, 0.78887665832104348, 0.61455443496183226], [1.0293210149806075e-12, 0.78700046037128402, 0.61695565807087016], [1.1131637125883072e-11, 0.78899918527276014, 0.61439654167606506]], [[0.0, 0.6790416599958826, 0.5290286140740371], [7.1401954439942645e-12, 0.79040575558717041, 0.6125852624070226], [4.7249411439066363e-12, 0.79040575558717041, 0.6125852624070226], [4.8144513004596963e-14, 0.79040575558717041, 0.6125852624070226], [3.860374870822183e-12, 0.79040575558717041, 0.6125852624070226], [0.41413277317933217, 0.0, 0.62198260045500486], [3.3504799665511241e-13, 0.0, 0.56034102627950222], [1.0312829129977449e-12, 0.79040575558717041, 0.6125852624070226], [8.8680778806794896e-11, 0.0, 0.62198260045500486], [5.1466050749030375e-13, 0.79040575558717041, 0.6125852624070226], [5.5658185629415361e-12, 0.0, 0.56034102627950222]]))], (4, 'rNSGAII_solution', 'DTLZ3'): [([1.5771978491655923e-07, 2.0641064813584318e-09, 4.4275512693850958, 9.5493575296012321e-10], ([0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [0.2689052881646564, 0.02931604321256609, 0.9628672367001413, 0.7836866298559854]), ([[4.5966260465976005e-08, 4.2745468494567653, 0.8202582929755311, 4.755057513926994e-11], [0.017062402309007667, 0.0015157077265705503, 2.8244960076438947, 3.470461474486315e-12], [3.854937951642222, 1.1626188759428421e-10, 6.0549422860559368, 1.7907874283512811e-05], [9.9073774576398542e-12, 4.0262807495483885e-15, 5.2153625822990097, 0.31257212689593816], [0.049246778142995412, 2.1982842361500547, 0.087755683343417046, 1.967957509832706e-10], [0.023595546833961329, 1.0397680547198058e-08, 5.1828708675499557, 7.8297390841418682e-09], [4.3626989078522991, 0.0011941795460187455, 0.0058789527952426666, 1.2635449580350137e-11], [3.9224094719233129, 0.00027385628716905969, 1.3897586042793022, 6.5762188822383042e-10], [1.2633038490286629e-06, 2.2206713407535439e-09, 1.1758553048440685, 0.00012068216578243419], [1.5771978491655923e-07, 2.0641064813584318e-09, 4.4275512693850958, 9.5493575296012321e-10]], [[0.06992195127287737, 0.0, 0.6936931166589081, 0.0], [4.5966260465976005e-08, 1.0, 0.75697570481721965, 2.377528756963497e-11], [0.017062402309007667, 0.00075785386328527514, 1.0, 1.7352307372431575e-12], [4.5966260465976005e-08, 5.8130943797142103e-11, 1.0, 8.9539371417564054e-06], [1.0, 2.0131403747741942e-15, 1.0, 1.7907874283512811e-05], [0.017062402309007667, 1.0, 0.3907244000011626, 9.83978754916353e-11], [0.023595546833961329, 5.1988402735990289e-09, 1.0, 3.9148695420709341e-09], [0.017062402309007667, 0.00059708977300937274, 0.34978603472707537, 6.3177247901750684e-12], [1.0, 0.00013692814358452985, 1.0, 3.2881094411191521e-10], [1.0, 1.1103356703767719e-09, 0.93477421075148825, 6.0341082891217094e-05], [1.2633038490286629e-06, 1.0320532406792159e-09, 1.0, 4.7746787648006161e-10]])), ([2.5695965285641114, 2.5672253616242027, 0.005837589113562875, 2.4123833745979401e-05], ([0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [0.08232852832673387, 0.4637536567277941, 0.45795251910701495, 0.4473418936732124]), ([[4.1303298359288441, 4.0984936994997066e-07, 5.1097586414687742e-08, 7.35646344392241e-05], [4.611806290268623, 6.1180202986626975e-09, 1.7809274466995648e-07, 0.011889189793831346], [3.6310031899309863, 0.00012518113590548395, 2.2957706363969606, 0.071676388926755277], [0.63121468779574141, 2.0000723804609453, 0.0051276068744798703, 0.43213513697521871], [2.9908378468996633, 0.00047090555810090613, 1.7014504305232308, 0.00087805379425190866], [0.10742711892501645, 2.8037934490274017, 3.8556506248623522e-10, 0.055081987587741209], [6.1499198656215208, 0.0011078812501180608, 0.49677095341653654, 0.01038841593500653], [0.0045546773262231067, 2.0733818688037622, 3.2500155144173852e-05, 0.025875467292898444], [2.1617247979441565, 0.049149372497294813, 0.14109823248800665, 0.11397975826154665], [2.5695965285641114, 2.5672253616242027, 0.005837589113562875, 2.4123833745979401e-05]], [[0.27607561333361563, 0.2915219226816358, 0.366837119878929, 0.33880911017747906], [1.0, 0.14576116626550287, 5.1097586414687742e-08, 7.35646344392241e-05], [1.0, 0.14576096439982805, 5.1097586414687742e-08, 7.35646344392241e-05], [1.0, 0.14582355190877064, 1.0, 0.071676388926755277], [1.0, 1.0, 5.1097586414687742e-08, 0.43213513697521871], [0.63121468779574141, 0.14599641411986836, 5.1097586414687742e-08, 0.43213513697521871], [1.0, 1.0, 5.1097586414687742e-08, 0.00087805379425190866], [1.0, 0.14631490196587693, 1.0, 0.43213513697521871], [1.0, 1.0, 5.1097586414687742e-08, 7.35646344392241e-05], [1.0, 0.17033564758946529, 0.14109823248800665, 0.11397975826154665], [1.0, 1.0, 0.005837589113562875, 0.43213513697521871]])), ([1.9928563250971219, 4.3146817967606881e-06, 0.65293578815816877, 0.74890257595900522], ([0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.49410459464023737, 0.0961636375717283, 0.14559382999071668, 0.8718958420416848]), ([[0.90494228184157888, 0.36856203836223889, 1.5440282432728816, 1.1316038898075786], [0.083520738915939724, 2.0366616025059625, 1.1372212729308584, 0.87430357035330031], [2.104821378828236e-06, 1.7083329768515114, 1.5650784820013737, 0.43584683074644859], [0.015687853326977225, 0.81305766524595979, 4.0427843990999648, 0.025574418394314822], [0.38097409665468301, 0.19716916197397155, 0.77470543542193537, 0.85260833097359201], [0.80055522113736999, 0.15314134857844966, 1.0306110658647101, 0.53330316601511385], [2.4435351105856378, 0.35489815640342209, 1.3401753395309481, 1.1404020894668296], [0.39554537970975773, 2.7505855041231277, 6.4507632227707301, 1.4570435323730258], [1.3696258039225269, 0.37314465454699891, 1.1768515316330765, 1.2745037385050855], [1.9928563250971219, 4.3146817967606881e-06, 0.65293578815816877, 0.74890257595900522]], [[0.5487266917589726, 0.3475330227478646, 0.9184840628486147, 0.8300315486612688], [0.90494228184157888, 0.36856203836223889, 1.0, 0.98081771923442373], [0.90494228184157888, 0.36856203836223889, 1.0, 0.85216755950728462], [2.104821378828236e-06, 1.0, 1.0, 0.63293918970385876], [0.083520738915939724, 1.0, 1.0, 0.42780298352779184], [0.38097409665468301, 0.19716916197397155, 0.77470543542193537, 0.84131993981743047], [0.80055522113736999, 0.15314134857844966, 1.0, 0.68166735733819128], [0.80055522113736999, 0.15314134857844966, 1.0, 0.53330316601511385], [0.38097409665468301, 0.19716916197397155, 1.0, 1.0], [0.38097409665468301, 0.19716916197397155, 0.77470543542193537, 1.0], [1.0, 4.3146817967606881e-06, 0.65293578815816877, 0.78946706231013697]])), ([1.1850171941792194, 6.1470492611989291, 7.4218599453568905e-06, 6.206526179990407e-10], ([0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [0.7797606329258145, 0.49301085370218456, 0.905527378935073, 0.8169719449549107]), ([[4.0828479339264119, 0.13769516819916428, 0.0082359534085695452, 3.5444739241489606e-08], [4.1487191075542329, 4.6407573774222532, 3.9858345464775161e-07, 3.6627464786925398e-10], [0.18019646076666476, 3.9340061520551015, 1.9731187556440082, 2.0793192139289592e-05], [3.825083508501488e-05, 3.1822054629189944, 0.0069101412279861316, 8.8227892218401538e-14], [3.0704182133397837, 0.020801438223329874, 0.0013639951598165367, 0.0082162912639850635], [0.00036114538912801415, 5.104183727884509, 0.6395256822748322, 4.1027373700596857e-10], [1.0225252043795441e-06, 1.0083884106568283e-05, 4.1102277577578263, 0.047110050260930736], [1.2339794948755185e-13, 2.2555946925036147e-11, 8.839369709504858, 0.0003145481320376253], [3.2787900599879933e-08, 2.9718856305450614, 6.8434180517543366, 1.485221593501825e-08], [1.1850171941792194, 6.1470492611989291, 7.4218599453568905e-06, 6.206526179990407e-10]], [[0.011038966494290814, 0.4034562762106457, 0.0, 0.0], [1.0, 0.13769516819916428, 0.0041179767042847726, 1.7722369620744803e-08], [1.0, 1.0, 1.992917273238758e-07, 1.8313732393462699e-10], [1.0, 0.13769516819916428, 0.98655937782200409, 1.0396596069644796e-05], [1.0, 1.0, 0.0034550706139930658, 4.4113946109200769e-14], [3.825083508501488e-05, 1.0, 0.00068199757990826837, 0.0041081456319925318], [1.0, 1.0, 0.3197628411374161, 2.0513686850298429e-10], [1.0225252043795441e-06, 1.0, 1.0, 8.8227892218401538e-14], [1.2339794948755185e-13, 1.0, 0.0082359534085695452, 0.047110050260930736], [1.2339794948755185e-13, 1.0, 1.0, 7.426107967509125e-09], [1.0, 1.0, 3.7109299726784452e-06, 3.1032630899952035e-10]])), ([1.1602808514001308, 6.0678791254282176, 0.0031537669556460273, 0.02168929071418986], ([0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.7568260093734565, 0.058663247129592255, 0.5360980722499572, 0.3522236546519115]), ([[0.1020049715566848, 3.12722507767046, 1.1040869114254062e-06, 0.90830865138983696], [7.8852665862806228e-05, 8.1285296771959068, 1.9683463338298577e-05, 3.0137639054080674e-11], [2.5778189226278131e-06, 4.1782538485448804, 1.0793538768466091, 0.08206726926322018], [1.3146434866831107, 2.1435424898648612, 2.4220578220590449e-07, 1.1097946085386308e-08], [0.038001045140743518, 0.0026999527757003306, 6.611512103367164, 6.4058899739435232e-14], [0.67441294193654922, 4.0257376391264819, 0.00061858674071615623, 0.00034123683002077804], [2.2716981346896901e-14, 2.0545263940144224e-11, 1.0779305474242085, 5.1000032333124246], [0.52579412621259136, 4.1602798318270731, 1.6760673309699586e-08, 8.1090594330722549e-09], [0.0012236437844758985, 3.1435145838611986, 1.2992411575045588e-09, 8.712099900457186e-08], [1.1602808514001308, 6.0678791254282176, 0.0031537669556460273, 0.02168929071418986]], [[0.39433376319991775, 0.9773165541766571, 0.0, 0.0], [0.24816936737830128, 1.0, 5.520434557127031e-07, 0.90830865138983696], [0.19720630793289029, 1.0, 9.8417316691492884e-06, 1.5068819527040337e-11], [0.1971681705094202, 1.0, 1.1040869114254062e-06, 0.90830865138983696], [0.85448862494151423, 1.0, 1.2110289110295225e-07, 5.5489730426931541e-09], [0.21616740417033065, 1.0, 1.0, 3.2029449869717616e-14], [0.53437335256823348, 1.0, 0.00030929337035807812, 0.00017061841501038902], [0.19716688159997023, 1.0, 1.0, 6.4058899739435232e-14], [0.46006394470625456, 1.0, 8.3803366548497932e-09, 4.0545297165361275e-09], [0.19777870349219684, 1.0, 6.4962057875227939e-10, 4.356049950228593e-08], [0.77730730730002429, 1.0, 0.0015768834778230136, 0.00034123683002077804]])), ([7.1639207048607267e-09, 2.3392795402034463, 3.4184005720550754, 1.1528435054243949e-09], ([0.0, 0.0, 0.9612129216540486, 0.0], [0.5380057632321813, 0.6103821439491545, 0.9698265003481557, 0.13908762837505262]), ([[8.5646787692285233, 0.27015444231698094, 0.017754981956242533, 4.1311757247764911e-08], [0.0061363179708423723, 0.025577604777291631, 4.1964526017502743, 9.2198420949558949e-10], [0.14405149553912761, 0.0002784409454752309, 7.4541212041452667, 2.1691511167514163e-11], [2.8041360769020782e-11, 1.6901900570463153e-12, 1.1023722354845671, 1.5675044206760525e-06], [2.5024087399027785e-07, 1.6738239339713068e-09, 5.1437505485195114, 1.5665347091122585e-07], [0.0010838748583125812, 2.0158684766328747, 2.5343417857074573, 0.0013586927051621512], [0.17547230200925903, 9.8980987240667227e-09, 5.3230569492957889, 1.9397967319403656e-10], [6.5386347125739697e-11, 2.3438523364366845e-08, 1.1862801769088303, 0.017408327473104116], [1.546691868514033e-07, 0.015323715442176368, 3.0350251047122745, 5.3152066546513835e-10], [7.1639207048607267e-09, 2.3392795402034463, 3.4184005720550754, 1.1528435054243949e-09]], [[0.0, 0.0, 0.9612129216540486, 0.0], [1.0, 0.27015444231698094, 0.48948395180514559, 2.0655878623882455e-08], [0.0030681589854211861, 0.025577604777291631, 1.0, 4.6099210474779475e-10], [1.0, 0.27015444231698094, 1.0, 1.0845755583757082e-11], [1.4020680384510391e-11, 8.4509502852315767e-13, 1.0, 7.8375221033802627e-07], [1.2512043699513893e-07, 8.3691196698565339e-10, 1.0, 1.5675044206760525e-06], [0.00054193742915629062, 1.6738239339713068e-09, 1.0, 0.00067934635258107561], [0.0061363179708423723, 4.9490493620333613e-09, 1.0, 9.6989836597018281e-11], [3.2693173562869848e-11, 1.1719261682183423e-08, 1.0, 1.5675044206760525e-06], [1.546691868514033e-07, 0.015323715442176368, 1.0, 2.6576033273256917e-10], [3.5819603524303634e-09, 1.6901900570463153e-12, 1.0, 5.7642175271219743e-10]])), ([4.6334975043007177e-05, 5.6244385551117846, 0.33944599683508075, 1.1036763595713627e-08], ([0.0, 0.770734170045047, 0.0, 0.08920513181533896], [0.09379908248267677, 0.30324743360311346, 0.11522667312160806, 0.7309859064726725]), ([[1.2470113419558313e-09, 6.1448316041159803, 0.00038427079534229916, 6.6331925364396723e-11], [2.3838432664508865e-08, 7.2420679492657491, 0.0085695484235335188, 0.215677052316222], [5.6594745586936205e-07, 4.7006005914892297, 0.036524200319454331, 0.0053683566165111255], [6.7063239473252282e-05, 3.3007481562822654, 1.4427301870314081e-07, 5.1334488109038572e-11], [5.4595843876799426e-09, 1.1604578409231912, 2.5405862236610544e-07, 1.7839520615512765], [2.2862968749500536e-05, 4.1918673044486487, 6.668977151505047e-09, 6.0834387982248222e-07], [2.0626543532552169e-05, 4.1646415299060306, 0.66294370777802314, 1.0005560728174759e-07], [7.0321247510726327e-06, 5.0111300863976469, 6.2012840854248336e-09, 0.45246008477226463], [0.00087477415487205326, 2.6728372356618011, 2.9831578717423675e-08, 0.077153524113364555], [4.6334975043007177e-05, 5.6244385551117846, 0.33944599683508075, 1.1036763595713627e-08]], [[0.0, 0.770734170045047, 0.0, 0.08920513181533896], [6.2350567097791566e-10, 1.0, 0.00019213539767114958, 0.044602565940835441], [1.1919216332254432e-08, 1.0, 0.0042847742117667594, 0.15244109206578049], [2.8297372793468102e-07, 1.0, 0.036524200319454331, 0.04728674421592504], [3.3531619736626141e-05, 1.0, 7.2136509351570407e-08, 0.044602565933336724], [2.7297921938399713e-09, 1.0, 1.2702931118305272e-07, 0.93657859668330767], [1.1431484374750268e-05, 1.0, 3.3344885757525235e-09, 0.044602870079609391], [1.0313271766276085e-05, 1.0, 0.036524200319454331, 0.044602615935473121], [3.5160623755363164e-06, 1.0, 3.1006420427124168e-09, 0.27083260829380179], [0.00043738707743602663, 1.0, 1.4915789358711837e-08, 0.083179327964351762], [2.3167487521503589e-05, 1.0, 2.5405862236610544e-07, 0.044602571426051278]])), ([6.4283626764309237e-13, 1.2807294176674937e-11, 3.050588696085498, 8.0038491097814554e-09], ([0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.9579492639712833, 0.281577740048896, 0.5774982584802901, 0.581223711616129]), ([[4.981788525055125e-06, 0.0030061734756227591, 2.4132487762838903, 4.9991613353313821e-08], [6.4149085325454559e-05, 5.3021302041353957, 8.8773949634939656e-11, 9.6269425013733807e-13], [0.6798096937589293, 0.019405073862902834, 1.9741863587632524, 2.5625771594623331e-06], [0.029571752407378176, 0.008920867437578868, 1.1340010072036217, 5.8486297955232751e-13], [2.131593831367903, 0.17841962703256503, 0.0072174494447021535, 2.1819603237711376e-11], [0.57155427979341422, 1.4597957472467364e-08, 5.3004840946162526, 6.1497688766811379e-09], [1.9961887790440389, 0.94477861656766926, 2.5490175655216787e-05, 4.6113399920596832e-06], [5.5703981345983173, 4.6140453148231882e-11, 7.8999074010311687e-06, 0.0045036570532635345], [0.040836737871227199, 1.6624448456351869e-09, 0.0018567662827837657, 10.915794107506034], [6.4283626764309237e-13, 1.2807294176674937e-11, 3.050588696085498, 8.0038491097814554e-09]], [[0.2024010399029822, 0.0, 0.3600672196127662, 0.0], [0.10120301084575363, 0.0015030867378113795, 1.0, 2.4995806676656911e-08], [0.10123259449415382, 1.0, 1.0, 4.8134712506866904e-13], [0.44110536683095575, 1.0, 8.8773949634939656e-11, 1.2812885797311666e-06], [0.11598639615518018, 0.004460433718789434, 1.0, 2.9243148977616375e-13], [1.0, 0.019405073862902834, 1.0, 1.0909801618855688e-11], [0.38697765984819821, 0.008920867437578868, 1.0, 3.0748844383405689e-09], [1.0, 0.94477861656766926, 2.5490175655216787e-05, 2.3056699960298416e-06], [1.0, 2.3070226574115941e-11, 1.0, 0.0022518285266317673], [0.1216188888871047, 8.3122242281759345e-10, 1.0, 1.0], [0.10120051995181252, 6.4036470883374687e-12, 1.0, 4.0019245548907277e-09]])), ([6.8399500212881823e-09, 4.3654591181728852, 1.4765231065218453e-06, 0.00076548351146561147], ([0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [0.01622838457298137, 0.34885419310905763, 0.12707877388129385, 0.3364065122491783]), ([[7.515382597293837e-10, 6.758505102300119, 2.1949284893178449, 2.0137081494222965e-07], [5.0811911698137058, 2.0001178822345534e-07, 1.2770176651776819e-06, 0.11189509814350378], [0.18355136893742052, 0.76212695525204888, 8.336194923817116, 7.7402948909544733e-07], [3.8965205276349932, 0.0001374009422263473, 1.5953206202726082, 2.3241524425974143e-05], [4.0441543551364152, 3.6488036888729013e-06, 0.15726214813401698, 6.89101823549015e-06], [0.018602358029192599, 5.0798320390906033, 8.289448119638868e-07, 4.374232643677985e-06], [1.1841790319341964e-09, 6.9034513518544965e-09, 3.7552095321149674, 1.9397025261943426], [3.0806373620463456, 8.5290186531977125e-07, 1.1364110308139039e-05, 1.6131245291963763e-07], [1.4455261355744227e-07, 3.0925106824886655, 5.9913514579329031e-05, 0.0022645443175661768], [6.8399500212881823e-09, 4.3654591181728852, 1.4765231065218453e-06, 0.00076548351146561147]], [[0.0, 0.003861587163144997, 0.0, 0.07681819822993874], [3.7576912986469185e-10, 1.0, 1.0, 2.0137081494222965e-07], [1.0, 0.0019308935874666102, 6.3850883258884096e-07, 0.11189509814350378], [7.515382597293837e-10, 0.38299427120759694, 1.0, 0.11189509814350378], [7.515382597293837e-10, 0.0019994940526856723, 1.0, 2.3241524425974143e-05], [1.0, 0.0019326179834169351, 0.15726214813401698, 6.89101823549015e-06], [0.18355136893742052, 1.0, 4.144724059819434e-07, 7.7402948909544733e-07], [5.9208951596709819e-10, 0.0019307970332981745, 1.0, 7.7402948909544733e-07], [1.0, 0.0019312200325051585, 5.6820551540695193e-06, 2.3241524425974143e-05], [7.2276306778721133e-08, 1.0, 1.0, 1.0], [3.4199750106440911e-09, 1.0, 7.3826155326092264e-07, 4.374232643677985e-06]])), ([1.6497028376153073e-05, 0.83332722943653481, 4.3589028671479007, 2.5423362684171837], ([0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.6576097182225741, 0.43292741593780293, 0.586852033033112, 0.09639159167313893]), ([[0.81711499242265029, 9.3878578910439021, 0.088423987063250858, 1.1771070139168938e-09], [7.7675152928078762e-09, 5.1055878491340323, 5.7249498026427037e-05, 1.2761569988277818e-07], [3.5222599416253429e-07, 3.1847810526592526, 5.7081787649546265e-07, 0.11535698110580518], [2.3870053832668999e-09, 5.9980911770098171, 6.3835070544275885e-06, 0.82585253504929745], [2.3272738738150118, 3.7263986416306327, 1.2715320700412486e-10, 0.036253295927258157], [1.1739217262442146e-09, 5.015517521311156, 0.00018568823846836993, 0.71189638605877414], [2.3602149839593996e-06, 8.1414746437926553, 3.4281574906791619e-05, 0.016080193794779811], [0.35815654277444214, 4.4123259725401338, 0.0014005950300158062, 1.1521810671539297e-05], [0.000352799885872349, 6.2970529681838672, 1.0336720369797108e-10, 0.097688851247401301], [1.6497028376153073e-05, 0.83332722943653481, 4.3589028671479007, 2.5423362684171837]], [[0.0, 0.5958026145468293, 0.0, 0.42920361775657356], [0.40855749621132514, 1.0, 0.088423987063250858, 1.1771070139168938e-09], [3.8837576464039381e-09, 1.0, 2.8624749013213519e-05, 1.2761569988277818e-07], [1.7611299708126715e-07, 1.0, 2.8540893824773133e-07, 0.11535698110580518], [1.1935026916334499e-09, 1.0, 3.1917535272137943e-06, 0.82585253504929745], [1.0, 1.0, 6.3576603502062429e-11, 1.1771070139168938e-09], [5.8696086312210731e-10, 1.0, 9.2844119234184965e-05, 0.11535698110580518], [1.1801074919796998e-06, 1.0, 1.714078745339581e-05, 0.11535698110580518], [0.17907827138722107, 1.0, 0.00070029751500790308, 0.11535698110580518], [0.0001763999429361745, 1.0, 5.168360184898554e-11, 0.097688851247401301], [8.2485141880765363e-06, 1.0, 0.088423987063250858, 1.1771070139168938e-09]]))]}

In [2]:
%store -r POs
%store -r results
POs=prj_ref(results,POs,evals=20000)

In [ ]:
from joblib import Parallel, delayed
from random import random
from scipy import spatial
import itertools

import pyMOEA



nf=3

methods=[pyMOEA.ACH_solution,pyMOEA.rNSGAII_solution]
problems=['DTLZ1','DTLZ2','DTLZ3','DTLZ4']
pref=random_pref(nf)

res_nf3=[]
for r in range(10):
    res=Parallel(n_jobs=8)(
       delayed(pyMOEA.agent_paraller)(target,bounds,pref,evals=20000) 
            for target in itertools.product(methods,problems)         
    )
    res_nf3.append(res)
%store pref
%store res_nf3


In [ ]:
res_nf2,pref_nf2=agent_run(2)
%store res_nf2
%store pref_nf2
res_nf4,pref_nf4=agent_run(4)
%store res_nf4
%store pref_nf4


In [ ]:
res_nf4=[res_nf4]
for i in range(9):
    res=pyMOEA.agent_run(nf=4,pref=pref_nf4)
    res_nf4.append(res)
sa

In [ ]:
%store -r res_nf3
%store -r pref


    

In [ ]:
res_nf2=[[0.30572319801095654, 1.193770738302777], [0.1802256464189552, 0.9836642452220091], [4.410216554030713, 2.3515399340829757], [0.5673285232611179, 0.8245258392191691], [0.15303183211261273, 0.34749906239401018], [0.1578600562781062, 0.98746214986383785], [3.0684266508487421, 4.2810229899932406e-08], [0.49768539655825667, 0.86771258088093572]]
pref_nf2=([0.19081049089320778, 0.7065239411942785], [0.7244251409337009, None])
res_nf3=[[[0.3719363248759723, 0.12806317610524182, 5.00000000995593e-07], [0.9985371187313212, 0.05468315741801278, 1.7894886310718836e-06], [0.570046608369235, 0.5699948075701542, 2.8887436217056432], [0.994017526773867, 0.10985498461750158, 3.5943781094714556e-54], [0.20994612497014231, 0.17888638726374706, 0.17492724013438596], [0.67807004870620158, 0.68432885983982805, 0.26970914760822862], [4.6438451798367604, 4.989159887164119, 0.0047433087707516603], [0.71769952165776418, 0.69753156951624673, 0.0]], [[1.1663354969214497, 1.1663387201303304, 0.16636008024761578], [0.03906075930526216, 0.999264110870513, 1.677367976094315e-06], [3.5854414817368086e-06, 3.055524406519828e-06, 2.998987300395458], [0.9992695407115165, 0.03883243956444546, 6.435251260132861e-81], [0.99989907542519851, 3.2316884663068272e-11, 4.7912457958505572e-07], [0.67636613854520922, 0.65689262962916461, 0.34301579858622894], [9.9431307903491337, 1.8276336960373358, 0.0082961682116783099], [0.70970875987133974, 0.70633291539488563, 0.0]], [[0.08539889379558611, 0.4146006062166707, 5.000000000266347e-07], [0.7022789523175088, 0.7022789472373772, 0.13391468908857448], [1.692409500915607, 1.6923258337157445, 4.387715995955346], [0.6412969447902299, 0.6412967906348869, 0.4311956731819669], [0.2516539601590233, 0.2563315871505446, 1.5114590321441656e-13], [0.71416384866358207, 0.70062139233377529, 0.017294644279722042], [2.9315405063317472, 0.63016436478400928, 0.19309127663810677], [0.7371988633005635, 0.67570691770825886, 0.0]], [[0.3060054601588664, 0.19399403985361244, 5.000000000268567e-07], [0.7077061372942114, 0.7077061322768978, 0.002738032830053965], [0.11252606545960209, 0.11362413971171692, 3.995287156185288], [0.998333151757849, 0.058237555363221755, 4.884948988385351e-31], [0.2575037905193937, 0.24770363148176952, 5.6089291211720938e-17], [0.70111078776200286, 0.72691423405305555, 0.025841236642865698], [1.2859308548633583, 5.1896091729132312, 0.30726859540403278], [0.69016990634455844, 0.72365086534075529, 0.0]], [[0.5281524985877286, 0.4715933269833277, 9.9974682534663e-07], [0.377265128503263, 0.9618632851947828, 1.6229526306848501e-06], [1.9910921104046044, 1.9909659954537429, 4.129363620386241], [0.9970361623918355, 0.07790262560184386, 3.0124121671577128e-62], [0.24462760783147591, 0.26479063371302164, 2.5676780886076226e-14], [0.70285898120820678, 0.72564883875031838, 4.0130308546203307e-05], [0.79544550228897337, 7.2007555323205583, 0.064197445249304028], [0.70734320815074347, 0.71104105362526748, 0.0]], [[0.12961824795679794, 0.8701275842523195, 9.99746831984698e-07], [0.7080172429538057, 0.7080172401327299, 0.0006696764869984781], [7.378153107744089, 7.378153089805778, 7.743886172725715], [0.6779960113307388, 0.6779954271931176, 0.2872531575743491], [0.7601003554473369, 0.74936141957948932, 5.6978533392495798e-15], [0.70682147701570641, 0.69218171513326343, 0.17549628362328765], [3.1180151893786814, 1.1913932685249287, 0.00068232781676388031], [0.68845916123275785, 0.72527966321737825, 0.0]], [[0.9661915130997896, 0.9661913861646345, 0.06685835061703142], [0.03893991084225833, 0.9992694239598783, 1.6438706922260456e-06], [2.031479708640614, 2.0315456005027492, 4.089768681735271], [0.9398131775374391, 0.3564715042448807, 1.0215645841944924e-50], [0.27765351262940302, 0.24759635685253251, 2.9157224939010177e-16], [0.7246257230916886, 0.69513299462300704, 0.0052385649397643404], [1.5030295707674495, 0.25887645572547263, 8.3107679418142979], [0.73197218676014519, 0.68309144714064851, 3.8132532352547794e-184]], [[0.6275580668818962, 0.8719340838119977, 1.499493650230663e-06], [0.9812991678017775, 0.22610669552543636, 1.5818100692511017e-06], [1.096984230518186, 1.097730490950283, 5.793206453274836], [0.994130173517859, 0.11023328050785978, 8.085534291763287e-61], [1.1964394552126174, 1.1960277766828864e-11, 8.055962572689341e-12], [0.64452422370836193, 0.67099973739762431, 0.38305651480632741], [3.8276985847304315, 2.2464326611927317, 0.16829481618360065], [0.65379897151044253, 0.7566692415266959, 0.0]], [[0.2327333418028574, 0.7670124835499736, 9.997468251284045e-07], [0.9241307236703897, 0.45231342664514473, 1.6161693032155178e-06], [1.566231443302353, 1.5636680004580863, 4.48126010124995], [0.9855150941194872, 0.21461612076179581, 8.796563799037188e-44], [0.49025382816612878, 0.51538896536241452, 1.5016760693776694e-13], [0.67327480861096134, 0.67811157608867634, 0.31630716078962762], [2.9173013115341901, 1.1589028384492566, 0.12907214066486311], [1.0001487733125354, 6.6560480088221962e-06, 0.0]], [[0.5495394516899639, 0.4502063736643771, 9.997468251299144e-07], [0.9620528086377651, 0.2874424490266273, 1.57719901334597e-06], [1.6803678919345122, 1.702652081380885, 6.575328150352381], [0.9992670219720744, 0.03901457192624638, 5.261890291265693e-13], [0.27002827089871972, 0.24710137864178561, 1.297169241261232e-10], [0.7237414436511308, 0.61271500867194739, 0.42796306841073528], [3.9636818420215842, 1.0008479129640513, 0.41163082317757477], [0.92201041395001448, 0.38716898479256606, 0.0]]]
pref_nf3=([0.4776904546967242, 0.0, 0.0], [None, None, 0.7839132004035231])
res_nf4=[[3.2290565380045934e-08, 0.03229053308948055, 0.6461241516833925, 0.8210789333782578], [4.2613683817557627e-07, 0.00015536421451806313, 1.0000003385762453, 0.010996829449094242], [2.9989874927728333, 4.710798537755277e-06, 4.710798537761089e-06, 4.7107985377669e-06], [2.7985551999549708e-06, 3.612738331240162e-93, 0.015798575131059967, 0.9999936328669745], [2.1224752017014162e-10, 0.29047547177703364, 0.24255090459136538, 5.3260034041111837e-16], [1.6934942501385586e-16, 0.59782283053130369, 0.66543936958073535, 0.49981247131422429], [0.8838482995190029, 1.3850138325302046, 1.4412737884015405, 0.79104821350601917], [0.14137412541664443, 0.63601324011266347, 0.58161603813759066, 0.62723891241772955]]
pref_nf4=([0.0, 0.8744634420810271, 0.6773821110504221, 0.0], [0.7622471305931988, None, None, 0.16613673766247128])
%store res_nf2 
%store pref_nf2
%store res_nf3
%store pref_nf3
%store res_nf4
%store pref_nf4

In [ ]:
print_results(res_nf2,pref_nf2)

In [ ]:
nf3_multiple=agent_run(single_pref=False,runs=2,evals=20,nf=3)

In [ ]:

res=[]
for i,obj in enumerate(["Two","Three","Four"]):
    print "%s objectives"%obj
    nf=i+2
    ref=[0.5]*nf
    for method in ['DTLZ1','DTLZ2','DTLZ3','DTLZ4','DTLZ7']:
        print method
        res_.append(make_run(method,ref,nf,runs=10))
        a=np.array(res[-1][-1])
        print np.std(a,axis=0),np.mean(a,axis=0),np.min(a,axis=0)
    print ""


In [ ]:
import numpy  as np
%store -r res
for r in res:
    a=np.array(r[-1])
    print np.mean(a,axis=0),np.min(a,axis=0),np.std(a,axis=0)

In [ ]:
import time
from math import sqrt

import joblib

def my_sqrt(x):
    time.sleep(0.1)
    return sqrt(x)

joblib.Parallel(n_jobs=2, verbose=11)(joblib.delayed(my_sqrt)(i**2) for i in range(100))

In [ ]:
%qtconsole

In [ ]:
%store -r
pref_nf3=pref
%load_ext ipycache
%%cache single_10_runs.pkl res_nf2 pref_nf2 res_nf3 res_nf4 pref_nf4
%qtconsole


In [2]:
%store -r results

24


In [21]:
for i in range(7):
    for nf in [5,6]:
        res=agent_run(nf=nf,runs=1,single_pref=False)
        for k in res.keys():
            results[(nf,k[0].__name__,k[1])].extend(res[k])
        print "======================================================================="
        print nf
        %store results
import pickle
pickle.dump(results,open("results.dmp","w"))

5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)
5
Stored 'results' (defaultdict)
6
Stored 'results' (defaultdict)


In [22]:
print len(results)

40


In [7]:
%store -r POs
%store POs > POs.txt
print len(POs)

Writing 'POs' (dict) to file 'POs.txt'.
36


In [13]:
prj_ref(results,POs,evals=20000)

(2, 'DTLZ1', (0.8521054652075405, 0.5432635491282871))
(2, 'DTLZ1', (0.2295261844565868, 0.17176071551877525))
(2, 'DTLZ1', (0.0, 0.9893548997434237))
(2, 'DTLZ1', (0.21536704667589168, 0.875015209332337))
(2, 'DTLZ2', (0.8521054652075405, 0.5432635491282871))
(2, 'DTLZ2', (0.2295261844565868, 0.17176071551877525))
(2, 'DTLZ2', (0.0, 0.9893548997434237))
(2, 'DTLZ2', (0.21536704667589168, 0.875015209332337))
(2, 'DTLZ1', (0.8521054652075405, 0.5432635491282871))
(2, 'DTLZ1', (0.2295261844565868, 0.17176071551877525))
(2, 'DTLZ1', (0.0, 0.9893548997434237))
(2, 'DTLZ1', (0.21536704667589168, 0.875015209332337))
(2, 'DTLZ2', (0.8521054652075405, 0.5432635491282871))
(2, 'DTLZ2', (0.2295261844565868, 0.17176071551877525))
(2, 'DTLZ2', (0.0, 0.9893548997434237))
(2, 'DTLZ2', (0.21536704667589168, 0.875015209332337))
(3, 'DTLZ1', (0.0, 0.0, 0.2524577834161418))
(3, 'DTLZ1', (0.0, 0.35648910413162704, 0.027090592880264497))
(3, 'DTLZ1', (0.8328179786838068, 0.0, 0.5305746793573836))
(3, 'DTL

{(2, 'DTLZ1', (0.0, 0.0)): [0.9996202295774317, 0.999620247411252],
 (2, 'DTLZ1', (0.5389452641563056, 0.0)): [0.7693460434444864,
  0.2304007853878284],
 (2, 'DTLZ1', (0.5858121009812931, 0.0)): [1.0426530846225495,
  0.45684098391572636],
 (2, 'DTLZ1', (0.8233781347997915, 0.0)): [1.6611827132258348,
  0.8378045895518204],
 (2, 'DTLZ2', (0.0, 0.0)): [0.711152421836636, 0.7111524202065502],
 (2, 'DTLZ2', (0.5389452641563056, 0.0)): [0.9232191471779909,
  0.3842738690370552],
 (2, 'DTLZ2', (0.5858121009812931, 0.0)): [0.9364944662355161,
  0.3506823570517402],
 (2, 'DTLZ2', (0.8233781347997915, 0.0)): [0.9879163787777763,
  0.16453822895069764],
 (2, 'DTLZ3', (0.0, 0.0)): [2.827353169668456, 2.8273531548130486],
 (2, 'DTLZ3', (0.0, 0.9893548997434237)): [0.8298162914636574,
  1.8191711911990718],
 (2, 'DTLZ3', (0.21536704667589168, 0.875015209332337)): [2.4782257262467136,
  3.1378738673119755],
 (2, 'DTLZ3', (0.2295261844565868, 0.17176071551877525)): [3.5628664922742868,
  3.50510099

In [10]:
import pickle
pickle.dump(POs,open("POs.dmp","w"))
pickle.dump(results,open("results.dmp","w"))